In [1]:
# Data processing
import os
import pickle
import json
import pandas as pd
import numpy as np
import re

# Viewing
from IPython.display import display

In [3]:
latex_table_header = '''

\\begin{{table*}}[ht]

\\setlength\\tabcolsep{{2pt}} % Makes table columns tighter
\\renewcommand{{\\arraystretch}}{{0.9}}% Makes table rows tighter
\\caption{{{caption}}}
\\vskip -0.1in
\\label{{{label}}}
\\begin{{center}}
\\begin{{small}}
\\begin{{sc}}
\\resizebox{{\\textwidth}}{{!}}{{ %Completely zooms in or zooms out (shrinks) entire table!

'''

latex_table_footer = '''}}

\\end{{sc}}
\\end{{small}}
\\end{{center}}
\\vskip -0.1in
\\end{{table*}}


'''


def make_latex(results, str_info, display_latex = True, round_places = 2, multiclass = False):
    max_indicator = 9999
    if len(results):                       
        column_format="l|ccc|ccc|ccc|ccc|ccc|ccc|ccc"
        table = make_table(results, display_table = False)
        table = table.iloc[:, table.columns.get_level_values(1).isin([1,5,10])].round(round_places)
        # Bold max values
        table_copy = pd.DataFrame(table, copy = True).round(round_places)
        histories = [4, 5, 10, 30]
        futures = [1, 5, 10]
        for history in histories:
            for future in futures:
                grp = table_copy.loc[history][table_copy.columns.get_level_values(level=1) == future]
                max_indices = np.arange(len(grp.values))[grp.values == np.max(grp.values)]
                for max_idx in max_indices:
                    table_copy.loc[history, grp.index[max_idx]] += max_indicator
#                 last_max_idx = len(grp) - np.argmax(grp.values[::-1]) - 1
#                 table_copy.loc[history, grp.index[last_max_idx]] += max_indicator 
        tex = table_copy.to_latex(column_format = column_format) #, bold_rows = True)
        for w in [w for w in tex.split() if w.startswith(str(max_indicator))]:
            tex = tex.replace(w, "\\textbf{"+str(round(float(w) - max_indicator, round_places))+"}")
        tex = tex.replace("\\_", "-")
        for col in table.columns.get_level_values(0).unique():
            c = col.replace('_', '-')
            tex = tex.replace("{"+c+"}", "{{\\textbf{{{}}}}}".format(c))
        tex = tex.replace('voxaudio', 'audio')
        tex = tex.replace('\\textbf{text-video-audio}', '\\textbf{text-vid-aud}')
        # Bold columns
#         tex = tex.replace("1  &    5  &    10", "\\textbf{1}  &    \\textbf{5}  &    \\textbf{10}")
#         # Bold rows
#         for z in ['4  &', '5  &', '10 &', '30 &']:
#             i = int(z[:-2])
#             spaces = "  " if i < 10 else " "
#             tex = tex.replace(z, "\\textbf{"+str(i)+"}"+spaces+"&")
        tex = tex.replace("{", "{{")
        tex = tex.replace("}", "}}")
        tex = latex_table_header + tex + latex_table_footer
        # Prepare caption
        info = {z.split(":")[0]:z.split(":")[1].strip() for z in str_info.split(" | ")}
        if multiclass:
            pred = "which of person 1, 2, 3, or no one will be speaking." # using everyone's synchronized features.
        else:
            if info['predict_only_host'] == 'True' and info['use_all_perspectives'] == 'True':
                pred = "whether or not the host will be speaking in 1-10 seconds  using everyone's features."
            elif info['predict_only_host'] == 'False' and info['use_all_perspectives'] == 'False':
                pred = "whether or not a given person will be speaking in 1-10 seconds using only that person's features."
            elif info['predict_only_host'] == 'True' and info['use_all_perspectives'] == 'False':
                pred = "whether or not the host will be speaking in 1-10 seconds  given only the host's features."
        clf_name = 'Gaussian Naive Bayes' if 'NB' in info['clf'] else 'Random Forest'
        caption = "{} accuracy with {} ".format(
            'Cross-validation' if info['use_crossval'] == 'True' else 'Test', 
            clf_name,
        )
        caption += "predicting {}".format(pred)
        caption += " Rows reflect history used."
        caption += " Columns comprise features used and how far in the future we predict."
        caption += " Max score for each (past, future) pair is in bold."
        if multiclass:
            label = "table:{}_{}_{}".format(clf_name.replace(" ", "-"), 'multiclass', info['use_crossval'])
        else:   
            label = "table:{}_{}_{}_{}".format(clf_name.replace(" ", "-"), info['predict_only_host'], info['use_all_perspectives'], info['use_crossval'])
        tex = tex.format(
            caption = caption, 
            label = label,
        )
        # Center column headers
        tex = tex.replace("\multicolumn{3}{l}", "\multicolumn{3}{c}")
        # Add future and past column and row titles
        tex = tex.replace('\\toprule\n{} & \\multicolumn{3}{c}{\\textbf{text}}', '\\toprule\n\\multicolumn{4}{c}{Future (s),\\textbf{text}}')
        tex = tex.replace('\\\\\n{} &', '\\\\\n{Past (s)} &')
        if display_latex:
            print(tex)
        return tex
    else:
        return {}

In [4]:
def make_table(results, display_table = True):
    if len(results):
        table = pd.DataFrame({k:[i[1] for i in v] for k, v in results.items()}).T
        table.columns = [(k[0], k[1]+1) for k, v in list(results.items())[0][1]]
        table = table.T
        table.index = pd.MultiIndex.from_tuples(table.index)
        table = table.T
        if display_table:
            display(table.iloc[:, table.columns.get_level_values(1).isin([1,5,10])].round(3))
        return table
    else:
        return {}

In [1047]:
title = "LATEX TABLES FOR MULTICLASS TURN-TAKING"
all_results = {"use_crossval: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.3832786252172234),   (('text', 2), 0.35521983343017627),   (('text', 4), 0.31668933359238394),   (('text', 9), 0.29978460935970236),   (('video', 0), 0.32168372272639506),   (('video', 2), 0.3257795855122991),   (('video', 4), 0.32329512337283856),   (('video', 9), 0.3127080477775602),   (('voxaudio', 0), 0.4997103687970651),   (('voxaudio', 2), 0.43947317451094325),   (('voxaudio', 4), 0.41266757334369536),   (('voxaudio', 9), 0.3796749559428236),   (('text_video', 0), 0.35682564201583317),   (('text_video', 2), 0.33430176254115823),   (('text_video', 4), 0.3291237614144162),   (('text_video', 9), 0.31486195418053653),   (('text_voxaudio', 0), 0.45356246379609966),   (('text_voxaudio', 2), 0.4096455549099361),   (('text_voxaudio', 4), 0.3677870604235477),   (('text_voxaudio', 9), 0.33326806344233406),   (('video_voxaudio', 0), 0.41243483297933964),   (('video_voxaudio', 2), 0.36684098392407516),   (('video_voxaudio', 4), 0.3495239945599378),   (('video_voxaudio', 9), 0.33326806344233406),   (('text_video_voxaudio', 0), 0.4186136319752848),   (('text_video_voxaudio', 2), 0.3664536122409452),   (('text_video_voxaudio', 4), 0.35418690499319994),   (('text_video_voxaudio', 9), 0.3322890150773448)],  5: [(('text', 0), 0.37710084033613445),   (('text', 2), 0.31952247191011235),   (('text', 4), 0.31672535211267605),   (('text', 9), 0.29609929078014185),   (('video', 0), 0.3326330532212885),   (('video', 2), 0.32145365168539325),   (('video', 4), 0.3202464788732394),   (('video', 9), 0.3047872340425532),   (('voxaudio', 0), 0.48704481792717086),   (('voxaudio', 2), 0.4234550561797753),   (('voxaudio', 4), 0.4052816901408451),   (('voxaudio', 9), 0.37854609929078015),   (('text_video', 0), 0.35171568627450983),   (('text_video', 2), 0.32303370786516855),   (('text_video', 4), 0.3227112676056338),   (('text_video', 9), 0.30531914893617024),   (('text_voxaudio', 0), 0.4565826330532213),   (('text_voxaudio', 2), 0.38395365168539325),   (('text_voxaudio', 4), 0.3737676056338028),   (('text_voxaudio', 9), 0.3313829787234043),   (('video_voxaudio', 0), 0.39898459383753504),   (('video_voxaudio', 2), 0.35410814606741575),   (('video_voxaudio', 4), 0.34066901408450706),   (('video_voxaudio', 9), 0.32269503546099293),   (('text_video_voxaudio', 0), 0.40843837535014005),   (('text_video_voxaudio', 2), 0.35252808988764045),   (('text_video_voxaudio', 4), 0.3436619718309859),   (('text_video_voxaudio', 9), 0.3170212765957447)],  10: [(('text', 0), 0.42066290550070523),   (('text', 2), 0.39002828854314003),   (('text', 4), 0.3567375886524823),   (('text', 9), 0.34285714285714286),   (('video', 0), 0.317524682651622),   (('video', 2), 0.3150636492220651),   (('video', 4), 0.3122340425531915),   (('video', 9), 0.29428571428571426),   (('voxaudio', 0), 0.43600141043723556),   (('voxaudio', 2), 0.39745403111739747),   (('voxaudio', 4), 0.3881205673758865),   (('voxaudio', 9), 0.3630357142857143),   (('text_video', 0), 0.35895627644569816),   (('text_video', 2), 0.33716407355021216),   (('text_video', 4), 0.3351063829787234),   (('text_video', 9), 0.3023214285714286),   (('text_voxaudio', 0), 0.45081100141043723),   (('text_voxaudio', 2), 0.40664780763790664),   (('text_voxaudio', 4), 0.376063829787234),   (('text_voxaudio', 9), 0.36178571428571427),   (('video_voxaudio', 0), 0.37323695345557123),   (('video_voxaudio', 2), 0.35148514851485146),   (('video_voxaudio', 4), 0.3448581560283688),   (('video_voxaudio', 9), 0.31339285714285714),   (('text_video_voxaudio', 0), 0.39580394922425954),   (('text_video_voxaudio', 2), 0.3668670438472419),   (('text_video_voxaudio', 4), 0.3574468085106383),   (('text_video_voxaudio', 9), 0.32107142857142856)],  30: [(('text', 0), 0.3047895500725689),   (('text', 2), 0.28457059679767105),   (('text', 4), 0.25985401459854013),   (('text', 9), 0.240625),   (('video', 0), 0.2997097242380261),   (('video', 2), 0.30585880640465796),   (('video', 4), 0.29744525547445255),   (('video', 9), 0.30974264705882354),   (('voxaudio', 0), 0.376088534107402),   (('voxaudio', 2), 0.34042940320232895),   (('voxaudio', 4), 0.3235401459854015),   (('voxaudio', 9), 0.29466911764705883),   (('text_video', 0), 0.3205732946298984),   (('text_video', 2), 0.30622270742358076),   (('text_video', 4), 0.2945255474452555),   (('text_video', 9), 0.28860294117647056),   (('text_voxaudio', 0), 0.3447024673439768),   (('text_voxaudio', 2), 0.31240902474526927),   (('text_voxaudio', 4), 0.28686131386861313),   (('text_voxaudio', 9), 0.26029411764705884),   (('video_voxaudio', 0), 0.33327285921625543),   (('video_voxaudio', 2), 0.3220524017467249),   (('video_voxaudio', 4), 0.3156934306569343),   (('video_voxaudio', 9), 0.32169117647058826),   (('text_video_voxaudio', 0), 0.34379535558780844),   (('text_video_voxaudio', 2), 0.325509461426492),   (('text_video_voxaudio', 4), 0.31715328467153286),   (('text_video_voxaudio', 9), 0.31231617647058824)]}, "use_crossval: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.5352384630237498),   (('text', 2), 0.484601975595584),   (('text', 4), 0.47250825723722556),   (('text', 9), 0.4534952026630116),   (('video', 0), 0.4711334234408187),   (('video', 2), 0.4600038737168313),   (('video', 4), 0.4596852535457548),   (('video', 9), 0.4593694928529469),   (('voxaudio', 0), 0.5425757868314346),   (('voxaudio', 2), 0.4935115243075731),   (('voxaudio', 4), 0.4783368952788032),   (('voxaudio', 9), 0.46289406696690816),   (('text_video', 0), 0.5311836261826608),   (('text_video', 2), 0.48169668797210924),   (('text_video', 4), 0.46531960365261316),   (('text_video', 9), 0.46446054435089096),   (('text_voxaudio', 0), 0.5398725622707087),   (('text_voxaudio', 2), 0.49293046678287816),   (('text_voxaudio', 4), 0.47891975908296097),   (('text_voxaudio', 9), 0.46034854121793617),   (('video_voxaudio', 0), 0.5466306236725237),   (('video_voxaudio', 2), 0.49312415262444315),   (('video_voxaudio', 4), 0.4781426073440839),   (('video_voxaudio', 9), 0.4662228314078716),   (('text_video_voxaudio', 0), 0.5437343116431743),   (('text_video_voxaudio', 2), 0.49428626767383305),   (('text_video_voxaudio', 4), 0.4769768797357684),   (('text_video_voxaudio', 9), 0.46543959271588015)],  5: [(('text', 0), 0.5169817927170869),   (('text', 2), 0.47349016853932585),   (('text', 4), 0.4623239436619718),   (('text', 9), 0.45),   (('video', 0), 0.4564075630252101),   (('video', 2), 0.45189606741573035),   (('video', 4), 0.4508802816901408),   (('video', 9), 0.4482269503546099),   (('voxaudio', 0), 0.5274859943977591),   (('voxaudio', 2), 0.4845505617977528),   (('voxaudio', 4), 0.461443661971831),   (('voxaudio', 9), 0.45195035460992905),   (('text_video', 0), 0.5133053221288515),   (('text_video', 2), 0.4748946629213483),   (('text_video', 4), 0.45985915492957746),   (('text_video', 9), 0.44556737588652484),   (('text_voxaudio', 0), 0.5148809523809523),   (('text_voxaudio', 2), 0.4808637640449438),   (('text_voxaudio', 4), 0.4572183098591549),   (('text_voxaudio', 9), 0.45372340425531915),   (('video_voxaudio', 0), 0.5281862745098039),   (('video_voxaudio', 2), 0.48191713483146065),   (('video_voxaudio', 4), 0.46566901408450706),   (('video_voxaudio', 9), 0.45656028368794327),   (('text_video_voxaudio', 0), 0.5162815126050421),   (('text_video_voxaudio', 2), 0.4820926966292135),   (('text_video_voxaudio', 4), 0.4625),   (('text_video_voxaudio', 9), 0.4533687943262411)],  10: [(('text', 0), 0.4922425952045134),   (('text', 2), 0.4662305516265912),   (('text', 4), 0.4524822695035461),   (('text', 9), 0.4475),   (('video', 0), 0.4585684062059238),   (('video', 2), 0.4524398868458274),   (('video', 4), 0.44858156028368795),   (('video', 9), 0.44857142857142857),   (('voxaudio', 0), 0.4938293370944993),   (('voxaudio', 2), 0.4618104667609618),   (('voxaudio', 4), 0.4521276595744681),   (('voxaudio', 9), 0.45125),   (('text_video', 0), 0.48730606488011285),   (('text_video', 2), 0.46304809052333806),   (('text_video', 4), 0.46063829787234045),   (('text_video', 9), 0.44910714285714287),   (('text_voxaudio', 0), 0.492771509167842),   (('text_voxaudio', 2), 0.471004243281471),   (('text_voxaudio', 4), 0.45691489361702126),   (('text_voxaudio', 9), 0.44785714285714284),   (('video_voxaudio', 0), 0.4975317348377997),   (('video_voxaudio', 2), 0.46304809052333806),   (('video_voxaudio', 4), 0.45868794326241136),   (('video_voxaudio', 9), 0.45053571428571426),   (('text_video_voxaudio', 0), 0.49100846262341324),   (('text_video_voxaudio', 2), 0.474009900990099),   (('text_video_voxaudio', 4), 0.4615248226950355),   (('text_video_voxaudio', 9), 0.45267857142857143)],  30: [(('text', 0), 0.45555152394775034),   (('text', 2), 0.44759825327510916),   (('text', 4), 0.44653284671532845),   (('text', 9), 0.44283088235294116),   (('video', 0), 0.44992743105950656),   (('video', 2), 0.45014556040756915),   (('video', 4), 0.44908759124087594),   (('video', 9), 0.44632352941176473),   (('voxaudio', 0), 0.4589985486211901),   (('voxaudio', 2), 0.4455967976710335),   (('voxaudio', 4), 0.4364963503649635),   (('voxaudio', 9), 0.4391544117647059),   (('text_video', 0), 0.45500725689404936),   (('text_video', 2), 0.45141921397379914),   (('text_video', 4), 0.44908759124087594),   (('text_video', 9), 0.44779411764705884),   (('text_voxaudio', 0), 0.46407837445573297),   (('text_voxaudio', 2), 0.4532387190684134),   (('text_voxaudio', 4), 0.44215328467153286),   (('text_voxaudio', 9), 0.4400735294117647),   (('video_voxaudio', 0), 0.4617198838896952),   (('video_voxaudio', 2), 0.4534206695778748),   (('video_voxaudio', 4), 0.4531021897810219),   (('video_voxaudio', 9), 0.4514705882352941),   (('text_video_voxaudio', 0), 0.4591799709724238),   (('text_video_voxaudio', 2), 0.45160116448326054),   (('text_video_voxaudio', 4), 0.4489051094890511),   (('text_video_voxaudio', 9), 0.44871323529411766)]}, "use_crossval: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.47328011284199717),   (('text', 2), 0.42009985886675477),   (('text', 4), 0.3812053741139221),   (('text', 9), 0.34251248360361164),   (('video', 0), 0.3464497472000013),   (('video', 2), 0.33268281668722544),   (('video', 4), 0.3228921836876705),   (('video', 9), 0.31850701704356077),   (('voxaudio', 0), 0.5445880991652688),   (('voxaudio', 2), 0.4725381953334161),   (('voxaudio', 4), 0.4345332558323408),   (('voxaudio', 9), 0.38735920633411575),   (('text_video', 0), 0.38967112359829714),   (('text_video', 2), 0.34678927728022135),   (('text_video', 4), 0.3310851535801268),   (('text_video', 9), 0.31947950042514484),   (('text_voxaudio', 0), 0.5335030289222205),   (('text_voxaudio', 2), 0.46709845899963104),   (('text_voxaudio', 4), 0.42101793080629424),   (('text_voxaudio', 9), 0.3738104994021232),   (('video_voxaudio', 0), 0.4470651495953941),   (('video_voxaudio', 2), 0.38764695692106177),   (('video_voxaudio', 4), 0.35999387063220023),   (('video_voxaudio', 9), 0.33037734443405997),   (('text_video_voxaudio', 0), 0.4695703036201591),   (('text_video_voxaudio', 2), 0.39633915385770957),   (('text_video_voxaudio', 4), 0.365342852848258),   (('text_video_voxaudio', 9), 0.33016753281475925)],  5: [(('text', 0), 0.4694275984012341),   (('text', 2), 0.4088680252296566),   (('text', 4), 0.3740402683823255),   (('text', 9), 0.34411609770603074),   (('video', 0), 0.34577265446205263),   (('video', 2), 0.3296545204930509),   (('video', 4), 0.3192391946622637),   (('video', 9), 0.31076708068520736),   (('voxaudio', 0), 0.5331025811739564),   (('voxaudio', 2), 0.46475242502320785),   (('voxaudio', 4), 0.4298772689890303),   (('voxaudio', 9), 0.38663068193831895),   (('text_video', 0), 0.3853136707495169),   (('text_video', 2), 0.3472927063744698),   (('text_video', 4), 0.32666296572921105),   (('text_video', 9), 0.3129766104753921),   (('text_voxaudio', 0), 0.5223633170725014),   (('text_voxaudio', 2), 0.45148926173424575),   (('text_voxaudio', 4), 0.4153698870165961),   (('text_voxaudio', 9), 0.3693918838469124),   (('video_voxaudio', 0), 0.43282486141405246),   (('video_voxaudio', 2), 0.3792018999384052),   (('video_voxaudio', 4), 0.3540195975858112),   (('video_voxaudio', 9), 0.3282057805983774),   (('text_video_voxaudio', 0), 0.4511129205215879),   (('text_video_voxaudio', 2), 0.39184799169231715),   (('text_video_voxaudio', 4), 0.36141482866779545),   (('text_video_voxaudio', 9), 0.3286023166862545)],  10: [(('text', 0), 0.48314529386250504),   (('text', 2), 0.4380650790279713),   (('text', 4), 0.4060710700349247),   (('text', 9), 0.3703155520122249),   (('video', 0), 0.327641701557278),   (('video', 2), 0.3169740208373965),   (('video', 4), 0.3061972219070186),   (('video', 9), 0.2958140891851842),   (('voxaudio', 0), 0.47626043988104527),   (('voxaudio', 2), 0.4262958015041514),   (('voxaudio', 4), 0.4023648617380653),   (('voxaudio', 9), 0.3517443233679917),   (('text_video', 0), 0.37665658190512846),   (('text_video', 2), 0.34706323959179086),   (('text_video', 4), 0.33159555150302605),   (('text_video', 9), 0.307115367134048),   (('text_voxaudio', 0), 0.4942339667116188),   (('text_voxaudio', 2), 0.4431748716391306),   (('text_voxaudio', 4), 0.4069694171732873),   (('text_voxaudio', 9), 0.3700674572377843),   (('video_voxaudio', 0), 0.3889383796339288),   (('video_voxaudio', 2), 0.355979171423454),   (('video_voxaudio', 4), 0.3392263903365792),   (('video_voxaudio', 9), 0.3120805204463154),   (('text_video_voxaudio', 0), 0.41382555446545616),   (('text_video_voxaudio', 2), 0.37204333815537016),   (('text_video_voxaudio', 4), 0.35247053100954545),   (('text_video_voxaudio', 9), 0.32264183265229524)],  30: [(('text', 0), 0.3559453423250864),   (('text', 2), 0.3275266807004877),   (('text', 4), 0.29976649761033325),   (('text', 9), 0.2542077657694187),   (('video', 0), 0.2822727193586153),   (('video', 2), 0.27579688056332075),   (('video', 4), 0.2684245106778975),   (('video', 9), 0.26856811285776555),   (('voxaudio', 0), 0.35378725270335193),   (('voxaudio', 2), 0.3150438942848706),   (('voxaudio', 4), 0.29178484677345873),   (('voxaudio', 9), 0.24158111584529904),   (('text_video', 0), 0.31970336934358634),   (('text_video', 2), 0.297015250295548),   (('text_video', 4), 0.2854164820102498),   (('text_video', 9), 0.2696033712874484),   (('text_voxaudio', 0), 0.3834722907024821),   (('text_voxaudio', 2), 0.34203171641782854),   (('text_voxaudio', 4), 0.3068018104140496),   (('text_voxaudio', 9), 0.2507908587332751),   (('video_voxaudio', 0), 0.32089286906682724),   (('video_voxaudio', 2), 0.30511437700943955),   (('video_voxaudio', 4), 0.29248020676568737),   (('video_voxaudio', 9), 0.27993204956931683),   (('text_video_voxaudio', 0), 0.3407841476058759),   (('text_video_voxaudio', 2), 0.31679171339356965),   (('text_video_voxaudio', 4), 0.3013516424327839),   (('text_video_voxaudio', 9), 0.2783360110433907)]}, "use_crossval: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.5732740661626561),   (('text', 2), 0.5148554576188002),   (('text', 4), 0.4858821992092956),   (('text', 9), 0.4613597338376147),   (('video', 0), 0.41518871850379285),   (('video', 2), 0.3975344526712269),   (('video', 4), 0.3914305711126639),   (('video', 9), 0.3891138473660516),   (('voxaudio', 0), 0.5641648816061338),   (('voxaudio', 2), 0.4935944466880535),   (('voxaudio', 4), 0.4607048339016913),   (('voxaudio', 9), 0.4419803432790501),   (('text_video', 0), 0.542280057938123),   (('text_video', 2), 0.4779519889355003),   (('text_video', 4), 0.4485712235325316),   (('text_video', 9), 0.41343150247576715),   (('text_voxaudio', 0), 0.571950921069565),   (('text_voxaudio', 2), 0.503274872030984),   (('text_voxaudio', 4), 0.4721056212906121),   (('text_voxaudio', 9), 0.44570910394777197),   (('video_voxaudio', 0), 0.5490829530607771),   (('video_voxaudio', 2), 0.4709791000761349),   (('video_voxaudio', 4), 0.44275388099290486),   (('video_voxaudio', 9), 0.41377408201569843),   (('text_video_voxaudio', 0), 0.5543250010744407),   (('text_video_voxaudio', 2), 0.48464038036262486),   (('text_video_voxaudio', 4), 0.4515718793327529),   (('text_video_voxaudio', 9), 0.42367387516251825)],  5: [(('text', 0), 0.5649559830820807),   (('text', 2), 0.5114941233088991),   (('text', 4), 0.4879358750820031),   (('text', 9), 0.46249278283679995),   (('video', 0), 0.41076273730741664),   (('video', 2), 0.3968929247307587),   (('video', 4), 0.3948680368582404),   (('video', 9), 0.3903064982093396),   (('voxaudio', 0), 0.5499681735784),   (('voxaudio', 2), 0.49365959179458885),   (('voxaudio', 4), 0.46291344560095266),   (('voxaudio', 9), 0.4394223090127941),   (('text_video', 0), 0.521582693486374),   (('text_video', 2), 0.46696506079092315),   (('text_video', 4), 0.43982623582741054),   (('text_video', 9), 0.4101222723935223),   (('text_voxaudio', 0), 0.5568745661063044),   (('text_voxaudio', 2), 0.5016227145585722),   (('text_voxaudio', 4), 0.4712626016671543),   (('text_voxaudio', 9), 0.44148841351976864),   (('video_voxaudio', 0), 0.5215271210631907),   (('video_voxaudio', 2), 0.4583848627092671),   (('video_voxaudio', 4), 0.4293962715574975),   (('video_voxaudio', 9), 0.4027055328432499),   (('text_video_voxaudio', 0), 0.5320974071839031),   (('text_video_voxaudio', 2), 0.47582571263691964),   (('text_video_voxaudio', 4), 0.4437348535696559),   (('text_video_voxaudio', 9), 0.41244335429158197)],  10: [(('text', 0), 0.5196639885630715),   (('text', 2), 0.49060624385226514),   (('text', 4), 0.47104308432468256),   (('text', 9), 0.4508823779858192),   (('video', 0), 0.4026817032627658),   (('video', 2), 0.39362203223466696),   (('video', 4), 0.38844217465767106),   (('video', 9), 0.38122951534849187),   (('voxaudio', 0), 0.4937721971471539),   (('voxaudio', 2), 0.45711897854077793),   (('voxaudio', 4), 0.4401122018902086),   (('voxaudio', 9), 0.4227091879748029),   (('text_video', 0), 0.4798377894917083),   (('text_video', 2), 0.4446692078565043),   (('text_video', 4), 0.4253131821062361),   (('text_video', 9), 0.39777124669337444),   (('text_voxaudio', 0), 0.5026913053939792),   (('text_voxaudio', 2), 0.46516402642767807),   (('text_voxaudio', 4), 0.447551725713666),   (('text_voxaudio', 9), 0.4259186769540853),   (('video_voxaudio', 0), 0.47571652416632865),   (('video_voxaudio', 2), 0.4382279453523452),   (('video_voxaudio', 4), 0.4184191745922492),   (('video_voxaudio', 9), 0.3978549848557913),   (('text_video_voxaudio', 0), 0.48889160567001955),   (('text_video_voxaudio', 2), 0.4454296232026712),   (('text_video_voxaudio', 4), 0.4274389238478841),   (('text_video_voxaudio', 9), 0.40303966970341537)],  30: [(('text', 0), 0.43185854118009004),   (('text', 2), 0.41633860144186025),   (('text', 4), 0.41245190618788874),   (('text', 9), 0.4045238079976493),   (('video', 0), 0.373871009402539),   (('video', 2), 0.37103996846696835),   (('video', 4), 0.3695393249719361),   (('video', 9), 0.36869205646953396),   (('voxaudio', 0), 0.38164540892712173),   (('voxaudio', 2), 0.3697941990283121),   (('voxaudio', 4), 0.3627268528250846),   (('voxaudio', 9), 0.35259530589638177),   (('text_video', 0), 0.402892453115018),   (('text_video', 2), 0.3867669701935051),   (('text_video', 4), 0.3831660597561313),   (('text_video', 9), 0.3710707001881017),   (('text_voxaudio', 0), 0.3898343826837004),   (('text_voxaudio', 2), 0.37428784626670725),   (('text_voxaudio', 4), 0.3688730343469443),   (('text_voxaudio', 9), 0.3605171332444099),   (('video_voxaudio', 0), 0.3836371446113673),   (('video_voxaudio', 2), 0.36957153483592187),   (('video_voxaudio', 4), 0.3641448903577872),   (('video_voxaudio', 9), 0.3531824713348667),   (('text_video_voxaudio', 0), 0.39146653242941304),   (('text_video_voxaudio', 2), 0.3750638990136947),   (('text_video_voxaudio', 4), 0.36559093379145147),   (('text_video_voxaudio', 9), 0.3551704354357227)]}}
print(title + "\n" + "*"*len(title) + "\n")
for k, v in all_results.items():
    make_latex(v, k, multiclass = True)

LATEX TABLES FOR MULTICLASS TURN-TAKING
***************************************



\begin{table*}[t]

\setlength\tabcolsep{2pt} % Makes table columns tighter
\renewcommand{\arraystretch}{0.9}% Makes table rows tighter
\caption{Test accuracy with Gaussian Naive Bayes predicting which of person 1, 2, 3, or no one will be speaking. Rows reflect history used. Columns comprise features used and how far in the future we predict. Max score for each (past, future) pair is in bold.}
\vskip -0.1in
\label{table:Gaussian-Naive-Bayes_multiclass_False}
\begin{center}
\begin{small}
\begin{sc}
\resizebox{\textwidth}{!}{ %Completely zooms in or zooms out (shrinks) entire table!

\begin{tabular}{l|ccc|ccc|ccc|ccc|ccc|ccc|ccc}
\toprule
\multicolumn{4}{c}{Future (s),\textbf{text}} & \multicolumn{3}{c}{\textbf{video}} & \multicolumn{3}{c}{\textbf{audio}} & \multicolumn{3}{c}{\textbf{text-video}} & \multicolumn{3}{c}{\textbf{text-audio}} & \multicolumn{3}{c}{\textbf{video-audio}} & \multicolumn{3}{c}{\textb

In [1046]:
title = "TASK: PREDICTING BINARY WILL-BE-SPEAKING LABEL (2 classes: speaking or not)"
print(title + "\n" + "*"*len(title) + "\n")
all_results = {"use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6234794361845917),   (('text', 2), 0.5829943831105946),   (('text', 4), 0.5665436176413444),   (('text', 9), 0.539455649109066),   (('video', 0), 0.5514578103881058),   (('video', 2), 0.536897152818129),   (('video', 4), 0.534874684282106),   (('video', 9), 0.5337771685921284),   (('voxaudio', 0), 0.6481946321683723),   (('voxaudio', 2), 0.5953902769707534),   (('voxaudio', 4), 0.5686807849232562),   (('voxaudio', 9), 0.5402388878010573),   (('text_video', 0), 0.5945163158910987),   (('text_video', 2), 0.5614952546968817),   (('text_video', 4), 0.555469205362347),   (('text_video', 9), 0.5435676522420207),   (('text_voxaudio', 0), 0.6570766557250435),   (('text_voxaudio', 2), 0.5990703079604881),   (('text_voxaudio', 4), 0.5811152127452885),   (('text_voxaudio', 9), 0.5461131779909928),   (('video_voxaudio', 0), 0.6346785093647422),   (('video_voxaudio', 2), 0.5814448963780747),   (('video_voxaudio', 4), 0.5686807849232562),   (('video_voxaudio', 9), 0.5615821421578227),   (('text_video_voxaudio', 0), 0.6420158331724272),   (('text_video_voxaudio', 2), 0.590548130931629),   (('text_video_voxaudio', 4), 0.5772294540509034),   (('text_video_voxaudio', 9), 0.5611905228118269)],  5: [(('text', 0), 0.6183473389355743),   (('text', 2), 0.5660112359550562),   (('text', 4), 0.5521126760563381),   (('text', 9), 0.5132978723404256),   (('video', 0), 0.5689775910364145),   (('video', 2), 0.5626755617977528),   (('video', 4), 0.5619718309859155),   (('video', 9), 0.5606382978723404),   (('voxaudio', 0), 0.6411064425770309),   (('voxaudio', 2), 0.5935744382022472),   (('voxaudio', 4), 0.5732394366197183),   (('voxaudio', 9), 0.5452127659574468),   (('text_video', 0), 0.601890756302521),   (('text_video', 2), 0.5784761235955056),   (('text_video', 4), 0.5767605633802817),   (('text_video', 9), 0.5695035460992908),   (('text_voxaudio', 0), 0.6502100840336135),   (('text_voxaudio', 2), 0.598314606741573),   (('text_voxaudio', 4), 0.5765845070422535),   (('text_voxaudio', 9), 0.5427304964539007),   (('video_voxaudio', 0), 0.6260504201680672),   (('video_voxaudio', 2), 0.5953300561797753),   (('video_voxaudio', 4), 0.5802816901408451),   (('video_voxaudio', 9), 0.573936170212766),   (('text_video_voxaudio', 0), 0.6391806722689075),   (('text_video_voxaudio', 2), 0.6014747191011236),   (('text_video_voxaudio', 4), 0.5894366197183099),   (('text_video_voxaudio', 9), 0.573758865248227)],  10: [(('text', 0), 0.578984485190409),   (('text', 2), 0.5477369165487977),   (('text', 4), 0.5342198581560283),   (('text', 9), 0.5141071428571429),   (('video', 0), 0.5756346967559943),   (('video', 2), 0.570898161244696),   (('video', 4), 0.5656028368794326),   (('video', 9), 0.54625),   (('voxaudio', 0), 0.6126586741889986),   (('voxaudio', 2), 0.579031117397454),   (('voxaudio', 4), 0.5668439716312057),   (('voxaudio', 9), 0.55375),   (('text_video', 0), 0.6004936530324401),   (('text_video', 2), 0.5774398868458275),   (('text_video', 4), 0.574645390070922),   (('text_video', 9), 0.5510714285714285),   (('text_voxaudio', 0), 0.6140691114245416),   (('text_voxaudio', 2), 0.5756718528995757),   (('text_voxaudio', 4), 0.5641843971631205),   (('text_voxaudio', 9), 0.54625),   (('video_voxaudio', 0), 0.610543018335684),   (('video_voxaudio', 2), 0.5868104667609618),   (('video_voxaudio', 4), 0.5822695035460993),   (('video_voxaudio', 9), 0.5682142857142857),   (('text_video_voxaudio', 0), 0.6160084626234132),   (('text_video_voxaudio', 2), 0.5885785007072136),   (('text_video_voxaudio', 4), 0.5856382978723405),   (('text_video_voxaudio', 9), 0.5716071428571429)],  30: [(('text', 0), 0.5665820029027576),   (('text', 2), 0.5598617176128093),   (('text', 4), 0.5505474452554745),   (('text', 9), 0.5303308823529411),   (('video', 0), 0.5609579100145138),   (('video', 2), 0.5576783114992722),   (('video', 4), 0.5525547445255474),   (('video', 9), 0.546875),   (('voxaudio', 0), 0.5809143686502177),   (('voxaudio', 2), 0.5589519650655022),   (('voxaudio', 4), 0.5572992700729927),   (('voxaudio', 9), 0.5466911764705882),   (('text_video', 0), 0.5732946298984035),   (('text_video', 2), 0.5633187772925764),   (('text_video', 4), 0.5565693430656934),   (('text_video', 9), 0.5479779411764706),   (('text_voxaudio', 0), 0.5725689404934688),   (('text_voxaudio', 2), 0.5540393013100436),   (('text_voxaudio', 4), 0.5516423357664234),   (('text_voxaudio', 9), 0.5347426470588236),   (('video_voxaudio', 0), 0.5792815674891146),   (('video_voxaudio', 2), 0.5680494905385735),   (('video_voxaudio', 4), 0.5602189781021898),   (('video_voxaudio', 9), 0.5582720588235294),   (('text_video_voxaudio', 0), 0.5820029027576198),   (('text_video_voxaudio', 2), 0.5722343522561864),   (('text_video_voxaudio', 4), 0.5638686131386861),   (('text_video_voxaudio', 9), 0.5512867647058823)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6452951346011361),   (('text', 2), 0.5723270440251572),   (('text', 4), 0.5716697284146766),   (('text', 9), 0.551296786767278),   (('video', 0), 0.5657199308471228),   (('video', 2), 0.5463279354231665),   (('video', 4), 0.5419790477136189),   (('video', 9), 0.5399030533206737),   (('voxaudio', 0), 0.6749320819955544),   (('voxaudio', 2), 0.6355172584559006),   (('voxaudio', 4), 0.6221637455935654),   (('voxaudio', 9), 0.6089650692119334),   (('text_video', 0), 0.62904420844653),   (('text_video', 2), 0.5894616946466598),   (('text_video', 4), 0.5778759743806167),   (('text_video', 9), 0.5564939283394134),   (('text_voxaudio', 0), 0.6722647567300568),   (('text_voxaudio', 2), 0.6107066805328579),   (('text_voxaudio', 4), 0.6065240057593962),   (('text_voxaudio', 9), 0.5909749637699265),   (('video_voxaudio', 0), 0.6466287972338849),   (('video_voxaudio', 2), 0.6116476006536918),   (('video_voxaudio', 4), 0.5970408619234397),   (('video_voxaudio', 9), 0.576582879416321),   (('text_video_voxaudio', 0), 0.6637194368979995),   (('text_video_voxaudio', 2), 0.6155103253602734),   (('text_video_voxaudio', 4), 0.604885556824388),   (('text_video_voxaudio', 9), 0.5822797461396232)],  5: [(('text', 0), 0.6498554913294797),   (('text', 2), 0.6086536604210933),   (('text', 4), 0.5914988381099923),   (('text', 9), 0.5682061775309364),   (('video', 0), 0.5648843930635838),   (('video', 2), 0.5517191423604404),   (('video', 4), 0.5476374903175832),   (('video', 9), 0.5464289194192731),   (('voxaudio', 0), 0.6700385356454721),   (('voxaudio', 2), 0.6368553216148348),   (('voxaudio', 4), 0.6249515879163439),   (('voxaudio', 9), 0.6140017538731365),   (('text_video', 0), 0.6205202312138728),   (('text_video', 2), 0.5928626617732278),   (('text_video', 4), 0.5784759876065065),   (('text_video', 9), 0.5630907142161161),   (('text_voxaudio', 0), 0.6660404624277456),   (('text_voxaudio', 2), 0.6283079003283755),   (('text_voxaudio', 4), 0.6132358636715725),   (('text_voxaudio', 9), 0.591688590080873),   (('video_voxaudio', 0), 0.6435452793834296),   (('video_voxaudio', 2), 0.6117442534286266),   (('video_voxaudio', 4), 0.5979376452362509),   (('video_voxaudio', 9), 0.5822858813212511),   (('text_video_voxaudio', 0), 0.6624277456647398),   (('text_video_voxaudio', 2), 0.6268591848560943),   (('text_video_voxaudio', 4), 0.6103795507358637),   (('text_video_voxaudio', 9), 0.5895449673584722)],  10: [(('text', 0), 0.5935046049442559),   (('text', 2), 0.5626397123141219),   (('text', 4), 0.5499853843905291),   (('text', 9), 0.4525887428907629),   (('video', 0), 0.5523994183228308),   (('video', 2), 0.5373214112158616),   (('video', 4), 0.5342005261619409),   (('video', 9), 0.5295646205138262),   (('voxaudio', 0), 0.6484730974309258),   (('voxaudio', 2), 0.6339780347944407),   (('voxaudio', 4), 0.6300302055929066),   (('voxaudio', 9), 0.6314963718376152),   (('text_video', 0), 0.5888027144934561),   (('text_video', 2), 0.5645349402274273),   (('text_video', 4), 0.5576829387118776),   (('text_video', 9), 0.5395665816826829),   (('text_voxaudio', 0), 0.6113427047988367),   (('text_voxaudio', 2), 0.5836816017105647),   (('text_voxaudio', 4), 0.5723959855792653),   (('text_voxaudio', 9), 0.5500098058442832),   (('video_voxaudio', 0), 0.6206010664081435),   (('video_voxaudio', 2), 0.6008358441053553),   (('video_voxaudio', 4), 0.5911039657020365),   (('video_voxaudio', 9), 0.5808491861149245),   (('text_video_voxaudio', 0), 0.6283082888996607),   (('text_video_voxaudio', 2), 0.600155505880066),   (('text_video_voxaudio', 4), 0.5913475591932184),   (('text_video_voxaudio', 9), 0.5694744067464209)],  30: [(('text', 0), 0.4972650422675286),   (('text', 2), 0.4849935187954931),   (('text', 4), 0.4789063281015695),   (('text', 9), 0.4606560676192393),   (('video', 0), 0.5436101442068623),   (('video', 2), 0.5328547213082062),   (('video', 4), 0.5358392482255323),   (('video', 9), 0.5361742805393439),   (('voxaudio', 0), 0.6586772749875683),   (('voxaudio', 2), 0.6546016551999202),   (('voxaudio', 4), 0.6517544736579026),   (('voxaudio', 9), 0.6398671764942644),   (('text_video', 0), 0.5501740427647936),   (('text_video', 2), 0.5341509622095921),   (('text_video', 4), 0.5299410176946916),   (('text_video', 9), 0.5191185349164822),   (('text_voxaudio', 0), 0.5317752362008951),   (('text_voxaudio', 2), 0.5219862399042776),   (('text_voxaudio', 4), 0.5228431470558832),   (('text_voxaudio', 9), 0.5329040048299457),   (('video_voxaudio', 0), 0.6037792143212333),   (('video_voxaudio', 2), 0.5998105494067205),   (('video_voxaudio', 4), 0.5993202039388184),   (('video_voxaudio', 9), 0.5982592070839203),   (('text_video_voxaudio', 0), 0.5816011934361014),   (('text_video_voxaudio', 2), 0.5676039485492073),   (('text_video_voxaudio', 4), 0.5645306408077577),   (('text_video_voxaudio', 9), 0.5621352384785672)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6529232400901498),   (('text', 2), 0.6116943521594684),   (('text', 4), 0.5946449980018649),   (('text', 9), 0.5714285714285714),   (('video', 0), 0.5438154580405674),   (('video', 2), 0.5217275747508305),   (('video', 4), 0.5180498201678434),   (('video', 9), 0.5158134548378451),   (('voxaudio', 0), 0.6604799151531221),   (('voxaudio', 2), 0.6151495016611296),   (('voxaudio', 4), 0.5973091781004396),   (('voxaudio', 9), 0.5733047440364514),   (('text_video', 0), 0.5990985019223121),   (('text_video', 2), 0.5553488372093023),   (('text_video', 4), 0.5504196083655255),   (('text_video', 9), 0.5336370946127044),   (('text_voxaudio', 0), 0.6620707941137478),   (('text_voxaudio', 2), 0.6200664451827242),   (('text_voxaudio', 4), 0.605568136406021),   (('text_voxaudio', 9), 0.5755829536317342),   (('video_voxaudio', 0), 0.6476203102213973),   (('video_voxaudio', 2), 0.598671096345515),   (('video_voxaudio', 4), 0.5737311842280538),   (('video_voxaudio', 9), 0.5532028946663093),   (('text_video_voxaudio', 0), 0.6554421317778072),   (('text_video_voxaudio', 2), 0.6104983388704319),   (('text_video_voxaudio', 4), 0.596110297056081),   (('text_video_voxaudio', 9), 0.5682122755293487)],  5: [(('text', 0), 0.6466684364215556),   (('text', 2), 0.6101649813730707),   (('text', 4), 0.588957055214724),   (('text', 9), 0.5209982557359453),   (('video', 0), 0.5342447571011415),   (('video', 2), 0.518493879723257),   (('video', 4), 0.5194718591624433),   (('video', 9), 0.5196565141553737),   (('voxaudio', 0), 0.6506503849216884),   (('voxaudio', 2), 0.6113624268227781),   (('voxaudio', 4), 0.5921579087756735),   (('voxaudio', 9), 0.5701059975848651),   (('text_video', 0), 0.5914520838863817),   (('text_video', 2), 0.55401809473124),   (('text_video', 4), 0.5505468124833289),   (('text_video', 9), 0.5320005366966323),   (('text_voxaudio', 0), 0.6531722856384391),   (('text_voxaudio', 2), 0.6137573177221927),   (('text_voxaudio', 4), 0.5956255001333689),   (('text_voxaudio', 9), 0.5718502616396082),   (('video_voxaudio', 0), 0.6359171754711972),   (('video_voxaudio', 2), 0.5882118147951038),   (('video_voxaudio', 4), 0.5686849826620433),   (('video_voxaudio', 9), 0.5455521266604052),   (('text_video_voxaudio', 0), 0.6509158481550306),   (('text_video_voxaudio', 2), 0.6094997339010112),   (('text_video_voxaudio', 4), 0.5878901040277408),   (('text_video_voxaudio', 9), 0.5639339863142359)],  10: [(('text', 0), 0.6067565763119241),   (('text', 2), 0.5819836701914067),   (('text', 4), 0.5693009526365223),   (('text', 9), 0.5295626349892009),   (('video', 0), 0.5219655494725598),   (('video', 2), 0.5170659884888235),   (('video', 4), 0.511471890513887),   (('video', 9), 0.5024298056155507),   (('voxaudio', 0), 0.6263853652022967),   (('voxaudio', 2), 0.5971088207736581),   (('voxaudio', 4), 0.5840601100228096),   (('voxaudio', 9), 0.5599352051835853),   (('text_video', 0), 0.5589531312591801),   (('text_video', 2), 0.5347343059831348),   (('text_video', 4), 0.5211324298940024),   (('text_video', 9), 0.5067494600431965),   (('text_voxaudio', 0), 0.6305247696621712),   (('text_voxaudio', 2), 0.5993842859055013),   (('text_voxaudio', 4), 0.5847309808130954),   (('text_voxaudio', 9), 0.5592602591792657),   (('video_voxaudio', 0), 0.5807183869675524),   (('video_voxaudio', 2), 0.5486547985544104),   (('video_voxaudio', 4), 0.5338789749094325),   (('video_voxaudio', 9), 0.5133639308855291),   (('text_video_voxaudio', 0), 0.6189077313392977),   (('text_video_voxaudio', 2), 0.580779012180431),   (('text_video_voxaudio', 4), 0.5670199919495506),   (('text_video_voxaudio', 9), 0.5267278617710583)],  30: [(('text', 0), 0.566835408400602),   (('text', 2), 0.5506789192154711),   (('text', 4), 0.5495668912415784),   (('text', 9), 0.5355561704482568),   (('video', 0), 0.5097824599808455),   (('video', 2), 0.5052804827869977),   (('video', 4), 0.5024061597690087),   (('video', 9), 0.5051189817376868),   (('voxaudio', 0), 0.5792858120125872),   (('voxaudio', 2), 0.5560279797010013),   (('voxaudio', 4), 0.5484669324900316),   (('voxaudio', 9), 0.5444106253458771),   (('text_video', 0), 0.5363250786701327),   (('text_video', 2), 0.5155671375668632),   (('text_video', 4), 0.5083184380585728),   (('text_video', 9), 0.5019369120088545),   (('text_voxaudio', 0), 0.5846216992748666),   (('text_voxaudio', 2), 0.562885749554245),   (('text_voxaudio', 4), 0.5580915715660663),   (('text_voxaudio', 9), 0.5462091864969563),   (('video_voxaudio', 0), 0.5359146258038036),   (('video_voxaudio', 2), 0.5150185159786037),   (('video_voxaudio', 4), 0.5099683761858931),   (('video_voxaudio', 9), 0.5023519645821805),   (('text_video_voxaudio', 0), 0.5709399370638938),   (('text_video_voxaudio', 2), 0.5372376903031134),   (('text_video_voxaudio', 4), 0.5195930152619277),   (('text_video_voxaudio', 9), 0.5005534034311012)]}, "use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6539872562270709),   (('text', 2), 0.5961650203370134),   (('text', 4), 0.5805323489411307),   (('text', 9), 0.5490503230859605),   (('video', 0), 0.5864066422089207),   (('video', 2), 0.5649815998450514),   (('video', 4), 0.5490577035166117),   (('video', 9), 0.5392598394360681),   (('voxaudio', 0), 0.6472291948252559),   (('voxaudio', 2), 0.5930660468719736),   (('voxaudio', 4), 0.5748979988342724),   (('voxaudio', 9), 0.558449187389857),   (('text_video', 0), 0.6460706700135161),   (('text_video', 2), 0.5963587061785783),   (('text_video', 4), 0.5745094229648339),   (('text_video', 9), 0.5421969845310358),   (('text_voxaudio', 0), 0.6566904807877969),   (('text_voxaudio', 2), 0.6029440247917878),   (('text_voxaudio', 4), 0.5758694385078686),   (('text_voxaudio', 9), 0.5533581358919131),   (('video_voxaudio', 0), 0.653407993821201),   (('video_voxaudio', 2), 0.5979081929110982),   (('video_voxaudio', 4), 0.5702350884010103),   (('video_voxaudio', 9), 0.5568827100058743),   (('text_video_voxaudio', 0), 0.6441397953272833),   (('text_video_voxaudio', 2), 0.6048808832074375),   (('text_video_voxaudio', 4), 0.5733436953565184),   (('text_video_voxaudio', 9), 0.5578617583708635)],  5: [(('text', 0), 0.6465336134453782),   (('text', 2), 0.5967345505617978),   (('text', 4), 0.5725352112676056),   (('text', 9), 0.5574468085106383),   (('video', 0), 0.5835084033613446),   (('video', 2), 0.570751404494382),   (('video', 4), 0.5670774647887324),   (('video', 9), 0.5585106382978723),   (('voxaudio', 0), 0.6414565826330533),   (('voxaudio', 2), 0.6004213483146067),   (('voxaudio', 4), 0.5746478873239437),   (('voxaudio', 9), 0.5570921985815603),   (('text_video', 0), 0.6376050420168067),   (('text_video', 2), 0.5969101123595506),   (('text_video', 4), 0.5806338028169014),   (('text_video', 9), 0.5585106382978723),   (('text_voxaudio', 0), 0.6475840336134454),   (('text_voxaudio', 2), 0.6053370786516854),   (('text_voxaudio', 4), 0.585387323943662),   (('text_voxaudio', 9), 0.5593971631205674),   (('video_voxaudio', 0), 0.6432072829131653),   (('video_voxaudio', 2), 0.5956811797752809),   (('video_voxaudio', 4), 0.5748239436619719),   (('video_voxaudio', 9), 0.5539007092198581),   (('text_video_voxaudio', 0), 0.6465336134453782),   (('text_video_voxaudio', 2), 0.601123595505618),   (('text_video_voxaudio', 4), 0.5737676056338028),   (('text_video_voxaudio', 9), 0.5585106382978723)],  10: [(('text', 0), 0.6216502115655853),   (('text', 2), 0.5795615275813296),   (('text', 4), 0.5627659574468085),   (('text', 9), 0.5480357142857143),   (('video', 0), 0.590620592383639),   (('video', 2), 0.5694837340876945),   (('video', 4), 0.5666666666666667),   (('video', 9), 0.5523214285714285),   (('voxaudio', 0), 0.6167136812411848),   (('voxaudio', 2), 0.5815063649222065),   (('voxaudio', 4), 0.5732269503546099),   (('voxaudio', 9), 0.5582142857142857),   (('text_video', 0), 0.6100141043723555),   (('text_video', 2), 0.5781471004243282),   (('text_video', 4), 0.5673758865248227),   (('text_video', 9), 0.5521428571428572),   (('text_voxaudio', 0), 0.6242947813822285),   (('text_voxaudio', 2), 0.5949434229137199),   (('text_voxaudio', 4), 0.5762411347517731),   (('text_voxaudio', 9), 0.5592857142857143),   (('video_voxaudio', 0), 0.6174188998589563),   (('video_voxaudio', 2), 0.5848656294200849),   (('video_voxaudio', 4), 0.5776595744680851),   (('video_voxaudio', 9), 0.5623214285714285),   (('text_video_voxaudio', 0), 0.6161847672778561),   (('text_video_voxaudio', 2), 0.5878712871287128),   (('text_video_voxaudio', 4), 0.5792553191489361),   (('text_video_voxaudio', 9), 0.5655357142857143)],  30: [(('text', 0), 0.5693033381712627),   (('text', 2), 0.5516739446870451),   (('text', 4), 0.5474452554744526),   (('text', 9), 0.5479779411764706),   (('video', 0), 0.5662191582002902),   (('video', 2), 0.5609534206695779),   (('video', 4), 0.5490875912408759),   (('video', 9), 0.5424632352941177),   (('voxaudio', 0), 0.5705732946298984),   (('voxaudio', 2), 0.5591339155749636),   (('voxaudio', 4), 0.5593065693430657),   (('voxaudio', 9), 0.5424632352941177),   (('text_video', 0), 0.5749274310595065),   (('text_video', 2), 0.5587700145560408),   (('text_video', 4), 0.5541970802919708),   (('text_video', 9), 0.5564338235294117),   (('text_voxaudio', 0), 0.5702104499274311),   (('text_voxaudio', 2), 0.5531295487627366),   (('text_voxaudio', 4), 0.5598540145985401),   (('text_voxaudio', 9), 0.5481617647058824),   (('video_voxaudio', 0), 0.579644412191582),   (('video_voxaudio', 2), 0.5684133915574964),   (('video_voxaudio', 4), 0.5534671532846716),   (('video_voxaudio', 9), 0.5367647058823529),   (('text_video_voxaudio', 0), 0.5747460087082729),   (('text_video_voxaudio', 2), 0.5704148471615721),   (('text_video_voxaudio', 4), 0.5574817518248175),   (('text_video_voxaudio', 9), 0.5516544117647059)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6985922449987652),   (('text', 2), 0.6741445055217155),   (('text', 4), 0.6656571173228737),   (('text', 9), 0.6512418170006496),   (('video', 0), 0.6558162509261546),   (('video', 2), 0.6499281929381469),   (('video', 4), 0.6475348791023285),   (('video', 9), 0.647044125730848),   (('voxaudio', 0), 0.7197826623857743),   (('voxaudio', 2), 0.6952409250730451),   (('voxaudio', 4), 0.6834814557370538),   (('voxaudio', 9), 0.6702313727449902),   (('text_video', 0), 0.7045690293899728),   (('text_video', 2), 0.6796414599118507),   (('text_video', 4), 0.6708703639342635),   (('text_video', 9), 0.6503423117285493),   (('text_voxaudio', 0), 0.7202272165966905),   (('text_voxaudio', 2), 0.6930619521616401),   (('text_voxaudio', 4), 0.6864604538007051),   (('text_voxaudio', 9), 0.6727799710159412),   (('video_voxaudio', 0), 0.7134107186959743),   (('video_voxaudio', 2), 0.689694448571287),   (('video_voxaudio', 4), 0.6839283054466014),   (('video_voxaudio', 9), 0.6708810154415071),   (('text_video_voxaudio', 0), 0.7176092862435169),   (('text_video_voxaudio', 2), 0.6905363244688754),   (('text_video_voxaudio', 4), 0.6823395064793207),   (('text_video_voxaudio', 9), 0.6734795862275749)],  5: [(('text', 0), 0.6970616570327552),   (('text', 2), 0.6732663704848367),   (('text', 4), 0.6624709527498064),   (('text', 9), 0.6548280229952256),   (('video', 0), 0.6675818882466281),   (('video', 2), 0.6577168244156848),   (('video', 4), 0.6528853601859024),   (('video', 9), 0.6511254019292605),   (('voxaudio', 0), 0.7185934489402698),   (('voxaudio', 2), 0.6979428240293606),   (('voxaudio', 4), 0.6874515879163439),   (('voxaudio', 9), 0.6733411283250511),   (('text_video', 0), 0.7030346820809249),   (('text_video', 2), 0.6834073787908055),   (('text_video', 4), 0.6802381874515879),   (('text_video', 9), 0.6629153269024651),   (('text_voxaudio', 0), 0.7171001926782273),   (('text_voxaudio', 2), 0.6978462429978752),   (('text_voxaudio', 4), 0.6895817195972115),   (('text_voxaudio', 9), 0.6751437201597973),   (('video_voxaudio', 0), 0.7133429672447014),   (('video_voxaudio', 2), 0.6921479621402357),   (('video_voxaudio', 4), 0.6852730441518203),   (('video_voxaudio', 9), 0.6729026600409237),   (('text_video_voxaudio', 0), 0.7160886319845857),   (('text_video_voxaudio', 2), 0.6949971025690554),   (('text_video_voxaudio', 4), 0.685708752904725),   (('text_video_voxaudio', 9), 0.6748026892721426)],  10: [(('text', 0), 0.692195831313621),   (('text', 2), 0.6704247254349305),   (('text', 4), 0.6631102016954107),   (('text', 9), 0.652235732496568),   (('video', 0), 0.6529326223945711),   (('video', 2), 0.6511808727767519),   (('video', 4), 0.6497612783786417),   (('video', 9), 0.6486565993332026),   (('voxaudio', 0), 0.7106156083373728),   (('voxaudio', 2), 0.6960346000583147),   (('voxaudio', 4), 0.6876157069083114),   (('voxaudio', 9), 0.6737105314767602),   (('text_video', 0), 0.6936984973339796),   (('text_video', 2), 0.6765963650500535),   (('text_video', 4), 0.6596511741206275),   (('text_video', 9), 0.6500784467542655),   (('text_voxaudio', 0), 0.709112942317014),   (('text_voxaudio', 2), 0.6964719603460006),   (('text_voxaudio', 4), 0.6856669589788561),   (('text_voxaudio', 9), 0.6750833496764072),   (('video_voxaudio', 0), 0.7064469219583132),   (('video_voxaudio', 2), 0.6894256001555059),   (('video_voxaudio', 4), 0.6824515248952548),   (('video_voxaudio', 9), 0.6682682878995881),   (('text_video_voxaudio', 0), 0.7106640814348036),   (('text_video_voxaudio', 2), 0.6919039751190592),   (('text_video_voxaudio', 4), 0.6860079898665108),   (('text_video_voxaudio', 9), 0.6716022749558737)],  30: [(('text', 0), 0.6658876181004475),   (('text', 2), 0.6523083059128527),   (('text', 4), 0.6493052084374688),   (('text', 9), 0.647967397866774),   (('video', 0), 0.6552958727001492),   (('video', 2), 0.6520091733971483),   (('video', 4), 0.6522543237028892),   (('video', 9), 0.6492251962165425),   (('voxaudio', 0), 0.6872202884137245),   (('voxaudio', 2), 0.6780337022634361),   (('voxaudio', 4), 0.6726482055383385),   (('voxaudio', 9), 0.663614409337895),   (('text_video', 0), 0.6661859771258081),   (('text_video', 2), 0.6595871971283278),   (('text_video', 4), 0.658002599220234),   (('text_video', 9), 0.6545079492855705),   (('text_voxaudio', 0), 0.6894579811039284),   (('text_voxaudio', 2), 0.6811745936783329),   (('text_voxaudio', 4), 0.6743976806957913),   (('text_voxaudio', 9), 0.6663815657073858),   (('video_voxaudio', 0), 0.6819492789656887),   (('video_voxaudio', 2), 0.6795792202612424),   (('video_voxaudio', 4), 0.6737978606418075),   (('video_voxaudio', 9), 0.663211913865969),   (('text_video_voxaudio', 0), 0.6848831427150671),   (('text_video_voxaudio', 2), 0.6779339914248679),   (('text_video_voxaudio', 4), 0.676447065880236),   (('text_video_voxaudio', 9), 0.6611491245723485)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6559724247646825),   (('text', 2), 0.6112956810631229),   (('text', 4), 0.5971759690955109),   (('text', 9), 0.5722326454033771),   (('video', 0), 0.609041495426223),   (('video', 2), 0.575548172757475),   (('video', 4), 0.5585453576661782),   (('video', 9), 0.5533369069954436),   (('voxaudio', 0), 0.6692297494365637),   (('voxaudio', 2), 0.6235215946843854),   (('voxaudio', 4), 0.5974423871053683),   (('voxaudio', 9), 0.5767890645939426),   (('text_video', 0), 0.6569004374917141),   (('text_video', 2), 0.6175415282392026),   (('text_video', 4), 0.5997069401891568),   (('text_video', 9), 0.5809434467971053),   (('text_voxaudio', 0), 0.6758584117725043),   (('text_voxaudio', 2), 0.6256478405315614),   (('text_voxaudio', 4), 0.6045024643665912),   (('text_voxaudio', 9), 0.5802733851514339),   (('video_voxaudio', 0), 0.6635290998276547),   (('video_voxaudio', 2), 0.6194019933554817),   (('video_voxaudio', 4), 0.6034367923271613),   (('video_voxaudio', 9), 0.5797373358348968),   (('text_video_voxaudio', 0), 0.6697600424234389),   (('text_video_voxaudio', 2), 0.6256478405315614),   (('text_video_voxaudio', 4), 0.6104968695883842),   (('text_video_voxaudio', 9), 0.588448137228625)],  5: [(('text', 0), 0.6491903371383063),   (('text', 2), 0.6024481106971793),   (('text', 4), 0.5904241130968259),   (('text', 9), 0.5765463571716087),   (('video', 0), 0.6096363153703213),   (('video', 2), 0.5717136774880256),   (('video', 4), 0.55841557748733),   (('video', 9), 0.545417952502348),   (('voxaudio', 0), 0.6645872046721529),   (('voxaudio', 2), 0.6265300691857371),   (('voxaudio', 4), 0.608829021072286),   (('voxaudio', 9), 0.5752046155910372),   (('text_video', 0), 0.6525086275550837),   (('text_video', 2), 0.6152208621607238),   (('text_video', 4), 0.5945585489463857),   (('text_video', 9), 0.5734603515362942),   (('text_voxaudio', 0), 0.6693655428723122),   (('text_voxaudio', 2), 0.6263970196913252),   (('text_voxaudio', 4), 0.6041611096292344),   (('text_voxaudio', 9), 0.5872802898161814),   (('video_voxaudio', 0), 0.661667109105389),   (('video_voxaudio', 2), 0.6193453964874933),   (('video_voxaudio', 4), 0.6002934115764204),   (('video_voxaudio', 9), 0.5836575875486382),   (('text_video_voxaudio', 0), 0.6677727634722591),   (('text_video_voxaudio', 2), 0.6246673762639702),   (('text_video_voxaudio', 4), 0.6041611096292344),   (('text_video_voxaudio', 9), 0.588622031396753)],  10: [(('text', 0), 0.6215783148617973),   (('text', 2), 0.5934948467407308),   (('text', 4), 0.5906346437676103),   (('text', 9), 0.5715442764578834),   (('video', 0), 0.5780478034450528),   (('video', 2), 0.5548119394993977),   (('video', 4), 0.5475647390312626),   (('video', 9), 0.5265928725701944),   (('voxaudio', 0), 0.6418747496327948),   (('voxaudio', 2), 0.6141078838174274),   (('voxaudio', 4), 0.598819267409097),   (('voxaudio', 9), 0.5728941684665226),   (('text_video', 0), 0.631993590599546),   (('text_video', 2), 0.5991165841252845),   (('text_video', 4), 0.5698376492687508),   (('text_video', 9), 0.5322624190064795),   (('text_voxaudio', 0), 0.6490853251435439),   (('text_voxaudio', 2), 0.6189265158613305),   (('text_voxaudio', 4), 0.59962431235744),   (('text_voxaudio', 9), 0.5724892008639308),   (('video_voxaudio', 0), 0.6384029910535453),   (('video_voxaudio', 2), 0.6141078838174274),   (('video_voxaudio', 4), 0.5960016100898967),   (('video_voxaudio', 9), 0.5602051835853131),   (('text_video_voxaudio', 0), 0.6497529710241687),   (('text_video_voxaudio', 2), 0.6099585062240664),   (('text_video_voxaudio', 4), 0.5989534415671541),   (('text_video_voxaudio', 9), 0.5673596112311015)],  30: [(('text', 0), 0.5550690928991654),   (('text', 2), 0.5542449595391579),   (('text', 4), 0.5580915715660663),   (('text', 9), 0.5532650802434975),   (('video', 0), 0.5742235599945273),   (('video', 2), 0.5579481552599095),   (('video', 4), 0.5596040148494431),   (('video', 9), 0.5385998893193138),   (('voxaudio', 0), 0.600492543439595),   (('voxaudio', 2), 0.5840076807022356),   (('voxaudio', 4), 0.5667537467344974),   (('voxaudio', 9), 0.5534034311012729),   (('text_video', 0), 0.6026816253933507),   (('text_video', 2), 0.5962145110410094),   (('text_video', 4), 0.5695036436133645),   (('text_video', 9), 0.5485611510791367),   (('text_voxaudio', 0), 0.6029552606375701),   (('text_voxaudio', 2), 0.5856535454670141),   (('text_voxaudio', 4), 0.5763783858105321),   (('text_voxaudio', 9), 0.5534034311012729),   (('video_voxaudio', 0), 0.5992611848406074),   (('video_voxaudio', 2), 0.5829104375257166),   (('video_voxaudio', 4), 0.5798157569091159),   (('video_voxaudio', 9), 0.5532650802434975),   (('text_video_voxaudio', 0), 0.6009029963059243),   (('text_video_voxaudio', 2), 0.5933342477026471),   (('text_video_voxaudio', 4), 0.5866904991062836),   (('text_video_voxaudio', 9), 0.5539568345323741)]}, "use_crossval: True | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6495421578279535),   (('text', 2), 0.5878564811484824),   (('text', 4), 0.5533003759881916),   (('text', 9), 0.5189191297826732),   (('video', 0), 0.5192359413820778),   (('video', 2), 0.5051804947521831),   (('video', 4), 0.4972346271973011),   (('video', 9), 0.49625909688412584),   (('voxaudio', 0), 0.6847674467968973),   (('voxaudio', 2), 0.623508876445786),   (('voxaudio', 4), 0.5934301380226243),   (('voxaudio', 9), 0.5547619737142593),   (('text_video', 0), 0.5598077909248287),   (('text_video', 2), 0.5242564040462163),   (('text_video', 4), 0.5126536453371967),   (('text_video', 9), 0.4989114560751437),   (('text_voxaudio', 0), 0.6856497951795943),   (('text_voxaudio', 2), 0.623586628906604),   (('text_voxaudio', 4), 0.5962215354146921),   (('text_voxaudio', 9), 0.5555759631540683),   (('video_voxaudio', 0), 0.6439362344447342),   (('video_voxaudio', 2), 0.5791904546527735),   (('video_voxaudio', 4), 0.5506154899432978),   (('video_voxaudio', 9), 0.5195239039885351),   (('text_video_voxaudio', 0), 0.6570448111602356),   (('text_video_voxaudio', 2), 0.5884028140972526),   (('text_video_voxaudio', 4), 0.5567203312580375),   (('text_video_voxaudio', 9), 0.5212832267605743)],  5: [(('text', 0), 0.6293332682577489),   (('text', 2), 0.5725381480733935),   (('text', 4), 0.5509984628634959),   (('text', 9), 0.5146047308539099),   (('video', 0), 0.5131997544034145),   (('video', 2), 0.5009821759299846),   (('video', 4), 0.49538954697376675),   (('video', 9), 0.48720497403446644),   (('voxaudio', 0), 0.6781307293621237),   (('voxaudio', 2), 0.6240455462862923),   (('voxaudio', 4), 0.5953664608759018),   (('voxaudio', 9), 0.556584695911876),   (('text_video', 0), 0.5485174753508428),   (('text_video', 2), 0.5159269842859872),   (('text_video', 4), 0.5053696658950353),   (('text_video', 9), 0.49167687778685903),   (('text_voxaudio', 0), 0.6758935655362045),   (('text_voxaudio', 2), 0.6219146645590208),   (('text_voxaudio', 4), 0.5929198799364814),   (('text_voxaudio', 9), 0.5542064601463389),   (('video_voxaudio', 0), 0.6367992693470291),   (('video_voxaudio', 2), 0.5803327672389804),   (('video_voxaudio', 4), 0.5515348460754879),   (('video_voxaudio', 9), 0.5148602571679588),   (('text_video_voxaudio', 0), 0.6443497027146645),   (('text_video_voxaudio', 2), 0.5869780437216483),   (('text_video_voxaudio', 4), 0.5575232676128528),   (('text_video_voxaudio', 9), 0.5180867523869178)],  10: [(('text', 0), 0.6178053693988834),   (('text', 2), 0.5818258581896032),   (('text', 4), 0.5585227779257285),   (('text', 9), 0.5312027592742684),   (('video', 0), 0.510017427611629),   (('video', 2), 0.5000424730897097),   (('video', 4), 0.4949456954532815),   (('video', 9), 0.49452904971774253),   (('voxaudio', 0), 0.6384634536644452),   (('voxaudio', 2), 0.598868891473415),   (('voxaudio', 4), 0.5768354206541544),   (('voxaudio', 9), 0.5456873438450707),   (('text_video', 0), 0.5604133805155787),   (('text_video', 2), 0.5263519468901487),   (('text_video', 4), 0.5140485921163134),   (('text_video', 9), 0.5038823293744128),   (('text_voxaudio', 0), 0.6388698291130707),   (('text_voxaudio', 2), 0.5970204153245715),   (('text_voxaudio', 4), 0.5751184652218585),   (('text_voxaudio', 9), 0.5435472004175821),   (('video_voxaudio', 0), 0.605389037026129),   (('video_voxaudio', 2), 0.5620124601398042),   (('video_voxaudio', 4), 0.5398553919339418),   (('video_voxaudio', 9), 0.5178999911351139),   (('text_video_voxaudio', 0), 0.6141996704576366),   (('text_video_voxaudio', 2), 0.5720691790307317),   (('text_video_voxaudio', 4), 0.5493119402730458),   (('text_video_voxaudio', 9), 0.5231664349198406)],  30: [(('text', 0), 0.5786259010933945),   (('text', 2), 0.5620501700560834),   (('text', 4), 0.5459432048681542),   (('text', 9), 0.5281201234374271),   (('video', 0), 0.4919217145786135),   (('video', 2), 0.48577166850293196),   (('video', 4), 0.484370943842948),   (('video', 9), 0.4862690727222514),   (('voxaudio', 0), 0.5708241681394307),   (('voxaudio', 2), 0.5476818879818169),   (('voxaudio', 4), 0.5343189422756695),   (('voxaudio', 9), 0.5137034411128298),   (('text_video', 0), 0.5322858197359758),   (('text_video', 2), 0.5145081748940191),   (('text_video', 4), 0.5062856580242049),   (('text_video', 9), 0.4901602111269184),   (('text_voxaudio', 0), 0.5805626102181413),   (('text_voxaudio', 2), 0.5579727078321679),   (('text_voxaudio', 4), 0.5434962617806927),   (('text_voxaudio', 9), 0.5211499126398458),   (('video_voxaudio', 0), 0.5331987019395331),   (('video_voxaudio', 2), 0.5128989789300309),   (('video_voxaudio', 4), 0.5040330469631348),   (('video_voxaudio', 9), 0.4883124401513572),   (('text_video_voxaudio', 0), 0.5504345663224062),   (('text_video_voxaudio', 2), 0.5306792466098086),   (('text_video_voxaudio', 4), 0.51727122597475),   (('text_video_voxaudio', 9), 0.49497493921464975)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.671555662971522),   (('text', 2), 0.6123672704559651),   (('text', 4), 0.599591738660769),   (('text', 9), 0.5748549256198585),   (('video', 0), 0.5576084483926695),   (('video', 2), 0.5461145607524709),   (('video', 4), 0.542276239221862),   (('video', 9), 0.5310812002880885),   (('voxaudio', 0), 0.7031173301892262),   (('voxaudio', 2), 0.6603813792850057),   (('voxaudio', 4), 0.6436909856762745),   (('voxaudio', 9), 0.625969299650994),   (('text_video', 0), 0.6414308004973109),   (('text_video', 2), 0.6015064114340302),   (('text_video', 4), 0.585388173515563),   (('text_video', 9), 0.5584102798197941),   (('text_voxaudio', 0), 0.6990200529449961),   (('text_voxaudio', 2), 0.643935775434471),   (('text_voxaudio', 4), 0.6280211827849997),   (('text_voxaudio', 9), 0.6061136811592092),   (('video_voxaudio', 0), 0.6645193458483327),   (('video_voxaudio', 2), 0.6246463607304259),   (('video_voxaudio', 4), 0.6089922861517474),   (('video_voxaudio', 9), 0.5845457285507417),   (('text_video_voxaudio', 0), 0.6900486348399804),   (('text_video_voxaudio', 2), 0.6407980306426131),   (('text_video_voxaudio', 4), 0.6231443382952732),   (('text_video_voxaudio', 9), 0.5955040202445318)],  5: [(('text', 0), 0.6768598986991343),   (('text', 2), 0.6358361885295272),   (('text', 4), 0.6157670525686695),   (('text', 9), 0.5903164393472323),   (('video', 0), 0.5497097132219093),   (('video', 2), 0.538431091174597),   (('video', 4), 0.5336887848726968),   (('video', 9), 0.5216805206621352),   (('voxaudio', 0), 0.6997071686488399),   (('voxaudio', 2), 0.6626996172497414),   (('voxaudio', 4), 0.6455940272865319),   (('voxaudio', 9), 0.6282407319458313),   (('text_video', 0), 0.6193035772083382),   (('text_video', 2), 0.5861956819922309),   (('text_video', 4), 0.5717011667920494),   (('text_video', 9), 0.54240555986723),   (('text_voxaudio', 0), 0.6949345463957047),   (('text_voxaudio', 2), 0.6543679143796668),   (('text_voxaudio', 4), 0.6347353029760334),   (('text_voxaudio', 9), 0.6090486509604645),   (('video_voxaudio', 0), 0.6533998675490608),   (('video_voxaudio', 2), 0.618162936158367),   (('video_voxaudio', 4), 0.6015280675745843),   (('video_voxaudio', 9), 0.5752786693449676),   (('text_video_voxaudio', 0), 0.678201085546779),   (('text_video_voxaudio', 2), 0.6368692326877674),   (('text_video_voxaudio', 4), 0.6169705213430655),   (('text_video_voxaudio', 9), 0.5841696274003094)],  10: [(('text', 0), 0.5811783428377613),   (('text', 2), 0.523266273394551),   (('text', 4), 0.49142406670047645),   (('text', 9), 0.45736721161926097),   (('video', 0), 0.5408929586124918),   (('video', 2), 0.530378690668295),   (('video', 4), 0.5250830214934218),   (('video', 9), 0.5182381968600402),   (('voxaudio', 0), 0.6798408298789592),   (('voxaudio', 2), 0.6582110517150042),   (('voxaudio', 4), 0.6497171748639153),   (('voxaudio', 9), 0.6374654655431622),   (('text_video', 0), 0.5767013863623583),   (('text_video', 2), 0.552737957725175),   (('text_video', 4), 0.5395351125560695),   (('text_video', 9), 0.5185697731498239),   (('text_voxaudio', 0), 0.642117552145183),   (('text_voxaudio', 2), 0.6137364720771329),   (('text_voxaudio', 4), 0.5982409382357182),   (('text_voxaudio', 9), 0.5687714464323997),   (('video_voxaudio', 0), 0.6347275506172159),   (('video_voxaudio', 2), 0.6115731744271695),   (('video_voxaudio', 4), 0.5986092994389285),   (('video_voxaudio', 9), 0.5819537627177971),   (('text_video_voxaudio', 0), 0.6386857835879949),   (('text_video_voxaudio', 2), 0.6098677277725841),   (('text_video_voxaudio', 4), 0.5930889724087021),   (('text_video_voxaudio', 9), 0.5651757764826092)],  30: [(('text', 0), 0.5229911514822895),   (('text', 2), 0.5063262187497839),   (('text', 4), 0.49713388955774623),   (('text', 9), 0.4801891871949933),   (('video', 0), 0.5305670958281145),   (('video', 2), 0.5242317016369821),   (('video', 4), 0.5226766517726793),   (('video', 9), 0.5208450345767573),   (('voxaudio', 0), 0.6571161993204042),   (('voxaudio', 2), 0.6512731066345603),   (('voxaudio', 4), 0.6473285816123214),   (('voxaudio', 9), 0.6398188130665725),   (('text_video', 0), 0.552516997540663),   (('text_video', 2), 0.5344939702512123),   (('text_video', 4), 0.527451718677359),   (('text_video', 9), 0.5162805620124757),   (('text_voxaudio', 0), 0.565832251054358),   (('text_voxaudio', 2), 0.5562060298449196),   (('text_voxaudio', 4), 0.5529697529402194),   (('text_voxaudio', 9), 0.5563478781742364),   (('video_voxaudio', 0), 0.6093084253641647),   (('video_voxaudio', 2), 0.6045274376866789),   (('video_voxaudio', 4), 0.6015183879038801),   (('video_voxaudio', 9), 0.5949912370783761),   (('text_video_voxaudio', 0), 0.5824986991977159),   (('text_video_voxaudio', 2), 0.5704493027120277),   (('text_video_voxaudio', 4), 0.5656089226061074),   (('text_video_voxaudio', 9), 0.5629714896852759)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6622354344086439),   (('text', 2), 0.6142788698789332),   (('text', 4), 0.5923248634569396),   (('text', 9), 0.5455600786798769),   (('video', 0), 0.5240508663618886),   (('video', 2), 0.5090461351094048),   (('video', 4), 0.49847979645342244),   (('video', 9), 0.4894046457326583),   (('voxaudio', 0), 0.6771201317494551),   (('voxaudio', 2), 0.6203176995184589),   (('voxaudio', 4), 0.5945618302074402),   (('voxaudio', 9), 0.5617720058872255),   (('text_video', 0), 0.5821584791200417),   (('text_video', 2), 0.5398112164970759),   (('text_video', 4), 0.5205604797468495),   (('text_video', 9), 0.49861578594143746),   (('text_voxaudio', 0), 0.6760991881133678),   (('text_voxaudio', 2), 0.6201332567538473),   (('text_voxaudio', 4), 0.5955056682549278),   (('text_voxaudio', 9), 0.563589632928027),   (('video_voxaudio', 0), 0.6485759811538749),   (('video_voxaudio', 2), 0.5886099146338433),   (('video_voxaudio', 4), 0.5589765688518213),   (('video_voxaudio', 9), 0.5223868888140804),   (('text_video_voxaudio', 0), 0.6583969287919709),   (('text_video_voxaudio', 2), 0.5982510696875998),   (('text_video_voxaudio', 4), 0.5694424937237746),   (('text_video_voxaudio', 9), 0.5298219674420256)],  5: [(('text', 0), 0.6535871545054965),   (('text', 2), 0.6111295208590596),   (('text', 4), 0.568993791104759),   (('text', 9), 0.520637014787835),   (('video', 0), 0.5260404266999892),   (('video', 2), 0.5069270990076472),   (('video', 4), 0.49768385549275845),   (('video', 9), 0.47967816202561026),   (('voxaudio', 0), 0.6666801714114434),   (('voxaudio', 2), 0.6167046371470442),   (('voxaudio', 4), 0.591227209178731),   (('voxaudio', 9), 0.559156959378497),   (('text_video', 0), 0.582291906102476),   (('text_video', 2), 0.5430826146473824),   (('text_video', 4), 0.5248602419559717),   (('text_video', 9), 0.4975638683983908),   (('text_voxaudio', 0), 0.6663066606981422),   (('text_voxaudio', 2), 0.616184648814588),   (('text_voxaudio', 4), 0.5916235792500611),   (('text_voxaudio', 9), 0.5606056614539958),   (('video_voxaudio', 0), 0.6390827744407732),   (('video_voxaudio', 2), 0.5866435380202816),   (('video_voxaudio', 4), 0.559461187570421),   (('video_voxaudio', 9), 0.520403441325134),   (('text_video_voxaudio', 0), 0.6467601325036558),   (('text_video_voxaudio', 2), 0.593987064747505),   (('text_video_voxaudio', 4), 0.5689512043585678),   (('text_video_voxaudio', 9), 0.5316767639961519)],  10: [(('text', 0), 0.6091316039686244),   (('text', 2), 0.5790363707669173),   (('text', 4), 0.5628004786581935),   (('text', 9), 0.5248781354501963),   (('video', 0), 0.5075176118373624),   (('video', 2), 0.49618828097658874),   (('video', 4), 0.4881475948200252),   (('video', 9), 0.4878736164943245),   (('voxaudio', 0), 0.6273059071638538),   (('voxaudio', 2), 0.5913273208110502),   (('voxaudio', 4), 0.5738577217193968),   (('voxaudio', 9), 0.542428994610573),   (('text_video', 0), 0.5610368935749892),   (('text_video', 2), 0.5322051985072983),   (('text_video', 4), 0.517624912722511),   (('text_video', 9), 0.5023113315133424),   (('text_voxaudio', 0), 0.6252129214063562),   (('text_voxaudio', 2), 0.589400130539448),   (('text_voxaudio', 4), 0.5717749786491118),   (('text_voxaudio', 9), 0.5417160885125941),   (('video_voxaudio', 0), 0.5914268661454747),   (('video_voxaudio', 2), 0.5576223723104609),   (('video_voxaudio', 4), 0.5378931174952077),   (('video_voxaudio', 9), 0.512859047129431),   (('text_video_voxaudio', 0), 0.6063553092542625),   (('text_video_voxaudio', 2), 0.5664234397896134),   (('text_video_voxaudio', 4), 0.5475977842097561),   (('text_video_voxaudio', 9), 0.5188465258383493)],  30: [(('text', 0), 0.5547095370468154),   (('text', 2), 0.5382018334181828),   (('text', 4), 0.5282599383772776),   (('text', 9), 0.5018243774483537),   (('video', 0), 0.48423122995357276),   (('video', 2), 0.47900144954187496),   (('video', 4), 0.4787761544616636),   (('video', 9), 0.47435418538114515),   (('voxaudio', 0), 0.5661410978314572),   (('voxaudio', 2), 0.5460479365622382),   (('voxaudio', 4), 0.5310732782662514),   (('voxaudio', 9), 0.4972094232045915),   (('text_video', 0), 0.5175507546662015),   (('text_video', 2), 0.5013922657603975),   (('text_video', 4), 0.4929324232599769),   (('text_video', 9), 0.4763711432474424),   (('text_voxaudio', 0), 0.5682833999527865),   (('text_voxaudio', 2), 0.5462183211275046),   (('text_voxaudio', 4), 0.5330565138302108),   (('text_voxaudio', 9), 0.4975313991123021),   (('video_voxaudio', 0), 0.5246982577947348),   (('video_voxaudio', 2), 0.506665861868644),   (('video_voxaudio', 4), 0.4981340864390632),   (('video_voxaudio', 9), 0.4755560540877834),   (('text_video_voxaudio', 0), 0.5338182770669715),   (('text_video_voxaudio', 2), 0.5120244287004865),   (('text_video_voxaudio', 4), 0.5026541013377209),   (('text_video_voxaudio', 9), 0.47628555959788327)]}, "use_crossval: True | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6786669928159555),   (('text', 2), 0.6154924080953045),   (('text', 4), 0.5831756158265542),   (('text', 9), 0.5435233260299),   (('video', 0), 0.5229472237889518),   (('video', 2), 0.4952604253655618),   (('video', 4), 0.48089777594160293),   (('video', 9), 0.46921075342307067),   (('voxaudio', 0), 0.6763051968924172),   (('voxaudio', 2), 0.6105741992139958),   (('voxaudio', 4), 0.5768613698625413),   (('voxaudio', 9), 0.5339912456711712),   (('text_video', 0), 0.6379134835102558),   (('text_video', 2), 0.564223776458695),   (('text_video', 4), 0.5282261289613411),   (('text_video', 9), 0.4934209042025633),   (('text_voxaudio', 0), 0.6825090752387324),   (('text_voxaudio', 2), 0.6140353080155607),   (('text_voxaudio', 4), 0.5815317087816199),   (('text_voxaudio', 9), 0.5361706192914346),   (('video_voxaudio', 0), 0.6597440267172872),   (('video_voxaudio', 2), 0.5795786006263813),   (('video_voxaudio', 4), 0.538636209807746),   (('video_voxaudio', 9), 0.5003795034161754),   (('text_video_voxaudio', 0), 0.6621063650998698),   (('text_video_voxaudio', 2), 0.5844188877253111),   (('text_video_voxaudio', 4), 0.5421849706763986),   (('text_video_voxaudio', 9), 0.4989879104457852)],  5: [(('text', 0), 0.6647914122012433),   (('text', 2), 0.6075585665481404),   (('text', 4), 0.5765843531274193),   (('text', 9), 0.5353249689073586),   (('video', 0), 0.5303964783011952),   (('video', 2), 0.5055805570296095),   (('video', 4), 0.4882468047334682),   (('video', 9), 0.4818527768705841),   (('voxaudio', 0), 0.6667766674709903),   (('voxaudio', 2), 0.607614514198972),   (('voxaudio', 4), 0.5792559928897506),   (('voxaudio', 9), 0.5386088384676462),   (('text_video', 0), 0.6314859373877089),   (('text_video', 2), 0.568276011136535),   (('text_video', 4), 0.5409623657871065),   (('text_video', 9), 0.49926136555919554),   (('text_voxaudio', 0), 0.6714749434744673),   (('text_voxaudio', 2), 0.6079791508804865),   (('text_voxaudio', 4), 0.5773999512508251),   (('text_voxaudio', 9), 0.5399955214020339),   (('video_voxaudio', 0), 0.6446009622856386),   (('video_voxaudio', 2), 0.5775570899170075),   (('video_voxaudio', 4), 0.5469508347535375),   (('video_voxaudio', 9), 0.5105836171064462),   (('text_video_voxaudio', 0), 0.6494948234897244),   (('text_video_voxaudio', 2), 0.5836696980588119),   (('text_video_voxaudio', 4), 0.5503810312825914),   (('text_video_voxaudio', 9), 0.5091121647268659)],  10: [(('text', 0), 0.6209220510311375),   (('text', 2), 0.5804929881722003),   (('text', 4), 0.5536713459423139),   (('text', 9), 0.5236593139486949),   (('video', 0), 0.5128073775780873),   (('video', 2), 0.4965869570802733),   (('video', 4), 0.4863603578176221),   (('video', 9), 0.4773813404140862),   (('voxaudio', 0), 0.6163672658210728),   (('voxaudio', 2), 0.5768517061908366),   (('voxaudio', 4), 0.5579773844966569),   (('voxaudio', 9), 0.5237418969542558),   (('text_video', 0), 0.5884162734399557),   (('text_video', 2), 0.54860941589703),   (('text_video', 4), 0.5201248121956812),   (('text_video', 9), 0.4893136193415276),   (('text_voxaudio', 0), 0.6214910060521301),   (('text_voxaudio', 2), 0.5796238219674366),   (('text_voxaudio', 4), 0.5557154996515423),   (('text_voxaudio', 9), 0.5231382815058223),   (('video_voxaudio', 0), 0.5998566140024587),   (('video_voxaudio', 2), 0.5536930023649911),   (('video_voxaudio', 4), 0.5334508970883088),   (('video_voxaudio', 9), 0.49833886600414906),   (('text_video_voxaudio', 0), 0.6025133222018683),   (('text_video_voxaudio', 2), 0.5612215980952994),   (('text_video_voxaudio', 4), 0.537239031357809),   (('text_video_voxaudio', 9), 0.5005057030972333)],  30: [(('text', 0), 0.518257950199433),   (('text', 2), 0.49883654057982235),   (('text', 4), 0.4890981597160831),   (('text', 9), 0.4725502647746211),   (('video', 0), 0.49161622150830553),   (('video', 2), 0.47695047512464195),   (('video', 4), 0.4754429665743656),   (('video', 9), 0.46624990528528054),   (('voxaudio', 0), 0.5286604993296746),   (('voxaudio', 2), 0.5051321310566917),   (('voxaudio', 4), 0.49719091764223267),   (('voxaudio', 9), 0.48433553261744733),   (('text_video', 0), 0.5105114121755255),   (('text_video', 2), 0.4935936797942718),   (('text_video', 4), 0.47928085394098946),   (('text_video', 9), 0.471765238330042),   (('text_voxaudio', 0), 0.5336678052725623),   (('text_voxaudio', 2), 0.51142919508295),   (('text_voxaudio', 4), 0.4977474082746802),   (('text_voxaudio', 9), 0.4793811403325803),   (('video_voxaudio', 0), 0.524925435759266),   (('video_voxaudio', 2), 0.5041895928815355),   (('video_voxaudio', 4), 0.49499381178642377),   (('video_voxaudio', 9), 0.472690038543112),   (('text_video_voxaudio', 0), 0.5263639517105588),   (('text_video_voxaudio', 2), 0.5030523605098794),   (('text_video_voxaudio', 4), 0.4946325310234381),   (('text_video_voxaudio', 9), 0.4764411640837084)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.7190447998172413),   (('text', 2), 0.6861005988903993),   (('text', 4), 0.6706471284268235),   (('text', 9), 0.653335688580216),   (('video', 0), 0.6524177476979405),   (('video', 2), 0.6482345592828013),   (('video', 4), 0.6476989361257939),   (('video', 9), 0.6472930738301318),   (('voxaudio', 0), 0.7325241355995085),   (('voxaudio', 2), 0.7000183708711467),   (('voxaudio', 4), 0.6878566412121523),   (('voxaudio', 9), 0.6746072665450058),   (('text_video', 0), 0.7185610328317733),   (('text_video', 2), 0.6854392475291178),   (('text_video', 4), 0.6688201631672197),   (('text_video', 9), 0.6523644194357894),   (('text_voxaudio', 0), 0.7364381955509789),   (('text_voxaudio', 2), 0.7025829444832274),   (('text_voxaudio', 4), 0.6889690582173772),   (('text_voxaudio', 9), 0.6747629618690927),   (('video_voxaudio', 0), 0.7251357718508511),   (('video_voxaudio', 2), 0.6946320314509313),   (('video_voxaudio', 4), 0.683362725449031),   (('video_voxaudio', 9), 0.6709371739085171),   (('text_video_voxaudio', 0), 0.7325021323328562),   (('text_video_voxaudio', 2), 0.6970863798361318),   (('text_video_voxaudio', 4), 0.6849171693934755),   (('text_video_voxaudio', 9), 0.6721012251319759)],  5: [(('text', 0), 0.7117744460359982),   (('text', 2), 0.6823857964567941),   (('text', 4), 0.6677149072398999),   (('text', 9), 0.6526378355099858),   (('video', 0), 0.6484910899771137),   (('video', 2), 0.6440539229335357),   (('video', 4), 0.6427386002306087),   (('video', 9), 0.6408571366027865),   (('voxaudio', 0), 0.7279022640617216),   (('voxaudio', 2), 0.6992466770887684),   (('voxaudio', 4), 0.6880386565164474),   (('voxaudio', 9), 0.6732709823801721),   (('text_video', 0), 0.7101836419646813),   (('text_video', 2), 0.6795908341745364),   (('text_video', 4), 0.6650497501750195),   (('text_video', 9), 0.648935767780445),   (('text_voxaudio', 0), 0.731674747436489),   (('text_voxaudio', 2), 0.7011378174835274),   (('text_voxaudio', 4), 0.6877493192177584),   (('text_voxaudio', 9), 0.6731406921578656),   (('video_voxaudio', 0), 0.7197133693957987),   (('video_voxaudio', 2), 0.6932010430546389),   (('video_voxaudio', 4), 0.6821600552548029),   (('video_voxaudio', 9), 0.668702759060409),   (('text_video_voxaudio', 0), 0.7264023237449871),   (('text_video_voxaudio', 2), 0.6949402836576625),   (('text_video_voxaudio', 4), 0.6820762672110356),   (('text_video_voxaudio', 9), 0.6687104509745592)],  10: [(('text', 0), 0.6969406216931839),   (('text', 2), 0.6779100833395695),   (('text', 4), 0.6665541428551505),   (('text', 9), 0.6569759986976763),   (('video', 0), 0.6495389748294895),   (('video', 2), 0.6481408054899305),   (('video', 4), 0.6469574413562629),   (('video', 9), 0.6456775240858363),   (('voxaudio', 0), 0.7096777498921039),   (('voxaudio', 2), 0.6921639276889598),   (('voxaudio', 4), 0.6823480594135107),   (('voxaudio', 9), 0.6698180625306237),   (('text_video', 0), 0.6932110233068345),   (('text_video', 2), 0.6717466855016697),   (('text_video', 4), 0.6609036246733814),   (('text_video', 9), 0.6531249522400088),   (('text_voxaudio', 0), 0.7114929520338535),   (('text_voxaudio', 2), 0.693639813348945),   (('text_voxaudio', 4), 0.6834827785612505),   (('text_voxaudio', 9), 0.6717551541748804),   (('video_voxaudio', 0), 0.7017532824323341),   (('video_voxaudio', 2), 0.684815220773115),   (('video_voxaudio', 4), 0.6775332497096678),   (('video_voxaudio', 9), 0.6650639661364374),   (('text_video_voxaudio', 0), 0.7051243973872957),   (('text_video_voxaudio', 2), 0.6870710503864299),   (('text_video_voxaudio', 4), 0.6775869192259496),   (('text_video_voxaudio', 9), 0.6668775938477353)],  30: [(('text', 0), 0.655522581358392),   (('text', 2), 0.6520026182846841),   (('text', 4), 0.6506420666972346),   (('text', 9), 0.6503488048522519),   (('video', 0), 0.641131399359721),   (('video', 2), 0.639776928615092),   (('video', 4), 0.6393255111178164),   (('video', 9), 0.6401399466879888),   (('voxaudio', 0), 0.6665713182834371),   (('voxaudio', 2), 0.6585886196529799),   (('voxaudio', 4), 0.6542663711451311),   (('voxaudio', 9), 0.6449076972020669),   (('text_video', 0), 0.6527449388787898),   (('text_video', 2), 0.6481239630924828),   (('text_video', 4), 0.6462010009111581),   (('text_video', 9), 0.6437020728188063),   (('text_voxaudio', 0), 0.6691323215447685),   (('text_voxaudio', 2), 0.6594573754666204),   (('text_voxaudio', 4), 0.6547485488188618),   (('text_voxaudio', 9), 0.6440699929755229),   (('video_voxaudio', 0), 0.6634532661386863),   (('video_voxaudio', 2), 0.6557571005129093),   (('video_voxaudio', 4), 0.6525708179223466),   (('video_voxaudio', 9), 0.6426921149542997),   (('text_video_voxaudio', 0), 0.6671052015378259),   (('text_video_voxaudio', 2), 0.658495449009991),   (('text_video_voxaudio', 4), 0.6520885758237636),   (('text_video_voxaudio', 9), 0.6439838774953888)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6610105759327962),   (('text', 2), 0.609304638577546),   (('text', 4), 0.5841364382417231),   (('text', 9), 0.5499382918166179),   (('video', 0), 0.5007938942690195),   (('video', 2), 0.4692929457312055),   (('video', 4), 0.45220162585616885),   (('video', 9), 0.44025144161767915),   (('voxaudio', 0), 0.678038594786425),   (('voxaudio', 2), 0.61411486863142),   (('voxaudio', 4), 0.5835202199275946),   (('voxaudio', 9), 0.5357079052533577),   (('text_video', 0), 0.6167851386247806),   (('text_video', 2), 0.5450081674608273),   (('text_video', 4), 0.5166995409563434),   (('text_video', 9), 0.4714161493723398),   (('text_voxaudio', 0), 0.6799986097189195),   (('text_voxaudio', 2), 0.6154452797783797),   (('text_voxaudio', 4), 0.5844232218433804),   (('text_voxaudio', 9), 0.5377525938369002),   (('video_voxaudio', 0), 0.6513726180493502),   (('video_voxaudio', 2), 0.5703903488622022),   (('video_voxaudio', 4), 0.529792268728395),   (('video_voxaudio', 9), 0.48000724810800743),   (('text_video_voxaudio', 0), 0.6549459530835373),   (('text_video_voxaudio', 2), 0.5720688416458378),   (('text_video_voxaudio', 4), 0.5306133646621429),   (('text_video_voxaudio', 9), 0.482258360554742)],  5: [(('text', 0), 0.6488563108718621),   (('text', 2), 0.6012895048149784),   (('text', 4), 0.5771070608408809),   (('text', 9), 0.5429718178141762),   (('video', 0), 0.4961408636375781),   (('video', 2), 0.4738291586174048),   (('video', 4), 0.45993248691262706),   (('video', 9), 0.44562779148967124),   (('voxaudio', 0), 0.6663894717156114),   (('voxaudio', 2), 0.6082788396656301),   (('voxaudio', 4), 0.5764176193723383),   (('voxaudio', 9), 0.5306688679881971),   (('text_video', 0), 0.606817573201454),   (('text_video', 2), 0.5385671961646292),   (('text_video', 4), 0.509111663495472),   (('text_video', 9), 0.47096553417449094),   (('text_voxaudio', 0), 0.6703526783581633),   (('text_voxaudio', 2), 0.6085701573551601),   (('text_voxaudio', 4), 0.578190658151425),   (('text_voxaudio', 9), 0.5329365381582621),   (('video_voxaudio', 0), 0.636198715957758),   (('video_voxaudio', 2), 0.5636972647401094),   (('video_voxaudio', 4), 0.5255467471521957),   (('video_voxaudio', 9), 0.47854342031408637),   (('text_video_voxaudio', 0), 0.6425273110481579),   (('text_video_voxaudio', 2), 0.5656734844297509),   (('text_video_voxaudio', 4), 0.5280288926577562),   (('text_video_voxaudio', 9), 0.4790474035730507)],  10: [(('text', 0), 0.5997132341990647),   (('text', 2), 0.5687789391482486),   (('text', 4), 0.5476845200321258),   (('text', 9), 0.5161244442874682),   (('video', 0), 0.4803737958392982),   (('video', 2), 0.4635754234011403),   (('video', 4), 0.45194604022274076),   (('video', 9), 0.4423952820766397),   (('voxaudio', 0), 0.6080848875850602),   (('voxaudio', 2), 0.5644101287317873),   (('voxaudio', 4), 0.5395252906721857),   (('voxaudio', 9), 0.4967783973670586),   (('text_video', 0), 0.5439726909044715),   (('text_video', 2), 0.504538550912345),   (('text_video', 4), 0.48653421632536437),   (('text_video', 9), 0.4568981375974082),   (('text_voxaudio', 0), 0.6148762592956907),   (('text_voxaudio', 2), 0.5683286322096256),   (('text_voxaudio', 4), 0.5406847294151207),   (('text_voxaudio', 9), 0.5005179097369329),   (('video_voxaudio', 0), 0.5694084534538677),   (('video_voxaudio', 2), 0.5227395950838656),   (('video_voxaudio', 4), 0.497870277138816),   (('video_voxaudio', 9), 0.4645496961340189),   (('text_video_voxaudio', 0), 0.5779296634906937),   (('text_video_voxaudio', 2), 0.5251805756191568),   (('text_video_voxaudio', 4), 0.500683209982378),   (('text_video_voxaudio', 9), 0.46610585941219285)],  30: [(('text', 0), 0.4933723186752541),   (('text', 2), 0.47651288673075987),   (('text', 4), 0.4656432579898997),   (('text', 9), 0.44231404794101925),   (('video', 0), 0.4464367539191999),   (('video', 2), 0.4391083456217009),   (('video', 4), 0.4408690683661513),   (('video', 9), 0.4322265554791917),   (('voxaudio', 0), 0.48671315576468305),   (('voxaudio', 2), 0.46577375762105994),   (('voxaudio', 4), 0.447201454542995),   (('voxaudio', 9), 0.42851484126840794),   (('text_video', 0), 0.47188763215220025),   (('text_video', 2), 0.4522496132036723),   (('text_video', 4), 0.44336396284368307),   (('text_video', 9), 0.4335573453458895),   (('text_voxaudio', 0), 0.48777354041990667),   (('text_voxaudio', 2), 0.46528456791691547),   (('text_voxaudio', 4), 0.4510603295932203),   (('text_voxaudio', 9), 0.42866463220803874),   (('video_voxaudio', 0), 0.47742363168074126),   (('video_voxaudio', 2), 0.4606701956310766),   (('video_voxaudio', 4), 0.45027137387684446),   (('video_voxaudio', 9), 0.43248439088911655),   (('text_video_voxaudio', 0), 0.4819835828343482),   (('text_video_voxaudio', 2), 0.4606489981881188),   (('text_video_voxaudio', 4), 0.45014338075659044),   (('text_video_voxaudio', 9), 0.4303814556586977)]}}
for k, v in all_results.items():
    make_latex(v, k)

TASK: PREDICTING BINARY WILL-BE-SPEAKING LABEL (2 classes: speaking or not)
***************************************************************************



\begin{table*}[t]

\setlength\tabcolsep{2pt} % Makes table columns tighter
\renewcommand{\arraystretch}{0.9}% Makes table rows tighter
\caption{Test accuracy with Gaussian Naive Bayes predicting whether or not the host will be speaking in 1-10 seconds  using everyone's features. Rows reflect history used. Columns comprise features used and how far in the future we predict. Max score for each (past, future) pair is in bold.}
\vskip -0.1in
\label{table:Gaussian-Naive-Bayes_True_True_False}
\begin{center}
\begin{small}
\begin{sc}
\resizebox{\textwidth}{!}{ %Completely zooms in or zooms out (shrinks) entire table!

\begin{tabular}{l|ccc|ccc|ccc|ccc|ccc|ccc|ccc}
\toprule
\multicolumn{4}{c}{Future (s),\textbf{text}} & \multicolumn{3}{c}{\textbf{video}} & \multicolumn{3}{c}{\textbf{audio}} & \multicolumn{3}{c}{\textbf{text-video}} & \multic

# Final tables

In [976]:
title = "TASK: PREDICTING BINARY WILL-BE-SPEAKING LABEL (2 classes: speaking or not)"
print(title + "\n" + "*"*len(title) + "\n")
all_results = {"use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6234794361845917),   (('text', 2), 0.5829943831105946),   (('text', 4), 0.5665436176413444),   (('text', 9), 0.539455649109066),   (('video', 0), 0.5514578103881058),   (('video', 2), 0.536897152818129),   (('video', 4), 0.534874684282106),   (('video', 9), 0.5337771685921284),   (('voxaudio', 0), 0.6481946321683723),   (('voxaudio', 2), 0.5953902769707534),   (('voxaudio', 4), 0.5686807849232562),   (('voxaudio', 9), 0.5402388878010573),   (('text_video', 0), 0.5945163158910987),   (('text_video', 2), 0.5614952546968817),   (('text_video', 4), 0.555469205362347),   (('text_video', 9), 0.5435676522420207),   (('text_voxaudio', 0), 0.6570766557250435),   (('text_voxaudio', 2), 0.5990703079604881),   (('text_voxaudio', 4), 0.5811152127452885),   (('text_voxaudio', 9), 0.5461131779909928),   (('video_voxaudio', 0), 0.6346785093647422),   (('video_voxaudio', 2), 0.5814448963780747),   (('video_voxaudio', 4), 0.5686807849232562),   (('video_voxaudio', 9), 0.5615821421578227),   (('text_video_voxaudio', 0), 0.6420158331724272),   (('text_video_voxaudio', 2), 0.590548130931629),   (('text_video_voxaudio', 4), 0.5772294540509034),   (('text_video_voxaudio', 9), 0.5611905228118269)],  5: [(('text', 0), 0.6183473389355743),   (('text', 2), 0.5660112359550562),   (('text', 4), 0.5521126760563381),   (('text', 9), 0.5132978723404256),   (('video', 0), 0.5689775910364145),   (('video', 2), 0.5626755617977528),   (('video', 4), 0.5619718309859155),   (('video', 9), 0.5606382978723404),   (('voxaudio', 0), 0.6411064425770309),   (('voxaudio', 2), 0.5935744382022472),   (('voxaudio', 4), 0.5732394366197183),   (('voxaudio', 9), 0.5452127659574468),   (('text_video', 0), 0.601890756302521),   (('text_video', 2), 0.5784761235955056),   (('text_video', 4), 0.5767605633802817),   (('text_video', 9), 0.5695035460992908),   (('text_voxaudio', 0), 0.6502100840336135),   (('text_voxaudio', 2), 0.598314606741573),   (('text_voxaudio', 4), 0.5765845070422535),   (('text_voxaudio', 9), 0.5427304964539007),   (('video_voxaudio', 0), 0.6260504201680672),   (('video_voxaudio', 2), 0.5953300561797753),   (('video_voxaudio', 4), 0.5802816901408451),   (('video_voxaudio', 9), 0.573936170212766),   (('text_video_voxaudio', 0), 0.6391806722689075),   (('text_video_voxaudio', 2), 0.6014747191011236),   (('text_video_voxaudio', 4), 0.5894366197183099),   (('text_video_voxaudio', 9), 0.573758865248227)],  10: [(('text', 0), 0.578984485190409),   (('text', 2), 0.5477369165487977),   (('text', 4), 0.5342198581560283),   (('text', 9), 0.5141071428571429),   (('video', 0), 0.5756346967559943),   (('video', 2), 0.570898161244696),   (('video', 4), 0.5656028368794326),   (('video', 9), 0.54625),   (('voxaudio', 0), 0.6126586741889986),   (('voxaudio', 2), 0.579031117397454),   (('voxaudio', 4), 0.5668439716312057),   (('voxaudio', 9), 0.55375),   (('text_video', 0), 0.6004936530324401),   (('text_video', 2), 0.5774398868458275),   (('text_video', 4), 0.574645390070922),   (('text_video', 9), 0.5510714285714285),   (('text_voxaudio', 0), 0.6140691114245416),   (('text_voxaudio', 2), 0.5756718528995757),   (('text_voxaudio', 4), 0.5641843971631205),   (('text_voxaudio', 9), 0.54625),   (('video_voxaudio', 0), 0.610543018335684),   (('video_voxaudio', 2), 0.5868104667609618),   (('video_voxaudio', 4), 0.5822695035460993),   (('video_voxaudio', 9), 0.5682142857142857),   (('text_video_voxaudio', 0), 0.6160084626234132),   (('text_video_voxaudio', 2), 0.5885785007072136),   (('text_video_voxaudio', 4), 0.5856382978723405),   (('text_video_voxaudio', 9), 0.5716071428571429)],  30: [(('text', 0), 0.5665820029027576),   (('text', 2), 0.5598617176128093),   (('text', 4), 0.5505474452554745),   (('text', 9), 0.5303308823529411),   (('video', 0), 0.5609579100145138),   (('video', 2), 0.5576783114992722),   (('video', 4), 0.5525547445255474),   (('video', 9), 0.546875),   (('voxaudio', 0), 0.5809143686502177),   (('voxaudio', 2), 0.5589519650655022),   (('voxaudio', 4), 0.5572992700729927),   (('voxaudio', 9), 0.5466911764705882),   (('text_video', 0), 0.5732946298984035),   (('text_video', 2), 0.5633187772925764),   (('text_video', 4), 0.5565693430656934),   (('text_video', 9), 0.5479779411764706),   (('text_voxaudio', 0), 0.5725689404934688),   (('text_voxaudio', 2), 0.5540393013100436),   (('text_voxaudio', 4), 0.5516423357664234),   (('text_voxaudio', 9), 0.5347426470588236),   (('video_voxaudio', 0), 0.5792815674891146),   (('video_voxaudio', 2), 0.5680494905385735),   (('video_voxaudio', 4), 0.5602189781021898),   (('video_voxaudio', 9), 0.5582720588235294),   (('text_video_voxaudio', 0), 0.5820029027576198),   (('text_video_voxaudio', 2), 0.5722343522561864),   (('text_video_voxaudio', 4), 0.5638686131386861),   (('text_video_voxaudio', 9), 0.5512867647058823)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6452951346011361),   (('text', 2), 0.5723270440251572),   (('text', 4), 0.5716697284146766),   (('text', 9), 0.551296786767278),   (('video', 0), 0.5657199308471228),   (('video', 2), 0.5463279354231665),   (('video', 4), 0.5419790477136189),   (('video', 9), 0.5399030533206737),   (('voxaudio', 0), 0.6749320819955544),   (('voxaudio', 2), 0.6355172584559006),   (('voxaudio', 4), 0.6221637455935654),   (('voxaudio', 9), 0.6089650692119334),   (('text_video', 0), 0.62904420844653),   (('text_video', 2), 0.5894616946466598),   (('text_video', 4), 0.5778759743806167),   (('text_video', 9), 0.5564939283394134),   (('text_voxaudio', 0), 0.6722647567300568),   (('text_voxaudio', 2), 0.6107066805328579),   (('text_voxaudio', 4), 0.6065240057593962),   (('text_voxaudio', 9), 0.5909749637699265),   (('video_voxaudio', 0), 0.6466287972338849),   (('video_voxaudio', 2), 0.6116476006536918),   (('video_voxaudio', 4), 0.5970408619234397),   (('video_voxaudio', 9), 0.576582879416321),   (('text_video_voxaudio', 0), 0.6637194368979995),   (('text_video_voxaudio', 2), 0.6155103253602734),   (('text_video_voxaudio', 4), 0.604885556824388),   (('text_video_voxaudio', 9), 0.5822797461396232)],  5: [(('text', 0), 0.6498554913294797),   (('text', 2), 0.6086536604210933),   (('text', 4), 0.5914988381099923),   (('text', 9), 0.5682061775309364),   (('video', 0), 0.5648843930635838),   (('video', 2), 0.5517191423604404),   (('video', 4), 0.5476374903175832),   (('video', 9), 0.5464289194192731),   (('voxaudio', 0), 0.6700385356454721),   (('voxaudio', 2), 0.6368553216148348),   (('voxaudio', 4), 0.6249515879163439),   (('voxaudio', 9), 0.6140017538731365),   (('text_video', 0), 0.6205202312138728),   (('text_video', 2), 0.5928626617732278),   (('text_video', 4), 0.5784759876065065),   (('text_video', 9), 0.5630907142161161),   (('text_voxaudio', 0), 0.6660404624277456),   (('text_voxaudio', 2), 0.6283079003283755),   (('text_voxaudio', 4), 0.6132358636715725),   (('text_voxaudio', 9), 0.591688590080873),   (('video_voxaudio', 0), 0.6435452793834296),   (('video_voxaudio', 2), 0.6117442534286266),   (('video_voxaudio', 4), 0.5979376452362509),   (('video_voxaudio', 9), 0.5822858813212511),   (('text_video_voxaudio', 0), 0.6624277456647398),   (('text_video_voxaudio', 2), 0.6268591848560943),   (('text_video_voxaudio', 4), 0.6103795507358637),   (('text_video_voxaudio', 9), 0.5895449673584722)],  10: [(('text', 0), 0.5935046049442559),   (('text', 2), 0.5626397123141219),   (('text', 4), 0.5499853843905291),   (('text', 9), 0.4525887428907629),   (('video', 0), 0.5523994183228308),   (('video', 2), 0.5373214112158616),   (('video', 4), 0.5342005261619409),   (('video', 9), 0.5295646205138262),   (('voxaudio', 0), 0.6484730974309258),   (('voxaudio', 2), 0.6339780347944407),   (('voxaudio', 4), 0.6300302055929066),   (('voxaudio', 9), 0.6314963718376152),   (('text_video', 0), 0.5888027144934561),   (('text_video', 2), 0.5645349402274273),   (('text_video', 4), 0.5576829387118776),   (('text_video', 9), 0.5395665816826829),   (('text_voxaudio', 0), 0.6113427047988367),   (('text_voxaudio', 2), 0.5836816017105647),   (('text_voxaudio', 4), 0.5723959855792653),   (('text_voxaudio', 9), 0.5500098058442832),   (('video_voxaudio', 0), 0.6206010664081435),   (('video_voxaudio', 2), 0.6008358441053553),   (('video_voxaudio', 4), 0.5911039657020365),   (('video_voxaudio', 9), 0.5808491861149245),   (('text_video_voxaudio', 0), 0.6283082888996607),   (('text_video_voxaudio', 2), 0.600155505880066),   (('text_video_voxaudio', 4), 0.5913475591932184),   (('text_video_voxaudio', 9), 0.5694744067464209)],  30: [(('text', 0), 0.4972650422675286),   (('text', 2), 0.4849935187954931),   (('text', 4), 0.4789063281015695),   (('text', 9), 0.4606560676192393),   (('video', 0), 0.5436101442068623),   (('video', 2), 0.5328547213082062),   (('video', 4), 0.5358392482255323),   (('video', 9), 0.5361742805393439),   (('voxaudio', 0), 0.6586772749875683),   (('voxaudio', 2), 0.6546016551999202),   (('voxaudio', 4), 0.6517544736579026),   (('voxaudio', 9), 0.6398671764942644),   (('text_video', 0), 0.5501740427647936),   (('text_video', 2), 0.5341509622095921),   (('text_video', 4), 0.5299410176946916),   (('text_video', 9), 0.5191185349164822),   (('text_voxaudio', 0), 0.5317752362008951),   (('text_voxaudio', 2), 0.5219862399042776),   (('text_voxaudio', 4), 0.5228431470558832),   (('text_voxaudio', 9), 0.5329040048299457),   (('video_voxaudio', 0), 0.6037792143212333),   (('video_voxaudio', 2), 0.5998105494067205),   (('video_voxaudio', 4), 0.5993202039388184),   (('video_voxaudio', 9), 0.5982592070839203),   (('text_video_voxaudio', 0), 0.5816011934361014),   (('text_video_voxaudio', 2), 0.5676039485492073),   (('text_video_voxaudio', 4), 0.5645306408077577),   (('text_video_voxaudio', 9), 0.5621352384785672)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6529232400901498),   (('text', 2), 0.6116943521594684),   (('text', 4), 0.5946449980018649),   (('text', 9), 0.5714285714285714),   (('video', 0), 0.5438154580405674),   (('video', 2), 0.5217275747508305),   (('video', 4), 0.5180498201678434),   (('video', 9), 0.5158134548378451),   (('voxaudio', 0), 0.6604799151531221),   (('voxaudio', 2), 0.6151495016611296),   (('voxaudio', 4), 0.5973091781004396),   (('voxaudio', 9), 0.5733047440364514),   (('text_video', 0), 0.5990985019223121),   (('text_video', 2), 0.5553488372093023),   (('text_video', 4), 0.5504196083655255),   (('text_video', 9), 0.5336370946127044),   (('text_voxaudio', 0), 0.6620707941137478),   (('text_voxaudio', 2), 0.6200664451827242),   (('text_voxaudio', 4), 0.605568136406021),   (('text_voxaudio', 9), 0.5755829536317342),   (('video_voxaudio', 0), 0.6476203102213973),   (('video_voxaudio', 2), 0.598671096345515),   (('video_voxaudio', 4), 0.5737311842280538),   (('video_voxaudio', 9), 0.5532028946663093),   (('text_video_voxaudio', 0), 0.6554421317778072),   (('text_video_voxaudio', 2), 0.6104983388704319),   (('text_video_voxaudio', 4), 0.596110297056081),   (('text_video_voxaudio', 9), 0.5682122755293487)],  5: [(('text', 0), 0.6466684364215556),   (('text', 2), 0.6101649813730707),   (('text', 4), 0.588957055214724),   (('text', 9), 0.5209982557359453),   (('video', 0), 0.5342447571011415),   (('video', 2), 0.518493879723257),   (('video', 4), 0.5194718591624433),   (('video', 9), 0.5196565141553737),   (('voxaudio', 0), 0.6506503849216884),   (('voxaudio', 2), 0.6113624268227781),   (('voxaudio', 4), 0.5921579087756735),   (('voxaudio', 9), 0.5701059975848651),   (('text_video', 0), 0.5914520838863817),   (('text_video', 2), 0.55401809473124),   (('text_video', 4), 0.5505468124833289),   (('text_video', 9), 0.5320005366966323),   (('text_voxaudio', 0), 0.6531722856384391),   (('text_voxaudio', 2), 0.6137573177221927),   (('text_voxaudio', 4), 0.5956255001333689),   (('text_voxaudio', 9), 0.5718502616396082),   (('video_voxaudio', 0), 0.6359171754711972),   (('video_voxaudio', 2), 0.5882118147951038),   (('video_voxaudio', 4), 0.5686849826620433),   (('video_voxaudio', 9), 0.5455521266604052),   (('text_video_voxaudio', 0), 0.6509158481550306),   (('text_video_voxaudio', 2), 0.6094997339010112),   (('text_video_voxaudio', 4), 0.5878901040277408),   (('text_video_voxaudio', 9), 0.5639339863142359)],  10: [(('text', 0), 0.6067565763119241),   (('text', 2), 0.5819836701914067),   (('text', 4), 0.5693009526365223),   (('text', 9), 0.5295626349892009),   (('video', 0), 0.5219655494725598),   (('video', 2), 0.5170659884888235),   (('video', 4), 0.511471890513887),   (('video', 9), 0.5024298056155507),   (('voxaudio', 0), 0.6263853652022967),   (('voxaudio', 2), 0.5971088207736581),   (('voxaudio', 4), 0.5840601100228096),   (('voxaudio', 9), 0.5599352051835853),   (('text_video', 0), 0.5589531312591801),   (('text_video', 2), 0.5347343059831348),   (('text_video', 4), 0.5211324298940024),   (('text_video', 9), 0.5067494600431965),   (('text_voxaudio', 0), 0.6305247696621712),   (('text_voxaudio', 2), 0.5993842859055013),   (('text_voxaudio', 4), 0.5847309808130954),   (('text_voxaudio', 9), 0.5592602591792657),   (('video_voxaudio', 0), 0.5807183869675524),   (('video_voxaudio', 2), 0.5486547985544104),   (('video_voxaudio', 4), 0.5338789749094325),   (('video_voxaudio', 9), 0.5133639308855291),   (('text_video_voxaudio', 0), 0.6189077313392977),   (('text_video_voxaudio', 2), 0.580779012180431),   (('text_video_voxaudio', 4), 0.5670199919495506),   (('text_video_voxaudio', 9), 0.5267278617710583)],  30: [(('text', 0), 0.566835408400602),   (('text', 2), 0.5506789192154711),   (('text', 4), 0.5495668912415784),   (('text', 9), 0.5355561704482568),   (('video', 0), 0.5097824599808455),   (('video', 2), 0.5052804827869977),   (('video', 4), 0.5024061597690087),   (('video', 9), 0.5051189817376868),   (('voxaudio', 0), 0.5792858120125872),   (('voxaudio', 2), 0.5560279797010013),   (('voxaudio', 4), 0.5484669324900316),   (('voxaudio', 9), 0.5444106253458771),   (('text_video', 0), 0.5363250786701327),   (('text_video', 2), 0.5155671375668632),   (('text_video', 4), 0.5083184380585728),   (('text_video', 9), 0.5019369120088545),   (('text_voxaudio', 0), 0.5846216992748666),   (('text_voxaudio', 2), 0.562885749554245),   (('text_voxaudio', 4), 0.5580915715660663),   (('text_voxaudio', 9), 0.5462091864969563),   (('video_voxaudio', 0), 0.5359146258038036),   (('video_voxaudio', 2), 0.5150185159786037),   (('video_voxaudio', 4), 0.5099683761858931),   (('video_voxaudio', 9), 0.5023519645821805),   (('text_video_voxaudio', 0), 0.5709399370638938),   (('text_video_voxaudio', 2), 0.5372376903031134),   (('text_video_voxaudio', 4), 0.5195930152619277),   (('text_video_voxaudio', 9), 0.5005534034311012)]}, "use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6539872562270709),   (('text', 2), 0.5961650203370134),   (('text', 4), 0.5805323489411307),   (('text', 9), 0.5490503230859605),   (('video', 0), 0.5864066422089207),   (('video', 2), 0.5649815998450514),   (('video', 4), 0.5490577035166117),   (('video', 9), 0.5392598394360681),   (('voxaudio', 0), 0.6472291948252559),   (('voxaudio', 2), 0.5930660468719736),   (('voxaudio', 4), 0.5748979988342724),   (('voxaudio', 9), 0.558449187389857),   (('text_video', 0), 0.6460706700135161),   (('text_video', 2), 0.5963587061785783),   (('text_video', 4), 0.5745094229648339),   (('text_video', 9), 0.5421969845310358),   (('text_voxaudio', 0), 0.6566904807877969),   (('text_voxaudio', 2), 0.6029440247917878),   (('text_voxaudio', 4), 0.5758694385078686),   (('text_voxaudio', 9), 0.5533581358919131),   (('video_voxaudio', 0), 0.653407993821201),   (('video_voxaudio', 2), 0.5979081929110982),   (('video_voxaudio', 4), 0.5702350884010103),   (('video_voxaudio', 9), 0.5568827100058743),   (('text_video_voxaudio', 0), 0.6441397953272833),   (('text_video_voxaudio', 2), 0.6048808832074375),   (('text_video_voxaudio', 4), 0.5733436953565184),   (('text_video_voxaudio', 9), 0.5578617583708635)],  5: [(('text', 0), 0.6465336134453782),   (('text', 2), 0.5967345505617978),   (('text', 4), 0.5725352112676056),   (('text', 9), 0.5574468085106383),   (('video', 0), 0.5835084033613446),   (('video', 2), 0.570751404494382),   (('video', 4), 0.5670774647887324),   (('video', 9), 0.5585106382978723),   (('voxaudio', 0), 0.6414565826330533),   (('voxaudio', 2), 0.6004213483146067),   (('voxaudio', 4), 0.5746478873239437),   (('voxaudio', 9), 0.5570921985815603),   (('text_video', 0), 0.6376050420168067),   (('text_video', 2), 0.5969101123595506),   (('text_video', 4), 0.5806338028169014),   (('text_video', 9), 0.5585106382978723),   (('text_voxaudio', 0), 0.6475840336134454),   (('text_voxaudio', 2), 0.6053370786516854),   (('text_voxaudio', 4), 0.585387323943662),   (('text_voxaudio', 9), 0.5593971631205674),   (('video_voxaudio', 0), 0.6432072829131653),   (('video_voxaudio', 2), 0.5956811797752809),   (('video_voxaudio', 4), 0.5748239436619719),   (('video_voxaudio', 9), 0.5539007092198581),   (('text_video_voxaudio', 0), 0.6465336134453782),   (('text_video_voxaudio', 2), 0.601123595505618),   (('text_video_voxaudio', 4), 0.5737676056338028),   (('text_video_voxaudio', 9), 0.5585106382978723)],  10: [(('text', 0), 0.6216502115655853),   (('text', 2), 0.5795615275813296),   (('text', 4), 0.5627659574468085),   (('text', 9), 0.5480357142857143),   (('video', 0), 0.590620592383639),   (('video', 2), 0.5694837340876945),   (('video', 4), 0.5666666666666667),   (('video', 9), 0.5523214285714285),   (('voxaudio', 0), 0.6167136812411848),   (('voxaudio', 2), 0.5815063649222065),   (('voxaudio', 4), 0.5732269503546099),   (('voxaudio', 9), 0.5582142857142857),   (('text_video', 0), 0.6100141043723555),   (('text_video', 2), 0.5781471004243282),   (('text_video', 4), 0.5673758865248227),   (('text_video', 9), 0.5521428571428572),   (('text_voxaudio', 0), 0.6242947813822285),   (('text_voxaudio', 2), 0.5949434229137199),   (('text_voxaudio', 4), 0.5762411347517731),   (('text_voxaudio', 9), 0.5592857142857143),   (('video_voxaudio', 0), 0.6174188998589563),   (('video_voxaudio', 2), 0.5848656294200849),   (('video_voxaudio', 4), 0.5776595744680851),   (('video_voxaudio', 9), 0.5623214285714285),   (('text_video_voxaudio', 0), 0.6161847672778561),   (('text_video_voxaudio', 2), 0.5878712871287128),   (('text_video_voxaudio', 4), 0.5792553191489361),   (('text_video_voxaudio', 9), 0.5655357142857143)],  30: [(('text', 0), 0.5693033381712627),   (('text', 2), 0.5516739446870451),   (('text', 4), 0.5474452554744526),   (('text', 9), 0.5479779411764706),   (('video', 0), 0.5662191582002902),   (('video', 2), 0.5609534206695779),   (('video', 4), 0.5490875912408759),   (('video', 9), 0.5424632352941177),   (('voxaudio', 0), 0.5705732946298984),   (('voxaudio', 2), 0.5591339155749636),   (('voxaudio', 4), 0.5593065693430657),   (('voxaudio', 9), 0.5424632352941177),   (('text_video', 0), 0.5749274310595065),   (('text_video', 2), 0.5587700145560408),   (('text_video', 4), 0.5541970802919708),   (('text_video', 9), 0.5564338235294117),   (('text_voxaudio', 0), 0.5702104499274311),   (('text_voxaudio', 2), 0.5531295487627366),   (('text_voxaudio', 4), 0.5598540145985401),   (('text_voxaudio', 9), 0.5481617647058824),   (('video_voxaudio', 0), 0.579644412191582),   (('video_voxaudio', 2), 0.5684133915574964),   (('video_voxaudio', 4), 0.5534671532846716),   (('video_voxaudio', 9), 0.5367647058823529),   (('text_video_voxaudio', 0), 0.5747460087082729),   (('text_video_voxaudio', 2), 0.5704148471615721),   (('text_video_voxaudio', 4), 0.5574817518248175),   (('text_video_voxaudio', 9), 0.5516544117647059)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6985922449987652),   (('text', 2), 0.6741445055217155),   (('text', 4), 0.6656571173228737),   (('text', 9), 0.6512418170006496),   (('video', 0), 0.6558162509261546),   (('video', 2), 0.6499281929381469),   (('video', 4), 0.6475348791023285),   (('video', 9), 0.647044125730848),   (('voxaudio', 0), 0.7197826623857743),   (('voxaudio', 2), 0.6952409250730451),   (('voxaudio', 4), 0.6834814557370538),   (('voxaudio', 9), 0.6702313727449902),   (('text_video', 0), 0.7045690293899728),   (('text_video', 2), 0.6796414599118507),   (('text_video', 4), 0.6708703639342635),   (('text_video', 9), 0.6503423117285493),   (('text_voxaudio', 0), 0.7202272165966905),   (('text_voxaudio', 2), 0.6930619521616401),   (('text_voxaudio', 4), 0.6864604538007051),   (('text_voxaudio', 9), 0.6727799710159412),   (('video_voxaudio', 0), 0.7134107186959743),   (('video_voxaudio', 2), 0.689694448571287),   (('video_voxaudio', 4), 0.6839283054466014),   (('video_voxaudio', 9), 0.6708810154415071),   (('text_video_voxaudio', 0), 0.7176092862435169),   (('text_video_voxaudio', 2), 0.6905363244688754),   (('text_video_voxaudio', 4), 0.6823395064793207),   (('text_video_voxaudio', 9), 0.6734795862275749)],  5: [(('text', 0), 0.6970616570327552),   (('text', 2), 0.6732663704848367),   (('text', 4), 0.6624709527498064),   (('text', 9), 0.6548280229952256),   (('video', 0), 0.6675818882466281),   (('video', 2), 0.6577168244156848),   (('video', 4), 0.6528853601859024),   (('video', 9), 0.6511254019292605),   (('voxaudio', 0), 0.7185934489402698),   (('voxaudio', 2), 0.6979428240293606),   (('voxaudio', 4), 0.6874515879163439),   (('voxaudio', 9), 0.6733411283250511),   (('text_video', 0), 0.7030346820809249),   (('text_video', 2), 0.6834073787908055),   (('text_video', 4), 0.6802381874515879),   (('text_video', 9), 0.6629153269024651),   (('text_voxaudio', 0), 0.7171001926782273),   (('text_voxaudio', 2), 0.6978462429978752),   (('text_voxaudio', 4), 0.6895817195972115),   (('text_voxaudio', 9), 0.6751437201597973),   (('video_voxaudio', 0), 0.7133429672447014),   (('video_voxaudio', 2), 0.6921479621402357),   (('video_voxaudio', 4), 0.6852730441518203),   (('video_voxaudio', 9), 0.6729026600409237),   (('text_video_voxaudio', 0), 0.7160886319845857),   (('text_video_voxaudio', 2), 0.6949971025690554),   (('text_video_voxaudio', 4), 0.685708752904725),   (('text_video_voxaudio', 9), 0.6748026892721426)],  10: [(('text', 0), 0.692195831313621),   (('text', 2), 0.6704247254349305),   (('text', 4), 0.6631102016954107),   (('text', 9), 0.652235732496568),   (('video', 0), 0.6529326223945711),   (('video', 2), 0.6511808727767519),   (('video', 4), 0.6497612783786417),   (('video', 9), 0.6486565993332026),   (('voxaudio', 0), 0.7106156083373728),   (('voxaudio', 2), 0.6960346000583147),   (('voxaudio', 4), 0.6876157069083114),   (('voxaudio', 9), 0.6737105314767602),   (('text_video', 0), 0.6936984973339796),   (('text_video', 2), 0.6765963650500535),   (('text_video', 4), 0.6596511741206275),   (('text_video', 9), 0.6500784467542655),   (('text_voxaudio', 0), 0.709112942317014),   (('text_voxaudio', 2), 0.6964719603460006),   (('text_voxaudio', 4), 0.6856669589788561),   (('text_voxaudio', 9), 0.6750833496764072),   (('video_voxaudio', 0), 0.7064469219583132),   (('video_voxaudio', 2), 0.6894256001555059),   (('video_voxaudio', 4), 0.6824515248952548),   (('video_voxaudio', 9), 0.6682682878995881),   (('text_video_voxaudio', 0), 0.7106640814348036),   (('text_video_voxaudio', 2), 0.6919039751190592),   (('text_video_voxaudio', 4), 0.6860079898665108),   (('text_video_voxaudio', 9), 0.6716022749558737)],  30: [(('text', 0), 0.6658876181004475),   (('text', 2), 0.6523083059128527),   (('text', 4), 0.6493052084374688),   (('text', 9), 0.647967397866774),   (('video', 0), 0.6552958727001492),   (('video', 2), 0.6520091733971483),   (('video', 4), 0.6522543237028892),   (('video', 9), 0.6492251962165425),   (('voxaudio', 0), 0.6872202884137245),   (('voxaudio', 2), 0.6780337022634361),   (('voxaudio', 4), 0.6726482055383385),   (('voxaudio', 9), 0.663614409337895),   (('text_video', 0), 0.6661859771258081),   (('text_video', 2), 0.6595871971283278),   (('text_video', 4), 0.658002599220234),   (('text_video', 9), 0.6545079492855705),   (('text_voxaudio', 0), 0.6894579811039284),   (('text_voxaudio', 2), 0.6811745936783329),   (('text_voxaudio', 4), 0.6743976806957913),   (('text_voxaudio', 9), 0.6663815657073858),   (('video_voxaudio', 0), 0.6819492789656887),   (('video_voxaudio', 2), 0.6795792202612424),   (('video_voxaudio', 4), 0.6737978606418075),   (('video_voxaudio', 9), 0.663211913865969),   (('text_video_voxaudio', 0), 0.6848831427150671),   (('text_video_voxaudio', 2), 0.6779339914248679),   (('text_video_voxaudio', 4), 0.676447065880236),   (('text_video_voxaudio', 9), 0.6611491245723485)]}, "use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6559724247646825),   (('text', 2), 0.6112956810631229),   (('text', 4), 0.5971759690955109),   (('text', 9), 0.5722326454033771),   (('video', 0), 0.609041495426223),   (('video', 2), 0.575548172757475),   (('video', 4), 0.5585453576661782),   (('video', 9), 0.5533369069954436),   (('voxaudio', 0), 0.6692297494365637),   (('voxaudio', 2), 0.6235215946843854),   (('voxaudio', 4), 0.5974423871053683),   (('voxaudio', 9), 0.5767890645939426),   (('text_video', 0), 0.6569004374917141),   (('text_video', 2), 0.6175415282392026),   (('text_video', 4), 0.5997069401891568),   (('text_video', 9), 0.5809434467971053),   (('text_voxaudio', 0), 0.6758584117725043),   (('text_voxaudio', 2), 0.6256478405315614),   (('text_voxaudio', 4), 0.6045024643665912),   (('text_voxaudio', 9), 0.5802733851514339),   (('video_voxaudio', 0), 0.6635290998276547),   (('video_voxaudio', 2), 0.6194019933554817),   (('video_voxaudio', 4), 0.6034367923271613),   (('video_voxaudio', 9), 0.5797373358348968),   (('text_video_voxaudio', 0), 0.6697600424234389),   (('text_video_voxaudio', 2), 0.6256478405315614),   (('text_video_voxaudio', 4), 0.6104968695883842),   (('text_video_voxaudio', 9), 0.588448137228625)],  5: [(('text', 0), 0.6491903371383063),   (('text', 2), 0.6024481106971793),   (('text', 4), 0.5904241130968259),   (('text', 9), 0.5765463571716087),   (('video', 0), 0.6096363153703213),   (('video', 2), 0.5717136774880256),   (('video', 4), 0.55841557748733),   (('video', 9), 0.545417952502348),   (('voxaudio', 0), 0.6645872046721529),   (('voxaudio', 2), 0.6265300691857371),   (('voxaudio', 4), 0.608829021072286),   (('voxaudio', 9), 0.5752046155910372),   (('text_video', 0), 0.6525086275550837),   (('text_video', 2), 0.6152208621607238),   (('text_video', 4), 0.5945585489463857),   (('text_video', 9), 0.5734603515362942),   (('text_voxaudio', 0), 0.6693655428723122),   (('text_voxaudio', 2), 0.6263970196913252),   (('text_voxaudio', 4), 0.6041611096292344),   (('text_voxaudio', 9), 0.5872802898161814),   (('video_voxaudio', 0), 0.661667109105389),   (('video_voxaudio', 2), 0.6193453964874933),   (('video_voxaudio', 4), 0.6002934115764204),   (('video_voxaudio', 9), 0.5836575875486382),   (('text_video_voxaudio', 0), 0.6677727634722591),   (('text_video_voxaudio', 2), 0.6246673762639702),   (('text_video_voxaudio', 4), 0.6041611096292344),   (('text_video_voxaudio', 9), 0.588622031396753)],  10: [(('text', 0), 0.6215783148617973),   (('text', 2), 0.5934948467407308),   (('text', 4), 0.5906346437676103),   (('text', 9), 0.5715442764578834),   (('video', 0), 0.5780478034450528),   (('video', 2), 0.5548119394993977),   (('video', 4), 0.5475647390312626),   (('video', 9), 0.5265928725701944),   (('voxaudio', 0), 0.6418747496327948),   (('voxaudio', 2), 0.6141078838174274),   (('voxaudio', 4), 0.598819267409097),   (('voxaudio', 9), 0.5728941684665226),   (('text_video', 0), 0.631993590599546),   (('text_video', 2), 0.5991165841252845),   (('text_video', 4), 0.5698376492687508),   (('text_video', 9), 0.5322624190064795),   (('text_voxaudio', 0), 0.6490853251435439),   (('text_voxaudio', 2), 0.6189265158613305),   (('text_voxaudio', 4), 0.59962431235744),   (('text_voxaudio', 9), 0.5724892008639308),   (('video_voxaudio', 0), 0.6384029910535453),   (('video_voxaudio', 2), 0.6141078838174274),   (('video_voxaudio', 4), 0.5960016100898967),   (('video_voxaudio', 9), 0.5602051835853131),   (('text_video_voxaudio', 0), 0.6497529710241687),   (('text_video_voxaudio', 2), 0.6099585062240664),   (('text_video_voxaudio', 4), 0.5989534415671541),   (('text_video_voxaudio', 9), 0.5673596112311015)],  30: [(('text', 0), 0.5550690928991654),   (('text', 2), 0.5542449595391579),   (('text', 4), 0.5580915715660663),   (('text', 9), 0.5532650802434975),   (('video', 0), 0.5742235599945273),   (('video', 2), 0.5579481552599095),   (('video', 4), 0.5596040148494431),   (('video', 9), 0.5385998893193138),   (('voxaudio', 0), 0.600492543439595),   (('voxaudio', 2), 0.5840076807022356),   (('voxaudio', 4), 0.5667537467344974),   (('voxaudio', 9), 0.5534034311012729),   (('text_video', 0), 0.6026816253933507),   (('text_video', 2), 0.5962145110410094),   (('text_video', 4), 0.5695036436133645),   (('text_video', 9), 0.5485611510791367),   (('text_voxaudio', 0), 0.6029552606375701),   (('text_voxaudio', 2), 0.5856535454670141),   (('text_voxaudio', 4), 0.5763783858105321),   (('text_voxaudio', 9), 0.5534034311012729),   (('video_voxaudio', 0), 0.5992611848406074),   (('video_voxaudio', 2), 0.5829104375257166),   (('video_voxaudio', 4), 0.5798157569091159),   (('video_voxaudio', 9), 0.5532650802434975),   (('text_video_voxaudio', 0), 0.6009029963059243),   (('text_video_voxaudio', 2), 0.5933342477026471),   (('text_video_voxaudio', 4), 0.5866904991062836),   (('text_video_voxaudio', 9), 0.5539568345323741)]}, "use_crossval: True | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6495421578279535),   (('text', 2), 0.5878564811484824),   (('text', 4), 0.5533003759881916),   (('text', 9), 0.5189191297826732),   (('video', 0), 0.5192359413820778),   (('video', 2), 0.5051804947521831),   (('video', 4), 0.4972346271973011),   (('video', 9), 0.49625909688412584),   (('voxaudio', 0), 0.6847674467968973),   (('voxaudio', 2), 0.623508876445786),   (('voxaudio', 4), 0.5934301380226243),   (('voxaudio', 9), 0.5547619737142593),   (('text_video', 0), 0.5598077909248287),   (('text_video', 2), 0.5242564040462163),   (('text_video', 4), 0.5126536453371967),   (('text_video', 9), 0.4989114560751437),   (('text_voxaudio', 0), 0.6856497951795943),   (('text_voxaudio', 2), 0.623586628906604),   (('text_voxaudio', 4), 0.5962215354146921),   (('text_voxaudio', 9), 0.5555759631540683),   (('video_voxaudio', 0), 0.6439362344447342),   (('video_voxaudio', 2), 0.5791904546527735),   (('video_voxaudio', 4), 0.5506154899432978),   (('video_voxaudio', 9), 0.5195239039885351),   (('text_video_voxaudio', 0), 0.6570448111602356),   (('text_video_voxaudio', 2), 0.5884028140972526),   (('text_video_voxaudio', 4), 0.5567203312580375),   (('text_video_voxaudio', 9), 0.5212832267605743)],  5: [(('text', 0), 0.6293332682577489),   (('text', 2), 0.5725381480733935),   (('text', 4), 0.5509984628634959),   (('text', 9), 0.5146047308539099),   (('video', 0), 0.5131997544034145),   (('video', 2), 0.5009821759299846),   (('video', 4), 0.49538954697376675),   (('video', 9), 0.48720497403446644),   (('voxaudio', 0), 0.6781307293621237),   (('voxaudio', 2), 0.6240455462862923),   (('voxaudio', 4), 0.5953664608759018),   (('voxaudio', 9), 0.556584695911876),   (('text_video', 0), 0.5485174753508428),   (('text_video', 2), 0.5159269842859872),   (('text_video', 4), 0.5053696658950353),   (('text_video', 9), 0.49167687778685903),   (('text_voxaudio', 0), 0.6758935655362045),   (('text_voxaudio', 2), 0.6219146645590208),   (('text_voxaudio', 4), 0.5929198799364814),   (('text_voxaudio', 9), 0.5542064601463389),   (('video_voxaudio', 0), 0.6367992693470291),   (('video_voxaudio', 2), 0.5803327672389804),   (('video_voxaudio', 4), 0.5515348460754879),   (('video_voxaudio', 9), 0.5148602571679588),   (('text_video_voxaudio', 0), 0.6443497027146645),   (('text_video_voxaudio', 2), 0.5869780437216483),   (('text_video_voxaudio', 4), 0.5575232676128528),   (('text_video_voxaudio', 9), 0.5180867523869178)],  10: [(('text', 0), 0.6178053693988834),   (('text', 2), 0.5818258581896032),   (('text', 4), 0.5585227779257285),   (('text', 9), 0.5312027592742684),   (('video', 0), 0.510017427611629),   (('video', 2), 0.5000424730897097),   (('video', 4), 0.4949456954532815),   (('video', 9), 0.49452904971774253),   (('voxaudio', 0), 0.6384634536644452),   (('voxaudio', 2), 0.598868891473415),   (('voxaudio', 4), 0.5768354206541544),   (('voxaudio', 9), 0.5456873438450707),   (('text_video', 0), 0.5604133805155787),   (('text_video', 2), 0.5263519468901487),   (('text_video', 4), 0.5140485921163134),   (('text_video', 9), 0.5038823293744128),   (('text_voxaudio', 0), 0.6388698291130707),   (('text_voxaudio', 2), 0.5970204153245715),   (('text_voxaudio', 4), 0.5751184652218585),   (('text_voxaudio', 9), 0.5435472004175821),   (('video_voxaudio', 0), 0.605389037026129),   (('video_voxaudio', 2), 0.5620124601398042),   (('video_voxaudio', 4), 0.5398553919339418),   (('video_voxaudio', 9), 0.5178999911351139),   (('text_video_voxaudio', 0), 0.6141996704576366),   (('text_video_voxaudio', 2), 0.5720691790307317),   (('text_video_voxaudio', 4), 0.5493119402730458),   (('text_video_voxaudio', 9), 0.5231664349198406)],  30: [(('text', 0), 0.5786259010933945),   (('text', 2), 0.5620501700560834),   (('text', 4), 0.5459432048681542),   (('text', 9), 0.5281201234374271),   (('video', 0), 0.4919217145786135),   (('video', 2), 0.48577166850293196),   (('video', 4), 0.484370943842948),   (('video', 9), 0.4862690727222514),   (('voxaudio', 0), 0.5708241681394307),   (('voxaudio', 2), 0.5476818879818169),   (('voxaudio', 4), 0.5343189422756695),   (('voxaudio', 9), 0.5137034411128298),   (('text_video', 0), 0.5322858197359758),   (('text_video', 2), 0.5145081748940191),   (('text_video', 4), 0.5062856580242049),   (('text_video', 9), 0.4901602111269184),   (('text_voxaudio', 0), 0.5805626102181413),   (('text_voxaudio', 2), 0.5579727078321679),   (('text_voxaudio', 4), 0.5434962617806927),   (('text_voxaudio', 9), 0.5211499126398458),   (('video_voxaudio', 0), 0.5331987019395331),   (('video_voxaudio', 2), 0.5128989789300309),   (('video_voxaudio', 4), 0.5040330469631348),   (('video_voxaudio', 9), 0.4883124401513572),   (('text_video_voxaudio', 0), 0.5504345663224062),   (('text_video_voxaudio', 2), 0.5306792466098086),   (('text_video_voxaudio', 4), 0.51727122597475),   (('text_video_voxaudio', 9), 0.49497493921464975)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.671555662971522),   (('text', 2), 0.6123672704559651),   (('text', 4), 0.599591738660769),   (('text', 9), 0.5748549256198585),   (('video', 0), 0.5576084483926695),   (('video', 2), 0.5461145607524709),   (('video', 4), 0.542276239221862),   (('video', 9), 0.5310812002880885),   (('voxaudio', 0), 0.7031173301892262),   (('voxaudio', 2), 0.6603813792850057),   (('voxaudio', 4), 0.6436909856762745),   (('voxaudio', 9), 0.625969299650994),   (('text_video', 0), 0.6414308004973109),   (('text_video', 2), 0.6015064114340302),   (('text_video', 4), 0.585388173515563),   (('text_video', 9), 0.5584102798197941),   (('text_voxaudio', 0), 0.6990200529449961),   (('text_voxaudio', 2), 0.643935775434471),   (('text_voxaudio', 4), 0.6280211827849997),   (('text_voxaudio', 9), 0.6061136811592092),   (('video_voxaudio', 0), 0.6645193458483327),   (('video_voxaudio', 2), 0.6246463607304259),   (('video_voxaudio', 4), 0.6089922861517474),   (('video_voxaudio', 9), 0.5845457285507417),   (('text_video_voxaudio', 0), 0.6900486348399804),   (('text_video_voxaudio', 2), 0.6407980306426131),   (('text_video_voxaudio', 4), 0.6231443382952732),   (('text_video_voxaudio', 9), 0.5955040202445318)],  5: [(('text', 0), 0.6768598986991343),   (('text', 2), 0.6358361885295272),   (('text', 4), 0.6157670525686695),   (('text', 9), 0.5903164393472323),   (('video', 0), 0.5497097132219093),   (('video', 2), 0.538431091174597),   (('video', 4), 0.5336887848726968),   (('video', 9), 0.5216805206621352),   (('voxaudio', 0), 0.6997071686488399),   (('voxaudio', 2), 0.6626996172497414),   (('voxaudio', 4), 0.6455940272865319),   (('voxaudio', 9), 0.6282407319458313),   (('text_video', 0), 0.6193035772083382),   (('text_video', 2), 0.5861956819922309),   (('text_video', 4), 0.5717011667920494),   (('text_video', 9), 0.54240555986723),   (('text_voxaudio', 0), 0.6949345463957047),   (('text_voxaudio', 2), 0.6543679143796668),   (('text_voxaudio', 4), 0.6347353029760334),   (('text_voxaudio', 9), 0.6090486509604645),   (('video_voxaudio', 0), 0.6533998675490608),   (('video_voxaudio', 2), 0.618162936158367),   (('video_voxaudio', 4), 0.6015280675745843),   (('video_voxaudio', 9), 0.5752786693449676),   (('text_video_voxaudio', 0), 0.678201085546779),   (('text_video_voxaudio', 2), 0.6368692326877674),   (('text_video_voxaudio', 4), 0.6169705213430655),   (('text_video_voxaudio', 9), 0.5841696274003094)],  10: [(('text', 0), 0.5811783428377613),   (('text', 2), 0.523266273394551),   (('text', 4), 0.49142406670047645),   (('text', 9), 0.45736721161926097),   (('video', 0), 0.5408929586124918),   (('video', 2), 0.530378690668295),   (('video', 4), 0.5250830214934218),   (('video', 9), 0.5182381968600402),   (('voxaudio', 0), 0.6798408298789592),   (('voxaudio', 2), 0.6582110517150042),   (('voxaudio', 4), 0.6497171748639153),   (('voxaudio', 9), 0.6374654655431622),   (('text_video', 0), 0.5767013863623583),   (('text_video', 2), 0.552737957725175),   (('text_video', 4), 0.5395351125560695),   (('text_video', 9), 0.5185697731498239),   (('text_voxaudio', 0), 0.642117552145183),   (('text_voxaudio', 2), 0.6137364720771329),   (('text_voxaudio', 4), 0.5982409382357182),   (('text_voxaudio', 9), 0.5687714464323997),   (('video_voxaudio', 0), 0.6347275506172159),   (('video_voxaudio', 2), 0.6115731744271695),   (('video_voxaudio', 4), 0.5986092994389285),   (('video_voxaudio', 9), 0.5819537627177971),   (('text_video_voxaudio', 0), 0.6386857835879949),   (('text_video_voxaudio', 2), 0.6098677277725841),   (('text_video_voxaudio', 4), 0.5930889724087021),   (('text_video_voxaudio', 9), 0.5651757764826092)],  30: [(('text', 0), 0.5229911514822895),   (('text', 2), 0.5063262187497839),   (('text', 4), 0.49713388955774623),   (('text', 9), 0.4801891871949933),   (('video', 0), 0.5305670958281145),   (('video', 2), 0.5242317016369821),   (('video', 4), 0.5226766517726793),   (('video', 9), 0.5208450345767573),   (('voxaudio', 0), 0.6571161993204042),   (('voxaudio', 2), 0.6512731066345603),   (('voxaudio', 4), 0.6473285816123214),   (('voxaudio', 9), 0.6398188130665725),   (('text_video', 0), 0.552516997540663),   (('text_video', 2), 0.5344939702512123),   (('text_video', 4), 0.527451718677359),   (('text_video', 9), 0.5162805620124757),   (('text_voxaudio', 0), 0.565832251054358),   (('text_voxaudio', 2), 0.5562060298449196),   (('text_voxaudio', 4), 0.5529697529402194),   (('text_voxaudio', 9), 0.5563478781742364),   (('video_voxaudio', 0), 0.6093084253641647),   (('video_voxaudio', 2), 0.6045274376866789),   (('video_voxaudio', 4), 0.6015183879038801),   (('video_voxaudio', 9), 0.5949912370783761),   (('text_video_voxaudio', 0), 0.5824986991977159),   (('text_video_voxaudio', 2), 0.5704493027120277),   (('text_video_voxaudio', 4), 0.5656089226061074),   (('text_video_voxaudio', 9), 0.5629714896852759)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.6622354344086439),   (('text', 2), 0.6142788698789332),   (('text', 4), 0.5923248634569396),   (('text', 9), 0.5455600786798769),   (('video', 0), 0.5240508663618886),   (('video', 2), 0.5090461351094048),   (('video', 4), 0.49847979645342244),   (('video', 9), 0.4894046457326583),   (('voxaudio', 0), 0.6771201317494551),   (('voxaudio', 2), 0.6203176995184589),   (('voxaudio', 4), 0.5945618302074402),   (('voxaudio', 9), 0.5617720058872255),   (('text_video', 0), 0.5821584791200417),   (('text_video', 2), 0.5398112164970759),   (('text_video', 4), 0.5205604797468495),   (('text_video', 9), 0.49861578594143746),   (('text_voxaudio', 0), 0.6760991881133678),   (('text_voxaudio', 2), 0.6201332567538473),   (('text_voxaudio', 4), 0.5955056682549278),   (('text_voxaudio', 9), 0.563589632928027),   (('video_voxaudio', 0), 0.6485759811538749),   (('video_voxaudio', 2), 0.5886099146338433),   (('video_voxaudio', 4), 0.5589765688518213),   (('video_voxaudio', 9), 0.5223868888140804),   (('text_video_voxaudio', 0), 0.6583969287919709),   (('text_video_voxaudio', 2), 0.5982510696875998),   (('text_video_voxaudio', 4), 0.5694424937237746),   (('text_video_voxaudio', 9), 0.5298219674420256)],  5: [(('text', 0), 0.6535871545054965),   (('text', 2), 0.6111295208590596),   (('text', 4), 0.568993791104759),   (('text', 9), 0.520637014787835),   (('video', 0), 0.5260404266999892),   (('video', 2), 0.5069270990076472),   (('video', 4), 0.49768385549275845),   (('video', 9), 0.47967816202561026),   (('voxaudio', 0), 0.6666801714114434),   (('voxaudio', 2), 0.6167046371470442),   (('voxaudio', 4), 0.591227209178731),   (('voxaudio', 9), 0.559156959378497),   (('text_video', 0), 0.582291906102476),   (('text_video', 2), 0.5430826146473824),   (('text_video', 4), 0.5248602419559717),   (('text_video', 9), 0.4975638683983908),   (('text_voxaudio', 0), 0.6663066606981422),   (('text_voxaudio', 2), 0.616184648814588),   (('text_voxaudio', 4), 0.5916235792500611),   (('text_voxaudio', 9), 0.5606056614539958),   (('video_voxaudio', 0), 0.6390827744407732),   (('video_voxaudio', 2), 0.5866435380202816),   (('video_voxaudio', 4), 0.559461187570421),   (('video_voxaudio', 9), 0.520403441325134),   (('text_video_voxaudio', 0), 0.6467601325036558),   (('text_video_voxaudio', 2), 0.593987064747505),   (('text_video_voxaudio', 4), 0.5689512043585678),   (('text_video_voxaudio', 9), 0.5316767639961519)],  10: [(('text', 0), 0.6091316039686244),   (('text', 2), 0.5790363707669173),   (('text', 4), 0.5628004786581935),   (('text', 9), 0.5248781354501963),   (('video', 0), 0.5075176118373624),   (('video', 2), 0.49618828097658874),   (('video', 4), 0.4881475948200252),   (('video', 9), 0.4878736164943245),   (('voxaudio', 0), 0.6273059071638538),   (('voxaudio', 2), 0.5913273208110502),   (('voxaudio', 4), 0.5738577217193968),   (('voxaudio', 9), 0.542428994610573),   (('text_video', 0), 0.5610368935749892),   (('text_video', 2), 0.5322051985072983),   (('text_video', 4), 0.517624912722511),   (('text_video', 9), 0.5023113315133424),   (('text_voxaudio', 0), 0.6252129214063562),   (('text_voxaudio', 2), 0.589400130539448),   (('text_voxaudio', 4), 0.5717749786491118),   (('text_voxaudio', 9), 0.5417160885125941),   (('video_voxaudio', 0), 0.5914268661454747),   (('video_voxaudio', 2), 0.5576223723104609),   (('video_voxaudio', 4), 0.5378931174952077),   (('video_voxaudio', 9), 0.512859047129431),   (('text_video_voxaudio', 0), 0.6063553092542625),   (('text_video_voxaudio', 2), 0.5664234397896134),   (('text_video_voxaudio', 4), 0.5475977842097561),   (('text_video_voxaudio', 9), 0.5188465258383493)],  30: [(('text', 0), 0.5547095370468154),   (('text', 2), 0.5382018334181828),   (('text', 4), 0.5282599383772776),   (('text', 9), 0.5018243774483537),   (('video', 0), 0.48423122995357276),   (('video', 2), 0.47900144954187496),   (('video', 4), 0.4787761544616636),   (('video', 9), 0.47435418538114515),   (('voxaudio', 0), 0.5661410978314572),   (('voxaudio', 2), 0.5460479365622382),   (('voxaudio', 4), 0.5310732782662514),   (('voxaudio', 9), 0.4972094232045915),   (('text_video', 0), 0.5175507546662015),   (('text_video', 2), 0.5013922657603975),   (('text_video', 4), 0.4929324232599769),   (('text_video', 9), 0.4763711432474424),   (('text_voxaudio', 0), 0.5682833999527865),   (('text_voxaudio', 2), 0.5462183211275046),   (('text_voxaudio', 4), 0.5330565138302108),   (('text_voxaudio', 9), 0.4975313991123021),   (('video_voxaudio', 0), 0.5246982577947348),   (('video_voxaudio', 2), 0.506665861868644),   (('video_voxaudio', 4), 0.4981340864390632),   (('video_voxaudio', 9), 0.4755560540877834),   (('text_video_voxaudio', 0), 0.5338182770669715),   (('text_video_voxaudio', 2), 0.5120244287004865),   (('text_video_voxaudio', 4), 0.5026541013377209),   (('text_video_voxaudio', 9), 0.47628555959788327)]}, "use_crossval: True | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6786669928159555),   (('text', 2), 0.6154924080953045),   (('text', 4), 0.5831756158265542),   (('text', 9), 0.5435233260299),   (('video', 0), 0.5229472237889518),   (('video', 2), 0.4952604253655618),   (('video', 4), 0.48089777594160293),   (('video', 9), 0.46921075342307067),   (('voxaudio', 0), 0.6763051968924172),   (('voxaudio', 2), 0.6105741992139958),   (('voxaudio', 4), 0.5768613698625413),   (('voxaudio', 9), 0.5339912456711712),   (('text_video', 0), 0.6379134835102558),   (('text_video', 2), 0.564223776458695),   (('text_video', 4), 0.5282261289613411),   (('text_video', 9), 0.4934209042025633),   (('text_voxaudio', 0), 0.6825090752387324),   (('text_voxaudio', 2), 0.6140353080155607),   (('text_voxaudio', 4), 0.5815317087816199),   (('text_voxaudio', 9), 0.5361706192914346),   (('video_voxaudio', 0), 0.6597440267172872),   (('video_voxaudio', 2), 0.5795786006263813),   (('video_voxaudio', 4), 0.538636209807746),   (('video_voxaudio', 9), 0.5003795034161754),   (('text_video_voxaudio', 0), 0.6621063650998698),   (('text_video_voxaudio', 2), 0.5844188877253111),   (('text_video_voxaudio', 4), 0.5421849706763986),   (('text_video_voxaudio', 9), 0.4989879104457852)],  5: [(('text', 0), 0.6647914122012433),   (('text', 2), 0.6075585665481404),   (('text', 4), 0.5765843531274193),   (('text', 9), 0.5353249689073586),   (('video', 0), 0.5303964783011952),   (('video', 2), 0.5055805570296095),   (('video', 4), 0.4882468047334682),   (('video', 9), 0.4818527768705841),   (('voxaudio', 0), 0.6667766674709903),   (('voxaudio', 2), 0.607614514198972),   (('voxaudio', 4), 0.5792559928897506),   (('voxaudio', 9), 0.5386088384676462),   (('text_video', 0), 0.6314859373877089),   (('text_video', 2), 0.568276011136535),   (('text_video', 4), 0.5409623657871065),   (('text_video', 9), 0.49926136555919554),   (('text_voxaudio', 0), 0.6714749434744673),   (('text_voxaudio', 2), 0.6079791508804865),   (('text_voxaudio', 4), 0.5773999512508251),   (('text_voxaudio', 9), 0.5399955214020339),   (('video_voxaudio', 0), 0.6446009622856386),   (('video_voxaudio', 2), 0.5775570899170075),   (('video_voxaudio', 4), 0.5469508347535375),   (('video_voxaudio', 9), 0.5105836171064462),   (('text_video_voxaudio', 0), 0.6494948234897244),   (('text_video_voxaudio', 2), 0.5836696980588119),   (('text_video_voxaudio', 4), 0.5503810312825914),   (('text_video_voxaudio', 9), 0.5091121647268659)],  10: [(('text', 0), 0.6209220510311375),   (('text', 2), 0.5804929881722003),   (('text', 4), 0.5536713459423139),   (('text', 9), 0.5236593139486949),   (('video', 0), 0.5128073775780873),   (('video', 2), 0.4965869570802733),   (('video', 4), 0.4863603578176221),   (('video', 9), 0.4773813404140862),   (('voxaudio', 0), 0.6163672658210728),   (('voxaudio', 2), 0.5768517061908366),   (('voxaudio', 4), 0.5579773844966569),   (('voxaudio', 9), 0.5237418969542558),   (('text_video', 0), 0.5884162734399557),   (('text_video', 2), 0.54860941589703),   (('text_video', 4), 0.5201248121956812),   (('text_video', 9), 0.4893136193415276),   (('text_voxaudio', 0), 0.6214910060521301),   (('text_voxaudio', 2), 0.5796238219674366),   (('text_voxaudio', 4), 0.5557154996515423),   (('text_voxaudio', 9), 0.5231382815058223),   (('video_voxaudio', 0), 0.5998566140024587),   (('video_voxaudio', 2), 0.5536930023649911),   (('video_voxaudio', 4), 0.5334508970883088),   (('video_voxaudio', 9), 0.49833886600414906),   (('text_video_voxaudio', 0), 0.6025133222018683),   (('text_video_voxaudio', 2), 0.5612215980952994),   (('text_video_voxaudio', 4), 0.537239031357809),   (('text_video_voxaudio', 9), 0.5005057030972333)],  30: [(('text', 0), 0.518257950199433),   (('text', 2), 0.49883654057982235),   (('text', 4), 0.4890981597160831),   (('text', 9), 0.4725502647746211),   (('video', 0), 0.49161622150830553),   (('video', 2), 0.47695047512464195),   (('video', 4), 0.4754429665743656),   (('video', 9), 0.46624990528528054),   (('voxaudio', 0), 0.5286604993296746),   (('voxaudio', 2), 0.5051321310566917),   (('voxaudio', 4), 0.49719091764223267),   (('voxaudio', 9), 0.48433553261744733),   (('text_video', 0), 0.5105114121755255),   (('text_video', 2), 0.4935936797942718),   (('text_video', 4), 0.47928085394098946),   (('text_video', 9), 0.471765238330042),   (('text_voxaudio', 0), 0.5336678052725623),   (('text_voxaudio', 2), 0.51142919508295),   (('text_voxaudio', 4), 0.4977474082746802),   (('text_voxaudio', 9), 0.4793811403325803),   (('video_voxaudio', 0), 0.524925435759266),   (('video_voxaudio', 2), 0.5041895928815355),   (('video_voxaudio', 4), 0.49499381178642377),   (('video_voxaudio', 9), 0.472690038543112),   (('text_video_voxaudio', 0), 0.5263639517105588),   (('text_video_voxaudio', 2), 0.5030523605098794),   (('text_video_voxaudio', 4), 0.4946325310234381),   (('text_video_voxaudio', 9), 0.4764411640837084)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.7190447998172413),   (('text', 2), 0.6861005988903993),   (('text', 4), 0.6706471284268235),   (('text', 9), 0.653335688580216),   (('video', 0), 0.6524177476979405),   (('video', 2), 0.6482345592828013),   (('video', 4), 0.6476989361257939),   (('video', 9), 0.6472930738301318),   (('voxaudio', 0), 0.7325241355995085),   (('voxaudio', 2), 0.7000183708711467),   (('voxaudio', 4), 0.6878566412121523),   (('voxaudio', 9), 0.6746072665450058),   (('text_video', 0), 0.7185610328317733),   (('text_video', 2), 0.6854392475291178),   (('text_video', 4), 0.6688201631672197),   (('text_video', 9), 0.6523644194357894),   (('text_voxaudio', 0), 0.7364381955509789),   (('text_voxaudio', 2), 0.7025829444832274),   (('text_voxaudio', 4), 0.6889690582173772),   (('text_voxaudio', 9), 0.6747629618690927),   (('video_voxaudio', 0), 0.7251357718508511),   (('video_voxaudio', 2), 0.6946320314509313),   (('video_voxaudio', 4), 0.683362725449031),   (('video_voxaudio', 9), 0.6709371739085171),   (('text_video_voxaudio', 0), 0.7325021323328562),   (('text_video_voxaudio', 2), 0.6970863798361318),   (('text_video_voxaudio', 4), 0.6849171693934755),   (('text_video_voxaudio', 9), 0.6721012251319759)],  5: [(('text', 0), 0.7117744460359982),   (('text', 2), 0.6823857964567941),   (('text', 4), 0.6677149072398999),   (('text', 9), 0.6526378355099858),   (('video', 0), 0.6484910899771137),   (('video', 2), 0.6440539229335357),   (('video', 4), 0.6427386002306087),   (('video', 9), 0.6408571366027865),   (('voxaudio', 0), 0.7279022640617216),   (('voxaudio', 2), 0.6992466770887684),   (('voxaudio', 4), 0.6880386565164474),   (('voxaudio', 9), 0.6732709823801721),   (('text_video', 0), 0.7101836419646813),   (('text_video', 2), 0.6795908341745364),   (('text_video', 4), 0.6650497501750195),   (('text_video', 9), 0.648935767780445),   (('text_voxaudio', 0), 0.731674747436489),   (('text_voxaudio', 2), 0.7011378174835274),   (('text_voxaudio', 4), 0.6877493192177584),   (('text_voxaudio', 9), 0.6731406921578656),   (('video_voxaudio', 0), 0.7197133693957987),   (('video_voxaudio', 2), 0.6932010430546389),   (('video_voxaudio', 4), 0.6821600552548029),   (('video_voxaudio', 9), 0.668702759060409),   (('text_video_voxaudio', 0), 0.7264023237449871),   (('text_video_voxaudio', 2), 0.6949402836576625),   (('text_video_voxaudio', 4), 0.6820762672110356),   (('text_video_voxaudio', 9), 0.6687104509745592)],  10: [(('text', 0), 0.6969406216931839),   (('text', 2), 0.6779100833395695),   (('text', 4), 0.6665541428551505),   (('text', 9), 0.6569759986976763),   (('video', 0), 0.6495389748294895),   (('video', 2), 0.6481408054899305),   (('video', 4), 0.6469574413562629),   (('video', 9), 0.6456775240858363),   (('voxaudio', 0), 0.7096777498921039),   (('voxaudio', 2), 0.6921639276889598),   (('voxaudio', 4), 0.6823480594135107),   (('voxaudio', 9), 0.6698180625306237),   (('text_video', 0), 0.6932110233068345),   (('text_video', 2), 0.6717466855016697),   (('text_video', 4), 0.6609036246733814),   (('text_video', 9), 0.6531249522400088),   (('text_voxaudio', 0), 0.7114929520338535),   (('text_voxaudio', 2), 0.693639813348945),   (('text_voxaudio', 4), 0.6834827785612505),   (('text_voxaudio', 9), 0.6717551541748804),   (('video_voxaudio', 0), 0.7017532824323341),   (('video_voxaudio', 2), 0.684815220773115),   (('video_voxaudio', 4), 0.6775332497096678),   (('video_voxaudio', 9), 0.6650639661364374),   (('text_video_voxaudio', 0), 0.7051243973872957),   (('text_video_voxaudio', 2), 0.6870710503864299),   (('text_video_voxaudio', 4), 0.6775869192259496),   (('text_video_voxaudio', 9), 0.6668775938477353)],  30: [(('text', 0), 0.655522581358392),   (('text', 2), 0.6520026182846841),   (('text', 4), 0.6506420666972346),   (('text', 9), 0.6503488048522519),   (('video', 0), 0.641131399359721),   (('video', 2), 0.639776928615092),   (('video', 4), 0.6393255111178164),   (('video', 9), 0.6401399466879888),   (('voxaudio', 0), 0.6665713182834371),   (('voxaudio', 2), 0.6585886196529799),   (('voxaudio', 4), 0.6542663711451311),   (('voxaudio', 9), 0.6449076972020669),   (('text_video', 0), 0.6527449388787898),   (('text_video', 2), 0.6481239630924828),   (('text_video', 4), 0.6462010009111581),   (('text_video', 9), 0.6437020728188063),   (('text_voxaudio', 0), 0.6691323215447685),   (('text_voxaudio', 2), 0.6594573754666204),   (('text_voxaudio', 4), 0.6547485488188618),   (('text_voxaudio', 9), 0.6440699929755229),   (('video_voxaudio', 0), 0.6634532661386863),   (('video_voxaudio', 2), 0.6557571005129093),   (('video_voxaudio', 4), 0.6525708179223466),   (('video_voxaudio', 9), 0.6426921149542997),   (('text_video_voxaudio', 0), 0.6671052015378259),   (('text_video_voxaudio', 2), 0.658495449009991),   (('text_video_voxaudio', 4), 0.6520885758237636),   (('text_video_voxaudio', 9), 0.6439838774953888)]}, "use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.6610105759327962),   (('text', 2), 0.609304638577546),   (('text', 4), 0.5841364382417231),   (('text', 9), 0.5499382918166179),   (('video', 0), 0.5007938942690195),   (('video', 2), 0.4692929457312055),   (('video', 4), 0.45220162585616885),   (('video', 9), 0.44025144161767915),   (('voxaudio', 0), 0.678038594786425),   (('voxaudio', 2), 0.61411486863142),   (('voxaudio', 4), 0.5835202199275946),   (('voxaudio', 9), 0.5357079052533577),   (('text_video', 0), 0.6167851386247806),   (('text_video', 2), 0.5450081674608273),   (('text_video', 4), 0.5166995409563434),   (('text_video', 9), 0.4714161493723398),   (('text_voxaudio', 0), 0.6799986097189195),   (('text_voxaudio', 2), 0.6154452797783797),   (('text_voxaudio', 4), 0.5844232218433804),   (('text_voxaudio', 9), 0.5377525938369002),   (('video_voxaudio', 0), 0.6513726180493502),   (('video_voxaudio', 2), 0.5703903488622022),   (('video_voxaudio', 4), 0.529792268728395),   (('video_voxaudio', 9), 0.48000724810800743),   (('text_video_voxaudio', 0), 0.6549459530835373),   (('text_video_voxaudio', 2), 0.5720688416458378),   (('text_video_voxaudio', 4), 0.5306133646621429),   (('text_video_voxaudio', 9), 0.482258360554742)],  5: [(('text', 0), 0.6488563108718621),   (('text', 2), 0.6012895048149784),   (('text', 4), 0.5771070608408809),   (('text', 9), 0.5429718178141762),   (('video', 0), 0.4961408636375781),   (('video', 2), 0.4738291586174048),   (('video', 4), 0.45993248691262706),   (('video', 9), 0.44562779148967124),   (('voxaudio', 0), 0.6663894717156114),   (('voxaudio', 2), 0.6082788396656301),   (('voxaudio', 4), 0.5764176193723383),   (('voxaudio', 9), 0.5306688679881971),   (('text_video', 0), 0.606817573201454),   (('text_video', 2), 0.5385671961646292),   (('text_video', 4), 0.509111663495472),   (('text_video', 9), 0.47096553417449094),   (('text_voxaudio', 0), 0.6703526783581633),   (('text_voxaudio', 2), 0.6085701573551601),   (('text_voxaudio', 4), 0.578190658151425),   (('text_voxaudio', 9), 0.5329365381582621),   (('video_voxaudio', 0), 0.636198715957758),   (('video_voxaudio', 2), 0.5636972647401094),   (('video_voxaudio', 4), 0.5255467471521957),   (('video_voxaudio', 9), 0.47854342031408637),   (('text_video_voxaudio', 0), 0.6425273110481579),   (('text_video_voxaudio', 2), 0.5656734844297509),   (('text_video_voxaudio', 4), 0.5280288926577562),   (('text_video_voxaudio', 9), 0.4790474035730507)],  10: [(('text', 0), 0.5997132341990647),   (('text', 2), 0.5687789391482486),   (('text', 4), 0.5476845200321258),   (('text', 9), 0.5161244442874682),   (('video', 0), 0.4803737958392982),   (('video', 2), 0.4635754234011403),   (('video', 4), 0.45194604022274076),   (('video', 9), 0.4423952820766397),   (('voxaudio', 0), 0.6080848875850602),   (('voxaudio', 2), 0.5644101287317873),   (('voxaudio', 4), 0.5395252906721857),   (('voxaudio', 9), 0.4967783973670586),   (('text_video', 0), 0.5439726909044715),   (('text_video', 2), 0.504538550912345),   (('text_video', 4), 0.48653421632536437),   (('text_video', 9), 0.4568981375974082),   (('text_voxaudio', 0), 0.6148762592956907),   (('text_voxaudio', 2), 0.5683286322096256),   (('text_voxaudio', 4), 0.5406847294151207),   (('text_voxaudio', 9), 0.5005179097369329),   (('video_voxaudio', 0), 0.5694084534538677),   (('video_voxaudio', 2), 0.5227395950838656),   (('video_voxaudio', 4), 0.497870277138816),   (('video_voxaudio', 9), 0.4645496961340189),   (('text_video_voxaudio', 0), 0.5779296634906937),   (('text_video_voxaudio', 2), 0.5251805756191568),   (('text_video_voxaudio', 4), 0.500683209982378),   (('text_video_voxaudio', 9), 0.46610585941219285)],  30: [(('text', 0), 0.4933723186752541),   (('text', 2), 0.47651288673075987),   (('text', 4), 0.4656432579898997),   (('text', 9), 0.44231404794101925),   (('video', 0), 0.4464367539191999),   (('video', 2), 0.4391083456217009),   (('video', 4), 0.4408690683661513),   (('video', 9), 0.4322265554791917),   (('voxaudio', 0), 0.48671315576468305),   (('voxaudio', 2), 0.46577375762105994),   (('voxaudio', 4), 0.447201454542995),   (('voxaudio', 9), 0.42851484126840794),   (('text_video', 0), 0.47188763215220025),   (('text_video', 2), 0.4522496132036723),   (('text_video', 4), 0.44336396284368307),   (('text_video', 9), 0.4335573453458895),   (('text_voxaudio', 0), 0.48777354041990667),   (('text_voxaudio', 2), 0.46528456791691547),   (('text_voxaudio', 4), 0.4510603295932203),   (('text_voxaudio', 9), 0.42866463220803874),   (('video_voxaudio', 0), 0.47742363168074126),   (('video_voxaudio', 2), 0.4606701956310766),   (('video_voxaudio', 4), 0.45027137387684446),   (('video_voxaudio', 9), 0.43248439088911655),   (('text_video_voxaudio', 0), 0.4819835828343482),   (('text_video_voxaudio', 2), 0.4606489981881188),   (('text_video_voxaudio', 4), 0.45014338075659044),   (('text_video_voxaudio', 9), 0.4303814556586977)]}}
# all_results = {"use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: GaussianNB(priors=None, var_smoothing=1e-09)" : {4: [(('text', 0), 0.6234794361845917), (('text', 2), 0.5829943831105946), (('text', 4), 0.5665436176413444), (('text', 9), 0.539455649109066), (('video', 0), 0.5514578103881058), (('video', 2), 0.536897152818129), (('video', 4), 0.534874684282106), (('video', 9), 0.5337771685921284), (('voxaudio', 0), 0.6481946321683723), (('voxaudio', 2), 0.5953902769707534), (('voxaudio', 4), 0.5686807849232562), (('voxaudio', 9), 0.5402388878010573), (('text_video', 0), 0.5945163158910987), (('text_video', 2), 0.5614952546968817), (('text_video', 4), 0.555469205362347), (('text_video', 9), 0.5435676522420207), (('text_voxaudio', 0), 0.6570766557250435), (('text_voxaudio', 2), 0.5990703079604881), (('text_voxaudio', 4), 0.5811152127452885), (('text_voxaudio', 9), 0.5461131779909928), (('video_voxaudio', 0), 0.6346785093647422), (('video_voxaudio', 2), 0.5814448963780747), (('video_voxaudio', 4), 0.5686807849232562), (('video_voxaudio', 9), 0.5615821421578227), (('text_video_voxaudio', 0), 0.6420158331724272), (('text_video_voxaudio', 2), 0.590548130931629), (('text_video_voxaudio', 4), 0.5772294540509034), (('text_video_voxaudio', 9), 0.5611905228118269)], 5: [(('text', 0), 0.6183473389355743), (('text', 2), 0.5660112359550562), (('text', 4), 0.5521126760563381), (('text', 9), 0.5132978723404256), (('video', 0), 0.5689775910364145), (('video', 2), 0.5626755617977528), (('video', 4), 0.5619718309859155), (('video', 9), 0.5606382978723404), (('voxaudio', 0), 0.6411064425770309), (('voxaudio', 2), 0.5935744382022472), (('voxaudio', 4), 0.5732394366197183), (('voxaudio', 9), 0.5452127659574468), (('text_video', 0), 0.601890756302521), (('text_video', 2), 0.5784761235955056), (('text_video', 4), 0.5767605633802817), (('text_video', 9), 0.5695035460992908), (('text_voxaudio', 0), 0.6502100840336135), (('text_voxaudio', 2), 0.598314606741573), (('text_voxaudio', 4), 0.5765845070422535), (('text_voxaudio', 9), 0.5427304964539007), (('video_voxaudio', 0), 0.6260504201680672), (('video_voxaudio', 2), 0.5953300561797753), (('video_voxaudio', 4), 0.5802816901408451), (('video_voxaudio', 9), 0.573936170212766), (('text_video_voxaudio', 0), 0.6391806722689075), (('text_video_voxaudio', 2), 0.6014747191011236), (('text_video_voxaudio', 4), 0.5894366197183099), (('text_video_voxaudio', 9), 0.573758865248227)], 10: [(('text', 0), 0.578984485190409), (('text', 2), 0.5477369165487977), (('text', 4), 0.5342198581560283), (('text', 9), 0.5141071428571429), (('video', 0), 0.5756346967559943), (('video', 2), 0.570898161244696), (('video', 4), 0.5656028368794326), (('video', 9), 0.54625), (('voxaudio', 0), 0.6126586741889986), (('voxaudio', 2), 0.579031117397454), (('voxaudio', 4), 0.5668439716312057), (('voxaudio', 9), 0.55375), (('text_video', 0), 0.6004936530324401), (('text_video', 2), 0.5774398868458275), (('text_video', 4), 0.574645390070922), (('text_video', 9), 0.5510714285714285), (('text_voxaudio', 0), 0.6140691114245416), (('text_voxaudio', 2), 0.5756718528995757), (('text_voxaudio', 4), 0.5641843971631205), (('text_voxaudio', 9), 0.54625), (('video_voxaudio', 0), 0.610543018335684), (('video_voxaudio', 2), 0.5868104667609618), (('video_voxaudio', 4), 0.5822695035460993), (('video_voxaudio', 9), 0.5682142857142857), (('text_video_voxaudio', 0), 0.6160084626234132), (('text_video_voxaudio', 2), 0.5885785007072136), (('text_video_voxaudio', 4), 0.5856382978723405), (('text_video_voxaudio', 9), 0.5716071428571429)], 30: [(('text', 0), 0.5665820029027576), (('text', 2), 0.5598617176128093), (('text', 4), 0.5505474452554745), (('text', 9), 0.5303308823529411), (('video', 0), 0.5609579100145138), (('video', 2), 0.5576783114992722), (('video', 4), 0.5525547445255474), (('video', 9), 0.546875), (('voxaudio', 0), 0.5809143686502177), (('voxaudio', 2), 0.5589519650655022), (('voxaudio', 4), 0.5572992700729927), (('voxaudio', 9), 0.5466911764705882), (('text_video', 0), 0.5732946298984035), (('text_video', 2), 0.5633187772925764), (('text_video', 4), 0.5565693430656934), (('text_video', 9), 0.5479779411764706), (('text_voxaudio', 0), 0.5725689404934688), (('text_voxaudio', 2), 0.5540393013100436), (('text_voxaudio', 4), 0.5516423357664234), (('text_voxaudio', 9), 0.5347426470588236), (('video_voxaudio', 0), 0.5792815674891146), (('video_voxaudio', 2), 0.5680494905385735), (('video_voxaudio', 4), 0.5602189781021898), (('video_voxaudio', 9), 0.5582720588235294), (('text_video_voxaudio', 0), 0.5820029027576198), (('text_video_voxaudio', 2), 0.5722343522561864), (('text_video_voxaudio', 4), 0.5638686131386861), (('text_video_voxaudio', 9), 0.5512867647058823)]} , "use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: GaussianNB(priors=None, var_smoothing=1e-09)" : {4: [(('text', 0), 0.6452951346011361), (('text', 2), 0.5723270440251572), (('text', 4), 0.5716697284146766), (('text', 9), 0.551296786767278), (('video', 0), 0.5657199308471228), (('video', 2), 0.5463279354231665), (('video', 4), 0.5419790477136189), (('video', 9), 0.5399030533206737), (('voxaudio', 0), 0.6749320819955544), (('voxaudio', 2), 0.6355172584559006), (('voxaudio', 4), 0.6221637455935654), (('voxaudio', 9), 0.6089650692119334), (('text_video', 0), 0.62904420844653), (('text_video', 2), 0.5894616946466598), (('text_video', 4), 0.5778759743806167), (('text_video', 9), 0.5564939283394134), (('text_voxaudio', 0), 0.6722647567300568), (('text_voxaudio', 2), 0.6107066805328579), (('text_voxaudio', 4), 0.6065240057593962), (('text_voxaudio', 9), 0.5909749637699265), (('video_voxaudio', 0), 0.6466287972338849), (('video_voxaudio', 2), 0.6116476006536918), (('video_voxaudio', 4), 0.5970408619234397), (('video_voxaudio', 9), 0.576582879416321), (('text_video_voxaudio', 0), 0.6637194368979995), (('text_video_voxaudio', 2), 0.6155103253602734), (('text_video_voxaudio', 4), 0.604885556824388), (('text_video_voxaudio', 9), 0.5822797461396232)], 5: [(('text', 0), 0.6498554913294797), (('text', 2), 0.6086536604210933), (('text', 4), 0.5914988381099923), (('text', 9), 0.5682061775309364), (('video', 0), 0.5648843930635838), (('video', 2), 0.5517191423604404), (('video', 4), 0.5476374903175832), (('video', 9), 0.5464289194192731), (('voxaudio', 0), 0.6700385356454721), (('voxaudio', 2), 0.6368553216148348), (('voxaudio', 4), 0.6249515879163439), (('voxaudio', 9), 0.6140017538731365), (('text_video', 0), 0.6205202312138728), (('text_video', 2), 0.5928626617732278), (('text_video', 4), 0.5784759876065065), (('text_video', 9), 0.5630907142161161), (('text_voxaudio', 0), 0.6660404624277456), (('text_voxaudio', 2), 0.6283079003283755), (('text_voxaudio', 4), 0.6132358636715725), (('text_voxaudio', 9), 0.591688590080873), (('video_voxaudio', 0), 0.6435452793834296), (('video_voxaudio', 2), 0.6117442534286266), (('video_voxaudio', 4), 0.5979376452362509), (('video_voxaudio', 9), 0.5822858813212511), (('text_video_voxaudio', 0), 0.6624277456647398), (('text_video_voxaudio', 2), 0.6268591848560943), (('text_video_voxaudio', 4), 0.6103795507358637), (('text_video_voxaudio', 9), 0.5895449673584722)], 10: [(('text', 0), 0.5935046049442559), (('text', 2), 0.5626397123141219), (('text', 4), 0.5499853843905291), (('text', 9), 0.4525887428907629), (('video', 0), 0.5523994183228308), (('video', 2), 0.5373214112158616), (('video', 4), 0.5342005261619409), (('video', 9), 0.5295646205138262), (('voxaudio', 0), 0.6484730974309258), (('voxaudio', 2), 0.6339780347944407), (('voxaudio', 4), 0.6300302055929066), (('voxaudio', 9), 0.6314963718376152), (('text_video', 0), 0.5888027144934561), (('text_video', 2), 0.5645349402274273), (('text_video', 4), 0.5576829387118776), (('text_video', 9), 0.5395665816826829), (('text_voxaudio', 0), 0.6113427047988367), (('text_voxaudio', 2), 0.5836816017105647), (('text_voxaudio', 4), 0.5723959855792653), (('text_voxaudio', 9), 0.5500098058442832), (('video_voxaudio', 0), 0.6206010664081435), (('video_voxaudio', 2), 0.6008358441053553), (('video_voxaudio', 4), 0.5911039657020365), (('video_voxaudio', 9), 0.5808491861149245), (('text_video_voxaudio', 0), 0.6283082888996607), (('text_video_voxaudio', 2), 0.600155505880066), (('text_video_voxaudio', 4), 0.5913475591932184), (('text_video_voxaudio', 9), 0.5694744067464209)], 30: [(('text', 0), 0.4972650422675286), (('text', 2), 0.4849935187954931), (('text', 4), 0.4789063281015695), (('text', 9), 0.4606560676192393), (('video', 0), 0.5436101442068623), (('video', 2), 0.5328547213082062), (('video', 4), 0.5358392482255323), (('video', 9), 0.5361742805393439), (('voxaudio', 0), 0.6586772749875683), (('voxaudio', 2), 0.6546016551999202), (('voxaudio', 4), 0.6517544736579026), (('voxaudio', 9), 0.6398671764942644), (('text_video', 0), 0.5501740427647936), (('text_video', 2), 0.5341509622095921), (('text_video', 4), 0.5299410176946916), (('text_video', 9), 0.5191185349164822), (('text_voxaudio', 0), 0.5317752362008951), (('text_voxaudio', 2), 0.5219862399042776), (('text_voxaudio', 4), 0.5228431470558832), (('text_voxaudio', 9), 0.5329040048299457), (('video_voxaudio', 0), 0.6037792143212333), (('video_voxaudio', 2), 0.5998105494067205), (('video_voxaudio', 4), 0.5993202039388184), (('video_voxaudio', 9), 0.5982592070839203), (('text_video_voxaudio', 0), 0.5816011934361014), (('text_video_voxaudio', 2), 0.5676039485492073), (('text_video_voxaudio', 4), 0.5645306408077577), (('text_video_voxaudio', 9), 0.5621352384785672)]} , "use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: GaussianNB(priors=None, var_smoothing=1e-09)" : {4: [(('text', 0), 0.6529232400901498), (('text', 2), 0.6116943521594684), (('text', 4), 0.5946449980018649), (('text', 9), 0.5714285714285714), (('video', 0), 0.5438154580405674), (('video', 2), 0.5217275747508305), (('video', 4), 0.5180498201678434), (('video', 9), 0.5158134548378451), (('voxaudio', 0), 0.6604799151531221), (('voxaudio', 2), 0.6151495016611296), (('voxaudio', 4), 0.5973091781004396), (('voxaudio', 9), 0.5733047440364514), (('text_video', 0), 0.5990985019223121), (('text_video', 2), 0.5553488372093023), (('text_video', 4), 0.5504196083655255), (('text_video', 9), 0.5336370946127044), (('text_voxaudio', 0), 0.6620707941137478), (('text_voxaudio', 2), 0.6200664451827242), (('text_voxaudio', 4), 0.605568136406021), (('text_voxaudio', 9), 0.5755829536317342), (('video_voxaudio', 0), 0.6476203102213973), (('video_voxaudio', 2), 0.598671096345515), (('video_voxaudio', 4), 0.5737311842280538), (('video_voxaudio', 9), 0.5532028946663093), (('text_video_voxaudio', 0), 0.6554421317778072), (('text_video_voxaudio', 2), 0.6104983388704319), (('text_video_voxaudio', 4), 0.596110297056081), (('text_video_voxaudio', 9), 0.5682122755293487)], 5: [(('text', 0), 0.6466684364215556), (('text', 2), 0.6101649813730707), (('text', 4), 0.588957055214724), (('text', 9), 0.5209982557359453), (('video', 0), 0.5342447571011415), (('video', 2), 0.518493879723257), (('video', 4), 0.5194718591624433), (('video', 9), 0.5196565141553737), (('voxaudio', 0), 0.6506503849216884), (('voxaudio', 2), 0.6113624268227781), (('voxaudio', 4), 0.5921579087756735), (('voxaudio', 9), 0.5701059975848651), (('text_video', 0), 0.5914520838863817), (('text_video', 2), 0.55401809473124), (('text_video', 4), 0.5505468124833289), (('text_video', 9), 0.5320005366966323), (('text_voxaudio', 0), 0.6531722856384391), (('text_voxaudio', 2), 0.6137573177221927), (('text_voxaudio', 4), 0.5956255001333689), (('text_voxaudio', 9), 0.5718502616396082), (('video_voxaudio', 0), 0.6359171754711972), (('video_voxaudio', 2), 0.5882118147951038), (('video_voxaudio', 4), 0.5686849826620433), (('video_voxaudio', 9), 0.5455521266604052), (('text_video_voxaudio', 0), 0.6509158481550306), (('text_video_voxaudio', 2), 0.6094997339010112), (('text_video_voxaudio', 4), 0.5878901040277408), (('text_video_voxaudio', 9), 0.5639339863142359)], 10: [(('text', 0), 0.6067565763119241), (('text', 2), 0.5819836701914067), (('text', 4), 0.5693009526365223), (('text', 9), 0.5295626349892009), (('video', 0), 0.5219655494725598), (('video', 2), 0.5170659884888235), (('video', 4), 0.511471890513887), (('video', 9), 0.5024298056155507), (('voxaudio', 0), 0.6263853652022967), (('voxaudio', 2), 0.5971088207736581), (('voxaudio', 4), 0.5840601100228096), (('voxaudio', 9), 0.5599352051835853), (('text_video', 0), 0.5589531312591801), (('text_video', 2), 0.5347343059831348), (('text_video', 4), 0.5211324298940024), (('text_video', 9), 0.5067494600431965), (('text_voxaudio', 0), 0.6305247696621712), (('text_voxaudio', 2), 0.5993842859055013), (('text_voxaudio', 4), 0.5847309808130954), (('text_voxaudio', 9), 0.5592602591792657), (('video_voxaudio', 0), 0.5807183869675524), (('video_voxaudio', 2), 0.5486547985544104), (('video_voxaudio', 4), 0.5338789749094325), (('video_voxaudio', 9), 0.5133639308855291), (('text_video_voxaudio', 0), 0.6189077313392977), (('text_video_voxaudio', 2), 0.580779012180431), (('text_video_voxaudio', 4), 0.5670199919495506), (('text_video_voxaudio', 9), 0.5267278617710583)], 30: [(('text', 0), 0.566835408400602), (('text', 2), 0.5506789192154711), (('text', 4), 0.5495668912415784), (('text', 9), 0.5355561704482568), (('video', 0), 0.5097824599808455), (('video', 2), 0.5052804827869977), (('video', 4), 0.5024061597690087), (('video', 9), 0.5051189817376868), (('voxaudio', 0), 0.5792858120125872), (('voxaudio', 2), 0.5560279797010013), (('voxaudio', 4), 0.5484669324900316), (('voxaudio', 9), 0.5444106253458771), (('text_video', 0), 0.5363250786701327), (('text_video', 2), 0.5155671375668632), (('text_video', 4), 0.5083184380585728), (('text_video', 9), 0.5019369120088545), (('text_voxaudio', 0), 0.5846216992748666), (('text_voxaudio', 2), 0.562885749554245), (('text_voxaudio', 4), 0.5580915715660663), (('text_voxaudio', 9), 0.5462091864969563), (('video_voxaudio', 0), 0.5359146258038036), (('video_voxaudio', 2), 0.5150185159786037), (('video_voxaudio', 4), 0.5099683761858931), (('video_voxaudio', 9), 0.5023519645821805), (('text_video_voxaudio', 0), 0.5709399370638938), (('text_video_voxaudio', 2), 0.5372376903031134), (('text_video_voxaudio', 4), 0.5195930152619277), (('text_video_voxaudio', 9), 0.5005534034311012)]} , "use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: RandomForestClassifier()" : {4: [(('text', 0), 0.654759606101564), (('text', 2), 0.5953902769707534), (('text', 4), 0.5731494074217991), (('text', 9), 0.5504209907969454), (('video', 0), 0.5954817532342151), (('video', 2), 0.5700174317257408), (('video', 4), 0.5541091898193122), (('video', 9), 0.5261405913452124), (('voxaudio', 0), 0.6485808071056188), (('voxaudio', 2), 0.600232423009878), (('voxaudio', 4), 0.5842238197007966), (('voxaudio', 9), 0.5557078519678872), (('text_video', 0), 0.638540258737208), (('text_video', 2), 0.5982955645942282), (('text_video', 4), 0.5634350106858365), (('text_video', 9), 0.5484628940669669), (('text_voxaudio', 0), 0.6483877196369956), (('text_voxaudio', 2), 0.6013945380592679), (('text_voxaudio', 4), 0.5785894695939382), (('text_voxaudio', 9), 0.5562952809868807), (('video_voxaudio', 0), 0.6530218188839544), (('video_voxaudio', 2), 0.5975208212279682), (('video_voxaudio', 4), 0.5745094229648339), (('video_voxaudio', 9), 0.555903661640885), (('text_video_voxaudio', 0), 0.6489669820428654), (('text_video_voxaudio', 2), 0.6012008522177029), (('text_video_voxaudio', 4), 0.5766465902467457), (('text_video_voxaudio', 9), 0.555903661640885)], 5: [(('text', 0), 0.6390056022408963), (('text', 2), 0.5990168539325843), (('text', 4), 0.5762323943661972), (('text', 9), 0.5540780141843972), (('video', 0), 0.5957633053221288), (('video', 2), 0.5684691011235955), (('video', 4), 0.551056338028169), (('video', 9), 0.548049645390071), (('voxaudio', 0), 0.6437324929971989), (('voxaudio', 2), 0.5986657303370787), (('voxaudio', 4), 0.5797535211267606), (('voxaudio', 9), 0.5570921985815603), (('text_video', 0), 0.6423319327731093), (('text_video', 2), 0.5981390449438202), (('text_video', 4), 0.5809859154929577), (('text_video', 9), 0.5617021276595745), (('text_voxaudio', 0), 0.6516106442577031), (('text_voxaudio', 2), 0.6018258426966292), (('text_voxaudio', 4), 0.586443661971831), (('text_voxaudio', 9), 0.5608156028368795), (('video_voxaudio', 0), 0.6446078431372549), (('video_voxaudio', 2), 0.5951544943820225), (('video_voxaudio', 4), 0.583274647887324), (('video_voxaudio', 9), 0.5586879432624113), (('text_video_voxaudio', 0), 0.6510854341736695), (('text_video_voxaudio', 2), 0.6007724719101124), (('text_video_voxaudio', 4), 0.5794014084507042), (('text_video_voxaudio', 9), 0.5712765957446808)], 10: [(('text', 0), 0.6124823695345557), (('text', 2), 0.5802687411598303), (('text', 4), 0.5583333333333333), (('text', 9), 0.5478571428571428), (('video', 0), 0.5877997179125529), (('video', 2), 0.5746110325318247), (('video', 4), 0.5689716312056737), (('video', 9), 0.5380357142857143), (('voxaudio', 0), 0.6161847672778561), (('voxaudio', 2), 0.5885785007072136), (('voxaudio', 4), 0.5732269503546099), (('voxaudio', 9), 0.5630357142857143), (('text_video', 0), 0.6145980253878702), (('text_video', 2), 0.5829207920792079), (('text_video', 4), 0.5732269503546099), (('text_video', 9), 0.5523214285714285), (('text_voxaudio', 0), 0.6212976022566996), (('text_voxaudio', 2), 0.5908769448373409), (('text_voxaudio', 4), 0.5732269503546099), (('text_voxaudio', 9), 0.5557142857142857), (('video_voxaudio', 0), 0.6144217207334274), (('video_voxaudio', 2), 0.5876944837340877), (('video_voxaudio', 4), 0.5831560283687943), (('video_voxaudio', 9), 0.55875), (('text_video_voxaudio', 0), 0.6188293370944993), (('text_video_voxaudio', 2), 0.5910537482319661), (('text_video_voxaudio', 4), 0.5762411347517731), (('text_video_voxaudio', 9), 0.5617857142857143)], 30: [(('text', 0), 0.5644049346879536), (('text', 2), 0.5462154294032023), (('text', 4), 0.5461678832116789), (('text', 9), 0.5419117647058823), (('video', 0), 0.5604136429608128), (('video', 2), 0.5516739446870451), (('video', 4), 0.5593065693430657), (('video', 9), 0.5431985294117647), (('voxaudio', 0), 0.5740203193033382), (('voxaudio', 2), 0.5587700145560408), (('voxaudio', 4), 0.5565693430656934), (('voxaudio', 9), 0.5380514705882353), (('text_video', 0), 0.5787373004354136), (('text_video', 2), 0.5571324599708879), (('text_video', 4), 0.5551094890510949), (('text_video', 9), 0.5408088235294117), (('text_voxaudio', 0), 0.5772859216255443), (('text_voxaudio', 2), 0.5649563318777293), (('text_voxaudio', 4), 0.5551094890510949), (('text_voxaudio', 9), 0.5534926470588235), (('video_voxaudio', 0), 0.5810957910014514), (('video_voxaudio', 2), 0.5658660844250364), (('video_voxaudio', 4), 0.5576642335766423), (('video_voxaudio', 9), 0.5523897058823529), (('text_video_voxaudio', 0), 0.5760159651669086), (('text_video_voxaudio', 2), 0.5689592430858806), (('text_video_voxaudio', 4), 0.5618613138686132), (('text_video_voxaudio', 9), 0.5498161764705882)]} , "use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: RandomForestClassifier" : {4: [(('text', 0), 0.7008150160533465), (('text', 2), 0.6727083642846531), (('text', 4), 0.6648627178392335), (('text', 9), 0.650392284243666), (('video', 0), 0.6585823660163003), (('video', 2), 0.6487396622591988), (('video', 4), 0.6479817288118763), (('video', 9), 0.6477937134575983), (('voxaudio', 0), 0.7212645097554952), (('voxaudio', 2), 0.6949437924033081), (('voxaudio', 4), 0.6850702547043345), (('voxaudio', 9), 0.6721303283194243), (('text_video', 0), 0.7057051123734256), (('text_video', 2), 0.6797900262467191), (('text_video', 4), 0.671118613772901), (('text_video', 9), 0.648793163759932), (('text_voxaudio', 0), 0.7233884909854285), (('text_voxaudio', 2), 0.6927152973802803), (('text_voxaudio', 4), 0.6871555533488903), (('text_voxaudio', 9), 0.672030383289191), (('video_voxaudio', 0), 0.7139540627315386), (('video_voxaudio', 2), 0.690338236022384), (('video_voxaudio', 4), 0.6789136587061219), (('video_voxaudio', 9), 0.6741292289240918), (('text_video_voxaudio', 0), 0.7175598913311929), (('text_video_voxaudio', 2), 0.692517208933789), (('text_video_voxaudio', 4), 0.6831835559306887), (('text_video_voxaudio', 9), 0.6725800809554745)], 5: [(('text', 0), 0.6982177263969171), (('text', 2), 0.6726385937801815), (('text', 4), 0.6631487219209915), (('text', 9), 0.6547793042969892), (('video', 0), 0.6677745664739885), (('video', 2), 0.656847595132316), (('video', 4), 0.6558384972889233), (('video', 9), 0.6558023969599532), (('voxaudio', 0), 0.7184489402697495), (('voxaudio', 2), 0.6964458180413367), (('voxaudio', 4), 0.6875), (('voxaudio', 9), 0.6730000974373965), (('text_video', 0), 0.700626204238921), (('text_video', 2), 0.6817172107398107), (('text_video', 4), 0.679124709527498), (('text_video', 9), 0.6600409237065186), (('text_voxaudio', 0), 0.7185452793834297), (('text_voxaudio', 2), 0.6949971025690554), (('text_voxaudio', 4), 0.6896785437645236), (('text_voxaudio', 9), 0.6759232193315795), (('video_voxaudio', 0), 0.7144508670520231), (('video_voxaudio', 2), 0.6904577940892409), (('video_voxaudio', 4), 0.685660340821069), (('video_voxaudio', 9), 0.6716846925850142), (('text_video_voxaudio', 0), 0.7160886319845857), (('text_video_voxaudio', 2), 0.6926791578134055), (('text_video_voxaudio', 4), 0.6870158791634392), (('text_video_voxaudio', 9), 0.675095001461561)], 10: [(('text', 0), 0.688899660688318), (('text', 2), 0.6721255709981534), (('text', 4), 0.6631589203936471), (('text', 9), 0.652137674053736), (('video', 0), 0.6544352884149297), (('video', 2), 0.6511808727767519), (('video', 4), 0.6516125889116243), (('video', 9), 0.6490978623259462), (('voxaudio', 0), 0.7102278235579254), (('voxaudio', 2), 0.6945767324326951), (('voxaudio', 4), 0.684887459807074), (('voxaudio', 9), 0.6706707197489704), (('text_video', 0), 0.6922927775084828), (('text_video', 2), 0.6725629312858392), (('text_video', 4), 0.6617460781447919), (('text_video', 9), 0.6502255344185134), (('text_voxaudio', 0), 0.7082888996606883), (('text_voxaudio', 2), 0.6966177471085625), (('text_voxaudio', 4), 0.6887362369677482), (('text_voxaudio', 9), 0.6742498529123357), (('video_voxaudio', 0), 0.7053805138148328), (('video_voxaudio', 2), 0.6922927398192245), (('video_voxaudio', 4), 0.6845464289194193), (('video_voxaudio', 9), 0.6714061580702099), (('text_video_voxaudio', 0), 0.7101793504604944), (('text_video_voxaudio', 2), 0.6930702692195548), (('text_video_voxaudio', 4), 0.6864951768488746), (('text_video_voxaudio', 9), 0.6709648950774662)], 30: [(('text', 0), 0.662357036300348), (('text', 2), 0.6535546913949546), (('text', 4), 0.650554833549935), (('text', 9), 0.6488730126786074), (('video', 0), 0.6575832918945798), (('video', 2), 0.6522584504935687), (('video', 4), 0.6504548635409377), (('video', 9), 0.6497786274904407), (('voxaudio', 0), 0.6884137245151666), (('voxaudio', 2), 0.6781334131020041), (('voxaudio', 4), 0.672148355493352), (('voxaudio', 9), 0.6627591064600523), (('text_video', 0), 0.6686723023371457), (('text_video', 2), 0.6561970286170107), (('text_video', 4), 0.6527541737478756), (('text_video', 9), 0.6531998390018112), (('text_voxaudio', 0), 0.6898060666335156), (('text_voxaudio', 2), 0.6803769069697876), (('text_voxaudio', 4), 0.6727981605518345), (('text_voxaudio', 9), 0.6658281344334877), (('video_voxaudio', 0), 0.6832421680755842), (('video_voxaudio', 2), 0.6807258949047762), (('video_voxaudio', 4), 0.6729481155653304), (('video_voxaudio', 9), 0.6592372710807003), (('text_video_voxaudio', 0), 0.6859771258080557), (('text_video_voxaudio', 2), 0.680576328646924), (('text_video_voxaudio', 4), 0.676447065880236), (('text_video_voxaudio', 9), 0.6611994365063393)]} , "use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: RandomForestClassifier" : {4: [(('text', 0), 0.6572981572318706), (('text', 2), 0.6097009966777409), (('text', 4), 0.5905155188490742), (('text', 9), 0.5745108549986598), (('video', 0), 0.6152724380220072), (('video', 2), 0.5720930232558139), (('video', 4), 0.5696017050752631), (('video', 9), 0.5682122755293487), (('voxaudio', 0), 0.667638870475938), (('voxaudio', 2), 0.6188704318936877), (('voxaudio', 4), 0.5997069401891568), (('voxaudio', 9), 0.575984990619137), (('text_video', 0), 0.6550444120376508), (('text_video', 2), 0.6122259136212624), (('text_video', 4), 0.6015718662581591), (('text_video', 9), 0.5869740016081479), (('text_voxaudio', 0), 0.6740023863184409), (('text_voxaudio', 2), 0.6287043189368771), (('text_voxaudio', 4), 0.5985080591447982), (('text_voxaudio', 9), 0.5812114714553739), (('video_voxaudio', 0), 0.6631313800874984), (('video_voxaudio', 2), 0.6220598006644519), (('video_voxaudio', 4), 0.6054349274010923), (('video_voxaudio', 9), 0.5904583221656392), (('text_video_voxaudio', 0), 0.6705554819037518), (('text_video_voxaudio', 2), 0.6252491694352159), (('text_video_voxaudio', 4), 0.6022379112828027), (('text_video_voxaudio', 9), 0.5905923344947736)], 5: [(('text', 0), 0.6452083886381736), (('text', 2), 0.6053751995742416), (('text', 4), 0.5886903174179782), (('text', 9), 0.5729236549040655), (('video', 0), 0.6191929917706398), (('video', 2), 0.5538850452368281), (('video', 4), 0.5732195252067218), (('video', 9), 0.5376358513350329), (('voxaudio', 0), 0.6644544730554818), (('voxaudio', 2), 0.6248004257583821), (('voxaudio', 4), 0.6068284875966925), (('voxaudio', 9), 0.5754729639071515), (('text_video', 0), 0.6521104327050703), (('text_video', 2), 0.6093666844065992), (('text_video', 4), 0.6009602560682848), (('text_video', 9), 0.5823158459680665), (('text_voxaudio', 0), 0.6712237855057075), (('text_voxaudio', 2), 0.6265300691857371), (('text_voxaudio', 4), 0.6080288076820486), (('text_voxaudio', 9), 0.5786931437005233), (('video_voxaudio', 0), 0.6607379877886913), (('video_voxaudio', 2), 0.6218733368813198), (('video_voxaudio', 4), 0.5998933048813017), (('video_voxaudio', 9), 0.5862068965517241), (('text_video_voxaudio', 0), 0.6649853995221662), (('text_video_voxaudio', 2), 0.6245343267695582), (('text_video_voxaudio', 4), 0.6044278474259802), (('text_video_voxaudio', 9), 0.5858043740775527)], 10: [(('text', 0), 0.6126318600614234), (('text', 2), 0.5980457770044171), (('text', 4), 0.5891587280289816), (('text', 9), 0.5599352051835853), (('video', 0), 0.568166644411804), (('video', 2), 0.5530718779279882), (('video', 4), 0.5466255199248625), (('video', 9), 0.5241630669546437), (('voxaudio', 0), 0.6412071037521698), (('voxaudio', 2), 0.6138401820372106), (('voxaudio', 4), 0.5965383067221253), (('voxaudio', 9), 0.5712742980561555), (('text_video', 0), 0.6287888903725464), (('text_video', 2), 0.5876054075759604), (('text_video', 4), 0.5737286998524084), (('text_video', 9), 0.5533207343412527), (('text_voxaudio', 0), 0.6504206169047937), (('text_voxaudio', 2), 0.6230758934546915), (('text_voxaudio', 4), 0.6048571045216691), (('text_voxaudio', 9), 0.5770788336933045), (('video_voxaudio', 0), 0.6409400453999199), (('video_voxaudio', 2), 0.6216035336634989), (('video_voxaudio', 4), 0.6055279753119549), (('video_voxaudio', 9), 0.562634989200864), (('text_video_voxaudio', 0), 0.6465482707971691), (('text_video_voxaudio', 2), 0.6174541560701379), (('text_video_voxaudio', 4), 0.6043204078894405), (('text_video_voxaudio', 9), 0.5758639308855291)], 30: [(('text', 0), 0.5627308797373102), (('text', 2), 0.5539706487450281), (('text', 4), 0.5589165406297264), (('text', 9), 0.5441339236303265), (('video', 0), 0.5688876727322479), (('video', 2), 0.5685091208339048), (('video', 4), 0.5644163343874604), (('video', 9), 0.5268400664084117), (('voxaudio', 0), 0.6018607196606923), (('voxaudio', 2), 0.5852420792758195), (('voxaudio', 4), 0.5751409322150419), (('voxaudio', 9), 0.5437188710570006), (('text_video', 0), 0.6032288958817896), (('text_video', 2), 0.5862021670552736), (('text_video', 4), 0.5905403547366974), (('text_video', 9), 0.5528500276701716), (('text_voxaudio', 0), 0.6084279655219592), (('text_voxaudio', 2), 0.5866136332464682), (('text_voxaudio', 4), 0.5831156331637564), (('text_voxaudio', 9), 0.5536801328168235), (('video_voxaudio', 0), 0.5973457381310713), (('video_voxaudio', 2), 0.583596214511041), (('video_voxaudio', 4), 0.5780283239378523), (('video_voxaudio', 9), 0.54344216934145), (('text_video_voxaudio', 0), 0.6121220413189219), (('text_video_voxaudio', 2), 0.5882594980112468), (('text_video_voxaudio', 4), 0.582565653787983), (('text_video_voxaudio', 9), 0.543027116768124)]} , "use_crossval: True | use_all_perspectives: True | predict_only_host: True | clf: GaussianNB(priors=None, var_smoothing=1e-09)" : {4: [(('text', 0), 0.6495421578279535), (('text', 2), 0.5878564811484824), (('text', 4), 0.5533003759881916), (('text', 9), 0.5189191297826732), (('video', 0), 0.5192359413820778), (('video', 2), 0.5051804947521831), (('video', 4), 0.4972346271973011), (('video', 9), 0.49625909688412584), (('voxaudio', 0), 0.6847674467968973), (('voxaudio', 2), 0.623508876445786), (('voxaudio', 4), 0.5934301380226243), (('voxaudio', 9), 0.5547619737142593), (('text_video', 0), 0.5598077909248287), (('text_video', 2), 0.5242564040462163), (('text_video', 4), 0.5126536453371967), (('text_video', 9), 0.4989114560751437), (('text_voxaudio', 0), 0.6856497951795943), (('text_voxaudio', 2), 0.623586628906604), (('text_voxaudio', 4), 0.5962215354146921), (('text_voxaudio', 9), 0.5555759631540683), (('video_voxaudio', 0), 0.6439362344447342), (('video_voxaudio', 2), 0.5791904546527735), (('video_voxaudio', 4), 0.5506154899432978), (('video_voxaudio', 9), 0.5195239039885351), (('text_video_voxaudio', 0), 0.6570448111602356), (('text_video_voxaudio', 2), 0.5884028140972526), (('text_video_voxaudio', 4), 0.5567203312580375), (('text_video_voxaudio', 9), 0.5212832267605743)], 5: [(('text', 0), 0.6293332682577489), (('text', 2), 0.5725381480733935), (('text', 4), 0.5509984628634959), (('text', 9), 0.5146047308539099), (('video', 0), 0.5131997544034145), (('video', 2), 0.5009821759299846), (('video', 4), 0.49538954697376675), (('video', 9), 0.48720497403446644), (('voxaudio', 0), 0.6781307293621237), (('voxaudio', 2), 0.6240455462862923), (('voxaudio', 4), 0.5953664608759018), (('voxaudio', 9), 0.556584695911876), (('text_video', 0), 0.5485174753508428), (('text_video', 2), 0.5159269842859872), (('text_video', 4), 0.5053696658950353), (('text_video', 9), 0.49167687778685903), (('text_voxaudio', 0), 0.6758935655362045), (('text_voxaudio', 2), 0.6219146645590208), (('text_voxaudio', 4), 0.5929198799364814), (('text_voxaudio', 9), 0.5542064601463389), (('video_voxaudio', 0), 0.6367992693470291), (('video_voxaudio', 2), 0.5803327672389804), (('video_voxaudio', 4), 0.5515348460754879), (('video_voxaudio', 9), 0.5148602571679588), (('text_video_voxaudio', 0), 0.6443497027146645), (('text_video_voxaudio', 2), 0.5869780437216483), (('text_video_voxaudio', 4), 0.5575232676128528), (('text_video_voxaudio', 9), 0.5180867523869178)], 10: [(('text', 0), 0.6178053693988834), (('text', 2), 0.5818258581896032), (('text', 4), 0.5585227779257285), (('text', 9), 0.5312027592742684), (('video', 0), 0.510017427611629), (('video', 2), 0.5000424730897097), (('video', 4), 0.4949456954532815), (('video', 9), 0.49452904971774253), (('voxaudio', 0), 0.6384634536644452), (('voxaudio', 2), 0.598868891473415), (('voxaudio', 4), 0.5768354206541544), (('voxaudio', 9), 0.5456873438450707), (('text_video', 0), 0.5604133805155787), (('text_video', 2), 0.5263519468901487), (('text_video', 4), 0.5140485921163134), (('text_video', 9), 0.5038823293744128), (('text_voxaudio', 0), 0.6388698291130707), (('text_voxaudio', 2), 0.5970204153245715), (('text_voxaudio', 4), 0.5751184652218585), (('text_voxaudio', 9), 0.5435472004175821), (('video_voxaudio', 0), 0.605389037026129), (('video_voxaudio', 2), 0.5620124601398042), (('video_voxaudio', 4), 0.5398553919339418), (('video_voxaudio', 9), 0.5178999911351139), (('text_video_voxaudio', 0), 0.6141996704576366), (('text_video_voxaudio', 2), 0.5720691790307317), (('text_video_voxaudio', 4), 0.5493119402730458), (('text_video_voxaudio', 9), 0.5231664349198406)], 30: [(('text', 0), 0.5786259010933945), (('text', 2), 0.5620501700560834), (('text', 4), 0.5459432048681542), (('text', 9), 0.5281201234374271), (('video', 0), 0.4919217145786135), (('video', 2), 0.48577166850293196), (('video', 4), 0.484370943842948), (('video', 9), 0.4862690727222514), (('voxaudio', 0), 0.5708241681394307), (('voxaudio', 2), 0.5476818879818169), (('voxaudio', 4), 0.5343189422756695), (('voxaudio', 9), 0.5137034411128298), (('text_video', 0), 0.5322858197359758), (('text_video', 2), 0.5145081748940191), (('text_video', 4), 0.5062856580242049), (('text_video', 9), 0.4901602111269184), (('text_voxaudio', 0), 0.5805626102181413), (('text_voxaudio', 2), 0.5579727078321679), (('text_voxaudio', 4), 0.5434962617806927), (('text_voxaudio', 9), 0.5211499126398458), (('video_voxaudio', 0), 0.5331987019395331), (('video_voxaudio', 2), 0.5128989789300309), (('video_voxaudio', 4), 0.5040330469631348), (('video_voxaudio', 9), 0.4883124401513572), (('text_video_voxaudio', 0), 0.5504345663224062), (('text_video_voxaudio', 2), 0.5306792466098086), (('text_video_voxaudio', 4), 0.51727122597475), (('text_video_voxaudio', 9), 0.49497493921464975)]} , "use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: GaussianNB(priors=None, var_smoothing=1e-09)" : {4: [(('text', 0), 0.671555662971522), (('text', 2), 0.6123672704559651), (('text', 4), 0.599591738660769), (('text', 9), 0.5748549256198585), (('video', 0), 0.5576084483926695), (('video', 2), 0.5461145607524709), (('video', 4), 0.542276239221862), (('video', 9), 0.5310812002880885), (('voxaudio', 0), 0.7031173301892262), (('voxaudio', 2), 0.6603813792850057), (('voxaudio', 4), 0.6436909856762745), (('voxaudio', 9), 0.625969299650994), (('text_video', 0), 0.6414308004973109), (('text_video', 2), 0.6015064114340302), (('text_video', 4), 0.585388173515563), (('text_video', 9), 0.5584102798197941), (('text_voxaudio', 0), 0.6990200529449961), (('text_voxaudio', 2), 0.643935775434471), (('text_voxaudio', 4), 0.6280211827849997), (('text_voxaudio', 9), 0.6061136811592092), (('video_voxaudio', 0), 0.6645193458483327), (('video_voxaudio', 2), 0.6246463607304259), (('video_voxaudio', 4), 0.6089922861517474), (('video_voxaudio', 9), 0.5845457285507417), (('text_video_voxaudio', 0), 0.6900486348399804), (('text_video_voxaudio', 2), 0.6407980306426131), (('text_video_voxaudio', 4), 0.6231443382952732), (('text_video_voxaudio', 9), 0.5955040202445318)], 5: [(('text', 0), 0.6768598986991343), (('text', 2), 0.6358361885295272), (('text', 4), 0.6157670525686695), (('text', 9), 0.5903164393472323), (('video', 0), 0.5497097132219093), (('video', 2), 0.538431091174597), (('video', 4), 0.5336887848726968), (('video', 9), 0.5216805206621352), (('voxaudio', 0), 0.6997071686488399), (('voxaudio', 2), 0.6626996172497414), (('voxaudio', 4), 0.6455940272865319), (('voxaudio', 9), 0.6282407319458313), (('text_video', 0), 0.6193035772083382), (('text_video', 2), 0.5861956819922309), (('text_video', 4), 0.5717011667920494), (('text_video', 9), 0.54240555986723), (('text_voxaudio', 0), 0.6949345463957047), (('text_voxaudio', 2), 0.6543679143796668), (('text_voxaudio', 4), 0.6347353029760334), (('text_voxaudio', 9), 0.6090486509604645), (('video_voxaudio', 0), 0.6533998675490608), (('video_voxaudio', 2), 0.618162936158367), (('video_voxaudio', 4), 0.6015280675745843), (('video_voxaudio', 9), 0.5752786693449676), (('text_video_voxaudio', 0), 0.678201085546779), (('text_video_voxaudio', 2), 0.6368692326877674), (('text_video_voxaudio', 4), 0.6169705213430655), (('text_video_voxaudio', 9), 0.5841696274003094)], 10: [(('text', 0), 0.5811783428377613), (('text', 2), 0.523266273394551), (('text', 4), 0.49142406670047645), (('text', 9), 0.45736721161926097), (('video', 0), 0.5408929586124918), (('video', 2), 0.530378690668295), (('video', 4), 0.5250830214934218), (('video', 9), 0.5182381968600402), (('voxaudio', 0), 0.6798408298789592), (('voxaudio', 2), 0.6582110517150042), (('voxaudio', 4), 0.6497171748639153), (('voxaudio', 9), 0.6374654655431622), (('text_video', 0), 0.5767013863623583), (('text_video', 2), 0.552737957725175), (('text_video', 4), 0.5395351125560695), (('text_video', 9), 0.5185697731498239), (('text_voxaudio', 0), 0.642117552145183), (('text_voxaudio', 2), 0.6137364720771329), (('text_voxaudio', 4), 0.5982409382357182), (('text_voxaudio', 9), 0.5687714464323997), (('video_voxaudio', 0), 0.6347275506172159), (('video_voxaudio', 2), 0.6115731744271695), (('video_voxaudio', 4), 0.5986092994389285), (('video_voxaudio', 9), 0.5819537627177971), (('text_video_voxaudio', 0), 0.6386857835879949), (('text_video_voxaudio', 2), 0.6098677277725841), (('text_video_voxaudio', 4), 0.5930889724087021), (('text_video_voxaudio', 9), 0.5651757764826092)], 30: [(('text', 0), 0.5229911514822895), (('text', 2), 0.5063262187497839), (('text', 4), 0.49713388955774623), (('text', 9), 0.4801891871949933), (('video', 0), 0.5305670958281145), (('video', 2), 0.5242317016369821), (('video', 4), 0.5226766517726793), (('video', 9), 0.5208450345767573), (('voxaudio', 0), 0.6571161993204042), (('voxaudio', 2), 0.6512731066345603), (('voxaudio', 4), 0.6473285816123214), (('voxaudio', 9), 0.6398188130665725), (('text_video', 0), 0.552516997540663), (('text_video', 2), 0.5344939702512123), (('text_video', 4), 0.527451718677359), (('text_video', 9), 0.5162805620124757), (('text_voxaudio', 0), 0.565832251054358), (('text_voxaudio', 2), 0.5562060298449196), (('text_voxaudio', 4), 0.5529697529402194), (('text_voxaudio', 9), 0.5563478781742364), (('video_voxaudio', 0), 0.6093084253641647), (('video_voxaudio', 2), 0.6045274376866789), (('video_voxaudio', 4), 0.6015183879038801), (('video_voxaudio', 9), 0.5949912370783761), (('text_video_voxaudio', 0), 0.5824986991977159), (('text_video_voxaudio', 2), 0.5704493027120277), (('text_video_voxaudio', 4), 0.5656089226061074), (('text_video_voxaudio', 9), 0.5629714896852759)]} , "use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: GaussianNB(priors=None, var_smoothing=1e-09)" : {4: [(('text', 0), 0.6622354344086439), (('text', 2), 0.6142788698789332), (('text', 4), 0.5923248634569396), (('text', 9), 0.5455600786798769), (('video', 0), 0.5240508663618886), (('video', 2), 0.5090461351094048), (('video', 4), 0.49847979645342244), (('video', 9), 0.4894046457326583), (('voxaudio', 0), 0.6771201317494551), (('voxaudio', 2), 0.6203176995184589), (('voxaudio', 4), 0.5945618302074402), (('voxaudio', 9), 0.5617720058872255), (('text_video', 0), 0.5821584791200417), (('text_video', 2), 0.5398112164970759), (('text_video', 4), 0.5205604797468495), (('text_video', 9), 0.49861578594143746), (('text_voxaudio', 0), 0.6760991881133678), (('text_voxaudio', 2), 0.6201332567538473), (('text_voxaudio', 4), 0.5955056682549278), (('text_voxaudio', 9), 0.563589632928027), (('video_voxaudio', 0), 0.6485759811538749), (('video_voxaudio', 2), 0.5886099146338433), (('video_voxaudio', 4), 0.5589765688518213), (('video_voxaudio', 9), 0.5223868888140804), (('text_video_voxaudio', 0), 0.6583969287919709), (('text_video_voxaudio', 2), 0.5982510696875998), (('text_video_voxaudio', 4), 0.5694424937237746), (('text_video_voxaudio', 9), 0.5298219674420256)], 5: [(('text', 0), 0.6535871545054965), (('text', 2), 0.6111295208590596), (('text', 4), 0.568993791104759), (('text', 9), 0.520637014787835), (('video', 0), 0.5260404266999892), (('video', 2), 0.5069270990076472), (('video', 4), 0.49768385549275845), (('video', 9), 0.47967816202561026), (('voxaudio', 0), 0.6666801714114434), (('voxaudio', 2), 0.6167046371470442), (('voxaudio', 4), 0.591227209178731), (('voxaudio', 9), 0.559156959378497), (('text_video', 0), 0.582291906102476), (('text_video', 2), 0.5430826146473824), (('text_video', 4), 0.5248602419559717), (('text_video', 9), 0.4975638683983908), (('text_voxaudio', 0), 0.6663066606981422), (('text_voxaudio', 2), 0.616184648814588), (('text_voxaudio', 4), 0.5916235792500611), (('text_voxaudio', 9), 0.5606056614539958), (('video_voxaudio', 0), 0.6390827744407732), (('video_voxaudio', 2), 0.5866435380202816), (('video_voxaudio', 4), 0.559461187570421), (('video_voxaudio', 9), 0.520403441325134), (('text_video_voxaudio', 0), 0.6467601325036558), (('text_video_voxaudio', 2), 0.593987064747505), (('text_video_voxaudio', 4), 0.5689512043585678), (('text_video_voxaudio', 9), 0.5316767639961519)], 10: [(('text', 0), 0.6091316039686244), (('text', 2), 0.5790363707669173), (('text', 4), 0.5628004786581935), (('text', 9), 0.5248781354501963), (('video', 0), 0.5075176118373624), (('video', 2), 0.49618828097658874), (('video', 4), 0.4881475948200252), (('video', 9), 0.4878736164943245), (('voxaudio', 0), 0.6273059071638538), (('voxaudio', 2), 0.5913273208110502), (('voxaudio', 4), 0.5738577217193968), (('voxaudio', 9), 0.542428994610573), (('text_video', 0), 0.5610368935749892), (('text_video', 2), 0.5322051985072983), (('text_video', 4), 0.517624912722511), (('text_video', 9), 0.5023113315133424), (('text_voxaudio', 0), 0.6252129214063562), (('text_voxaudio', 2), 0.589400130539448), (('text_voxaudio', 4), 0.5717749786491118), (('text_voxaudio', 9), 0.5417160885125941), (('video_voxaudio', 0), 0.5914268661454747), (('video_voxaudio', 2), 0.5576223723104609), (('video_voxaudio', 4), 0.5378931174952077), (('video_voxaudio', 9), 0.512859047129431), (('text_video_voxaudio', 0), 0.6063553092542625), (('text_video_voxaudio', 2), 0.5664234397896134), (('text_video_voxaudio', 4), 0.5475977842097561), (('text_video_voxaudio', 9), 0.5188465258383493)], 30: [(('text', 0), 0.5547095370468154), (('text', 2), 0.5382018334181828), (('text', 4), 0.5282599383772776), (('text', 9), 0.5018243774483537), (('video', 0), 0.48423122995357276), (('video', 2), 0.47900144954187496), (('video', 4), 0.4787761544616636), (('video', 9), 0.47435418538114515), (('voxaudio', 0), 0.5661410978314572), (('voxaudio', 2), 0.5460479365622382), (('voxaudio', 4), 0.5310732782662514), (('voxaudio', 9), 0.4972094232045915), (('text_video', 0), 0.5175507546662015), (('text_video', 2), 0.5013922657603975), (('text_video', 4), 0.4929324232599769), (('text_video', 9), 0.4763711432474424), (('text_voxaudio', 0), 0.5682833999527865), (('text_voxaudio', 2), 0.5462183211275046), (('text_voxaudio', 4), 0.5330565138302108), (('text_voxaudio', 9), 0.4975313991123021), (('video_voxaudio', 0), 0.5246982577947348), (('video_voxaudio', 2), 0.506665861868644), (('video_voxaudio', 4), 0.4981340864390632), (('video_voxaudio', 9), 0.4755560540877834), (('text_video_voxaudio', 0), 0.5338182770669715), (('text_video_voxaudio', 2), 0.5120244287004865), (('text_video_voxaudio', 4), 0.5026541013377209), (('text_video_voxaudio', 9), 0.47628555959788327)]} , "use_crossval: True | use_all_perspectives: True | predict_only_host: True | clf: RandomForestClassifier" : {4: [(('text', 0, True), 0.67783629596835), (('text', 2, True), 0.6144518051913449), (('text', 4, True), 0.5827583073237359), (('text', 9, True), 0.5440221756288179), (('video', 0, True), 0.5220127252131594), (('video', 2, True), 0.49546872474992226), (('video', 4, True), 0.48366289692197373), (('video', 9, True), 0.47165316303756644), (('voxaudio', 0, True), 0.6741506742327911), (('voxaudio', 2, True), 0.6121618733087315), (('voxaudio', 4, True), 0.5759219637525747), (('voxaudio', 9, True), 0.5328358007243859), (('text_video', 0, True), 0.6351619897262973), (('text_video', 2, True), 0.5641198672031316), (('text_video', 4, True), 0.5291917094009466), (('text_video', 9, True), 0.48927205827191056), (('text_voxaudio', 0, True), 0.6822754169016761), (('text_voxaudio', 2, True), 0.6151803823226666), (('text_voxaudio', 4, True), 0.5814273620929548), (('text_voxaudio', 9, True), 0.5356455402168745), (('video_voxaudio', 0, True), 0.6593807341458764), (('video_voxaudio', 2, True), 0.5817645893891189), (('video_voxaudio', 4, True), 0.5425760708207454), (('video_voxaudio', 9, True), 0.4951279024804093), (('text_video_voxaudio', 0, True), 0.6631965190975907), (('text_video_voxaudio', 2, True), 0.5819724349916957), (('text_video_voxaudio', 4, True), 0.545942179901647), (('text_video_voxaudio', 9, True), 0.5004320451102852)], 5: [(('text', 0, True), 0.6647075352313838), (('text', 2, True), 0.6065772319362133), (('text', 4, True), 0.5746445043289101), (('text', 9, True), 0.5340233398307933), (('video', 0, True), 0.5333046971050114), (('video', 2, True), 0.5103192377438239), (('video', 4, True), 0.49431962629374143), (('video', 9, True), 0.48083410524034714), (('voxaudio', 0, True), 0.6667487619351633), (('voxaudio', 2, True), 0.609212845527164), (('voxaudio', 4, True), 0.5791713952457341), (('voxaudio', 9, True), 0.5377315173309797), (('text_video', 0, True), 0.6318496558342174), (('text_video', 2, True), 0.5708839280652617), (('text_video', 4, True), 0.5443924121241183), (('text_video', 9, True), 0.5009030579101623), (('text_voxaudio', 0, True), 0.670663719511157), (('text_voxaudio', 2, True), 0.6088203313393571), (('text_voxaudio', 4, True), 0.5791149507047979), (('text_voxaudio', 9, True), 0.5394293100789612), (('video_voxaudio', 0, True), 0.643482589552182), (('video_voxaudio', 2, True), 0.5775849202465864), (('video_voxaudio', 4, True), 0.5439708032993714), (('video_voxaudio', 9, True), 0.5098476345408038), (('text_video_voxaudio', 0, True), 0.6472856629355674), (('text_video_voxaudio', 2, True), 0.5807534734445031), (('text_video_voxaudio', 4, True), 0.5495940724101922), (('text_video_voxaudio', 9, True), 0.5081214891759813)], 10: [(('text', 0, True), 0.6217353163044365), (('text', 2, True), 0.5789983159082841), (('text', 4, True), 0.5554703682894845), (('text', 9, True), 0.5223695452890634), (('video', 0, True), 0.5141902665612416), (('video', 2, True), 0.4987611950854303), (('video', 4, True), 0.4894669330179278), (('video', 9, True), 0.4771610791071995), (('voxaudio', 0, True), 0.6144697913652171), (('voxaudio', 2, True), 0.5744870106874262), (('voxaudio', 4, True), 0.5560149598238069), (('voxaudio', 9, True), 0.5241807152352224), (('text_video', 0, True), 0.5883348807785624), (('text_video', 2, True), 0.5486907566199977), (('text_video', 4, True), 0.5202610816787774), (('text_video', 9, True), 0.4868722795857644), (('text_voxaudio', 0, True), 0.6217619793543048), (('text_voxaudio', 2, True), 0.5795154440151944), (('text_voxaudio', 4, True), 0.555906365316291), (('text_voxaudio', 9, True), 0.5258263354558663), (('video_voxaudio', 0, True), 0.5961696914090915), (('video_voxaudio', 2, True), 0.5561117797151811), (('video_voxaudio', 4, True), 0.5330966365153085), (('video_voxaudio', 9, True), 0.49798197601992966), (('text_video_voxaudio', 0, True), 0.6035705523606986), (('text_video_voxaudio', 2, True), 0.5610589905316687), (('text_video_voxaudio', 4, True), 0.5359582140321958), (('text_video_voxaudio', 9, True), 0.49732402517471613)], 30: [(('text', 0, True), 0.5150213361849183), (('text', 2, True), 0.4970886445575925), (('text', 4, True), 0.489571150843758), (('text', 9, True), 0.47190634038501117), (('video', 0, True), 0.48835148195746847), (('video', 2, True), 0.48080598489574744), (('video', 4, True), 0.47800128745160836), (('video', 9, True), 0.46202402728439607), (('voxaudio', 0, True), 0.52996072182754), (('voxaudio', 2, True), 0.507462262800459), (('voxaudio', 4, True), 0.4966070243842376), (('voxaudio', 9, True), 0.4812285390771912), (('text_video', 0, True), 0.5089346754762735), (('text_video', 2, True), 0.4860767850703545), (('text_video', 4, True), 0.4759159074290892), (('text_video', 9, True), 0.4655788237738011), (('text_voxaudio', 0, True), 0.5348575307654144), (('text_voxaudio', 2, True), 0.5120670285914167), (('text_voxaudio', 4, True), 0.5023635325271408), (('text_voxaudio', 9, True), 0.4814246487871854), (('video_voxaudio', 0, True), 0.5272493703802132), (('video_voxaudio', 2, True), 0.5032741629632312), (('video_voxaudio', 4, True), 0.4935477992899603), (('video_voxaudio', 9, True), 0.47632897433528054), (('text_video_voxaudio', 0, True), 0.5301541949046096), (('text_video_voxaudio', 2, True), 0.5082396236568443), (('text_video_voxaudio', 4, True), 0.4961618690029203), (('text_video_voxaudio', 9, True), 0.47658105157313707)]} , "use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: RandomForestClassifier" : {4: [(('text', 0, False), 0.728346158476511),  (('text', 2, False), 0.6972156936839846),  (('text', 4, False), 0.6829334497024286),  (('text', 9, False), 0.6722420248806096),  (('video', 0, False), 0.6750150134699409),  (('video', 2, False), 0.6735000328582362),  (('video', 4, False), 0.6737297857629229),  (('video', 9, False), 0.673842701240878),  (('voxaudio', 0, False), 0.7381222129931121),  (('voxaudio', 2, False), 0.7080257535133339),  (('voxaudio', 4, False), 0.6970794477578149),  (('voxaudio', 9, False), 0.683943221881538),  (('text_video', 0, False), 0.7280160341462837),  (('text_video', 2, False), 0.6953319960724194),  (('text_video', 4, False), 0.6831036660411007),  (('text_video', 9, False), 0.6745275136517653),  (('text_voxaudio', 0, False), 0.7443855906610712),  (('text_voxaudio', 2, False), 0.7105458335315538),  (('text_voxaudio', 4, False), 0.6986190975834783),  (('text_voxaudio', 9, False), 0.6852614317826715),  (('video_voxaudio', 0, False), 0.7308177503510691),  (('video_voxaudio', 2, False), 0.7050135289355064),  (('video_voxaudio', 4, False), 0.6946381111604873),  (('video_voxaudio', 9, False), 0.6836093848296653),  (('text_video_voxaudio', 0, False), 0.7387146304061594),  (('text_video_voxaudio', 2, False), 0.7078305797303436),  (('text_video_voxaudio', 4, False), 0.6957609445886002),  (('text_video_voxaudio', 9, False), 0.6850559614982745)], 5: [(('text', 0, False), 0.723718307807475),  (('text', 2, False), 0.6970646005298672),  (('text', 4, False), 0.6819915699022789),  (('text', 9, False), 0.6726844506596017),  (('video', 0, False), 0.6708659589311796),  (('video', 2, False), 0.6699541163128337),  (('video', 4, False), 0.6700715926131344),  (('video', 9, False), 0.6700431055779225),  (('voxaudio', 0, False), 0.7347425319440493),  (('voxaudio', 2, False), 0.7066891075305419),  (('voxaudio', 4, False), 0.6973401523300063),  (('voxaudio', 9, False), 0.6847879255517864),  (('text_video', 0, False), 0.7235600610263138),  (('text_video', 2, False), 0.6919791062837628),  (('text_video', 4, False), 0.6791198785694856),  (('text_video', 9, False), 0.6711281073464196),  (('text_voxaudio', 0, False), 0.7395073814090356),  (('text_voxaudio', 2, False), 0.7090159222404184),  (('text_voxaudio', 4, False), 0.6982856150131064),  (('text_voxaudio', 9, False), 0.6848590722364418),  (('video_voxaudio', 0, False), 0.7257315695919035),  (('video_voxaudio', 2, False), 0.7035866964616941),  (('video_voxaudio', 4, False), 0.6931605771903763),  (('video_voxaudio', 9, False), 0.6814796308189172),  (('text_video_voxaudio', 0, False), 0.7333535132259934),  (('text_video_voxaudio', 2, False), 0.7045738222774289),  (('text_video_voxaudio', 4, False), 0.6947687905171265),  (('text_video_voxaudio', 9, False), 0.6826179844947434)], 10: [(('text', 0, False), 0.70814367343517),  (('text', 2, False), 0.6897359653883818),  (('text', 4, False), 0.6801732960061381),  (('text', 9, False), 0.6732745921673782),  (('video', 0, False), 0.6738114165058172),  (('video', 2, False), 0.6733503243563709),  (('video', 4, False), 0.6730537671877175),  (('video', 9, False), 0.6729561533043256),  (('voxaudio', 0, False), 0.7179729975418044),  (('voxaudio', 2, False), 0.7008701470281533),  (('voxaudio', 4, False), 0.692179956064044),  (('voxaudio', 9, False), 0.6827486601886121),  (('text_video', 0, False), 0.7043571394376901),  (('text_video', 2, False), 0.6841951619103849),  (('text_video', 4, False), 0.6763938311512042),  (('text_video', 9, False), 0.6739380439922581),  (('text_voxaudio', 0, False), 0.7192322716030786),  (('text_voxaudio', 2, False), 0.7017731801367044),  (('text_voxaudio', 4, False), 0.6948783506535667),  (('text_voxaudio', 9, False), 0.6835536458310205),  (('video_voxaudio', 0, False), 0.7121488730759703),  (('video_voxaudio', 2, False), 0.6977577957732919),  (('video_voxaudio', 4, False), 0.6909669670699734),  (('video_voxaudio', 9, False), 0.6826071154162424),  (('text_video_voxaudio', 0, False), 0.7147023441648235),  (('text_video_voxaudio', 2, False), 0.6996076805042508),  (('text_video_voxaudio', 4, False), 0.6924612044326401),  (('text_video_voxaudio', 9, False), 0.6826071075913551)], 30: [(('text', 0, False), 0.6751794048228492),  (('text', 2, False), 0.6734709029889541),  (('text', 4, False), 0.6729871015281452),  (('text', 9, False), 0.6714631664435162),  (('video', 0, False), 0.6725637677825941),  (('video', 2, False), 0.6713335734125561),  (('video', 4, False), 0.6717987031440273),  (('video', 9, False), 0.6726230159975495),  (('voxaudio', 0, False), 0.6869137821780837),  (('voxaudio', 2, False), 0.6798649295286294),  (('voxaudio', 4, False), 0.6741665435957058),  (('voxaudio', 9, False), 0.6674037656717902),  (('text_video', 0, False), 0.6752241783983501),  (('text_video', 2, False), 0.6744317885913522),  (('text_video', 4, False), 0.6742385811866373),  (('text_video', 9, False), 0.6741271808900233),  (('text_voxaudio', 0, False), 0.689081496910109),  (('text_voxaudio', 2, False), 0.6821639256658756),  (('text_voxaudio', 4, False), 0.6774798019170707),  (('text_voxaudio', 9, False), 0.6684639172085067),  (('video_voxaudio', 0, False), 0.687227262199325),  (('video_voxaudio', 2, False), 0.6814634067265907),  (('video_voxaudio', 4, False), 0.6767234556202345),  (('video_voxaudio', 9, False), 0.6734385489814915),  (('text_video_voxaudio', 0, False), 0.6898876643900969),  (('text_video_voxaudio', 2, False), 0.6830888574522793),  (('text_video_voxaudio', 4, False), 0.6782540693520611),  (('text_video_voxaudio', 9, False), 0.6731576505156371)]} , "use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: RandomForestClassifier" : {4: [(('text', 0, False), 0.6605408943780327), (('text', 2, False), 0.6084040582701247), (('text', 4, False), 0.5828435622404251), (('text', 9, False), 0.5509915308073883), (('video', 0, False), 0.5012431843398073), (('video', 2, False), 0.4717902067114358), (('video', 4, False), 0.45287885411318995), (('video', 9, False), 0.4372361138758124), (('voxaudio', 0, False), 0.6793044384150295), (('voxaudio', 2, False), 0.61671443347856), (('voxaudio', 4, False), 0.5826993913939733), (('voxaudio', 9, False), 0.538041653803295), (('text_video', 0, False), 0.6165810424440099), (('text_video', 2, False), 0.5455404760636684), (('text_video', 4, False), 0.5151806870656417), (('text_video', 9, False), 0.4739563047083795), (('text_voxaudio', 0, False), 0.6804682662611299), (('text_voxaudio', 2, False), 0.6180245288283019), (('text_voxaudio', 4, False), 0.5854903789424819), (('text_voxaudio', 9, False), 0.5429776072010492), (('video_voxaudio', 0, False), 0.6511275950784465), (('video_voxaudio', 2, False), 0.5689986069349391), (('video_voxaudio', 4, False), 0.5278427109638723), (('video_voxaudio', 9, False), 0.47831363279134054), (('text_video_voxaudio', 0, False), 0.6536800094047199), (('text_video_voxaudio', 2, False), 0.5740751923739511), (('text_video_voxaudio', 4, False), 0.5282736696033175), (('text_video_voxaudio', 9, False), 0.4839107900547173)], 5: [(('text', 0, False), 0.6478396251170686), (('text', 2, False), 0.6023087975141143), (('text', 4, False), 0.5799017516118312), (('text', 9, False), 0.5427200454420971), (('video', 0, False), 0.4979460107730963), (('video', 2, False), 0.47264339217425677), (('video', 4, False), 0.4578052515055243), (('video', 9, False), 0.4450820338300797), (('voxaudio', 0, False), 0.6675306280313421), (('voxaudio', 2, False), 0.6086948173435245), (('voxaudio', 4, False), 0.577502310747503), (('voxaudio', 9, False), 0.5326631205763586), (('text_video', 0, False), 0.6074399647572016), (('text_video', 2, False), 0.5411883649714933), (('text_video', 4, False), 0.51211510278386), (('text_video', 9, False), 0.4729177567657987), (('text_voxaudio', 0, False), 0.6704357111178151), (('text_voxaudio', 2, False), 0.6089446978312518), (('text_voxaudio', 4, False), 0.579296206734337), (('text_voxaudio', 9, False), 0.5353504738028798), (('video_voxaudio', 0, False), 0.6368834170668414), (('video_voxaudio', 2, False), 0.5630523307721247), (('video_voxaudio', 4, False), 0.5246917137332403), (('video_voxaudio', 9, False), 0.47657005713623823), (('text_video_voxaudio', 0, False), 0.6450381128526053), (('text_video_voxaudio', 2, False), 0.5654239761353619), (('text_video_voxaudio', 4, False), 0.5277369740409477), (('text_video_voxaudio', 9, False), 0.480244024752199)], 10: [(('text', 0, False), 0.5989443867563818), (('text', 2, False), 0.5699782958282434), (('text', 4, False), 0.5462463926149669), (('text', 9, False), 0.5191935276046132), (('video', 0, False), 0.4815910988523028), (('video', 2, False), 0.46107002649561935), (('video', 4, False), 0.45065780927117655), (('video', 9, False), 0.44109869035962623), (('voxaudio', 0, False), 0.6085760553817565), (('voxaudio', 2, False), 0.5640460227823392), (('voxaudio', 4, False), 0.5404270666290619), (('voxaudio', 9, False), 0.4964110634295532), (('text_video', 0, False), 0.5458307138036802), (('text_video', 2, False), 0.5057589930264077), (('text_video', 4, False), 0.4855895495850004), (('text_video', 9, False), 0.4562064653501399), (('text_voxaudio', 0, False), 0.6167342365816717), (('text_voxaudio', 2, False), 0.5678361062129553), (('text_voxaudio', 4, False), 0.5406846049470259), (('text_voxaudio', 9, False), 0.49960992318344993), (('video_voxaudio', 0, False), 0.5693017367464496), (('video_voxaudio', 2, False), 0.5219043581018068), (('video_voxaudio', 4, False), 0.49877212224463363), (('video_voxaudio', 9, False), 0.4685049948837866), (('text_video_voxaudio', 0, False), 0.5748116230714442), (('text_video_voxaudio', 2, False), 0.5275147075607829), (('text_video_voxaudio', 4, False), 0.5001894680998876), (('text_video_voxaudio', 9, False), 0.46673238648579635)], 30: [(('text', 0, False), 0.49398732976387566), (('text', 2, False), 0.4795111222872611), (('text', 4, False), 0.4648754628538028), (('text', 9, False), 0.4430008952032529), (('video', 0, False), 0.44717895997175744), (('video', 2, False), 0.43844921902806877), (('video', 4, False), 0.43852402079332087), (('video', 9, False), 0.43104645913501366), (('voxaudio', 0, False), 0.4874767101832786), (('voxaudio', 2, False), 0.4632645648460961), (('voxaudio', 4, False), 0.44969549906752215), (('voxaudio', 9, False), 0.43128286060313414), (('text_video', 0, False), 0.47016981859287255), (('text_video', 2, False), 0.4527597222793805), (('text_video', 4, False), 0.4441313511924375), (('text_video', 9, False), 0.4329993708891548), (('text_voxaudio', 0, False), 0.49133670215093056), (('text_voxaudio', 2, False), 0.4663903220318525), (('text_voxaudio', 4, False), 0.45103879401065977), (('text_voxaudio', 9, False), 0.4341368526915623), (('video_voxaudio', 0, False), 0.47604504592957786), (('video_voxaudio', 2, False), 0.4563323172481045), (('video_voxaudio', 4, False), 0.4486936009943211), (('video_voxaudio', 9, False), 0.43407199151642173), (('text_video_voxaudio', 0, False), 0.48043523359883267), (('text_video_voxaudio', 2, False), 0.4623501214634351), (('text_video_voxaudio', 4, False), 0.44922670683183574), (('text_video_voxaudio', 9, False), 0.43357870097973505)]} }
for k, v in all_results.items():
    print(k)
    table = make_table(v)

TASK: PREDICTING BINARY WILL-BE-SPEAKING LABEL (2 classes: speaking or not)
***************************************************************************

use_crossval: False | use_all_perspectives: True | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.623  0.567  0.539  0.551  0.535  0.534    0.648  0.569  0.540      0.595  0.555  0.544         0.657  0.581  0.546          0.635  0.569  0.562               0.642  0.577  0.561
5   0.618  0.552  0.513  0.569  0.562  0.561    0.641  0.573  0.545      0.602  0.577  0.570         0.650  0.577  0.543          0.626  0.580  0.574               0.639  0.589  0.574
10  0.579  0.534  0.514  0.576  0.566  0.546    0.613  0.567  0.554      0.600  0.575  0.551         0.614  0.564  0.546          0.611  0.582  0.568               0.616  0.586  0.572
30  0.567  0.551  0.530  0.561  0.553  0.547    0.581  0.557  0.547      0.573  0.557  0.548         0.573  0.552  0.535          0.579  0.560  0.558               0.582  0.564  0.551

use_crossval: False | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.645  0.572  0.551  0.566  0.542  0.540    0.675  0.622  0.609      0.629  0.578  0.556         0.672  0.607  0.591          0.647  0.597  0.577               0.664  0.605  0.582
5   0.650  0.591  0.568  0.565  0.548  0.546    0.670  0.625  0.614      0.621  0.578  0.563         0.666  0.613  0.592          0.644  0.598  0.582               0.662  0.610  0.590
10  0.594  0.550  0.453  0.552  0.534  0.530    0.648  0.630  0.631      0.589  0.558  0.540         0.611  0.572  0.550          0.621  0.591  0.581               0.628  0.591  0.569
30  0.497  0.479  0.461  0.544  0.536  0.536    0.659  0.652  0.640      0.550  0.530  0.519         0.532  0.523  0.533          0.604  0.599  0.598               0.582  0.565  0.562

use_crossval: False | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>


...

text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.679  0.583  0.544  0.523  0.481  0.469    0.676  0.577  0.534      0.638  0.528  0.493         0.683  0.582  0.536          0.660  0.539  0.500               0.662  0.542  0.499
5   0.665  0.577  0.535  0.530  0.488  0.482    0.667  0.579  0.539      0.631  0.541  0.499         0.671  0.577  0.540          0.645  0.547  0.511               0.649  0.550  0.509
10  0.621  0.554  0.524  0.513  0.486  0.477    0.616  0.558  0.524      0.588  0.520  0.489         0.621  0.556  0.523          0.600  0.533  0.498               0.603  0.537  0.501
30  0.518  0.489  0.473  0.492  0.475  0.466    0.529  0.497  0.484      0.511  0.479  0.472         0.534  0.498  0.479          0.525  0.495  0.473               0.526  0.495  0.476

use_crossval: True | use_all_perspectives: False | predict_only_host: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.719  0.671  0.653  0.652  0.648  0.647    0.733  0.688  0.675      0.719  0.669  0.652         0.736  0.689  0.675          0.725  0.683  0.671               0.733  0.685  0.672
5   0.712  0.668  0.653  0.648  0.643  0.641    0.728  0.688  0.673      0.710  0.665  0.649         0.732  0.688  0.673          0.720  0.682  0.669               0.726  0.682  0.669
10  0.697  0.667  0.657  0.650  0.647  0.646    0.710  0.682  0.670      0.693  0.661  0.653         0.711  0.683  0.672          0.702  0.678  0.665               0.705  0.678  0.667
30  0.656  0.651  0.650  0.641  0.639  0.640    0.667  0.654  0.645      0.653  0.646  0.644         0.669  0.655  0.644          0.663  0.653  0.643               0.667  0.652  0.644

use_crossval: True | use_all_perspectives: False | predict_only_host: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.661  0.584  0.550  0.501  0.452  0.440    0.678  0.584  0.536      0.617  0.517  0.471         0.680  0.584  0.538          0.651  0.530  0.480               0.655  0.531  0.482
5   0.649  0.577  0.543  0.496  0.460  0.446    0.666  0.576  0.531      0.607  0.509  0.471         0.670  0.578  0.533          0.636  0.526  0.479               0.643  0.528  0.479
10  0.600  0.548  0.516  0.480  0.452  0.442    0.608  0.540  0.497      0.544  0.487  0.457         0.615  0.541  0.501          0.569  0.498  0.465               0.578  0.501  0.466
30  0.493  0.466  0.442  0.446  0.441  0.432    0.487  0.447  0.429      0.472  0.443  0.434         0.488  0.451  0.429          0.477  0.450  0.432               0.482  0.450  0.430

In [620]:
title = "TASK: PREDICTING MULTI-SPEAKER MULTI-CLASS LABEL (4 classes: speaker1, speaker2, speaker3, nospeaker)"
print(title + "\n" + "*"*len(title) + "\n")
all_results = {"use_crossval: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.3832786252172234),   (('text', 2), 0.35521983343017627),   (('text', 4), 0.31668933359238394),   (('text', 9), 0.29978460935970236),   (('video', 0), 0.32168372272639506),   (('video', 2), 0.3257795855122991),   (('video', 4), 0.32329512337283856),   (('video', 9), 0.3127080477775602),   (('voxaudio', 0), 0.4997103687970651),   (('voxaudio', 2), 0.43947317451094325),   (('voxaudio', 4), 0.41266757334369536),   (('voxaudio', 9), 0.3796749559428236),   (('text_video', 0), 0.35682564201583317),   (('text_video', 2), 0.33430176254115823),   (('text_video', 4), 0.3291237614144162),   (('text_video', 9), 0.31486195418053653),   (('text_voxaudio', 0), 0.45356246379609966),   (('text_voxaudio', 2), 0.4096455549099361),   (('text_voxaudio', 4), 0.3677870604235477),   (('text_voxaudio', 9), 0.33326806344233406),   (('video_voxaudio', 0), 0.41243483297933964),   (('video_voxaudio', 2), 0.36684098392407516),   (('video_voxaudio', 4), 0.3495239945599378),   (('video_voxaudio', 9), 0.33326806344233406),   (('text_video_voxaudio', 0), 0.4186136319752848),   (('text_video_voxaudio', 2), 0.3664536122409452),   (('text_video_voxaudio', 4), 0.35418690499319994),   (('text_video_voxaudio', 9), 0.3322890150773448)],  5: [(('text', 0), 0.37710084033613445),   (('text', 2), 0.31952247191011235),   (('text', 4), 0.31672535211267605),   (('text', 9), 0.29609929078014185),   (('video', 0), 0.3326330532212885),   (('video', 2), 0.32145365168539325),   (('video', 4), 0.3202464788732394),   (('video', 9), 0.3047872340425532),   (('voxaudio', 0), 0.48704481792717086),   (('voxaudio', 2), 0.4234550561797753),   (('voxaudio', 4), 0.4052816901408451),   (('voxaudio', 9), 0.37854609929078015),   (('text_video', 0), 0.35171568627450983),   (('text_video', 2), 0.32303370786516855),   (('text_video', 4), 0.3227112676056338),   (('text_video', 9), 0.30531914893617024),   (('text_voxaudio', 0), 0.4565826330532213),   (('text_voxaudio', 2), 0.38395365168539325),   (('text_voxaudio', 4), 0.3737676056338028),   (('text_voxaudio', 9), 0.3313829787234043),   (('video_voxaudio', 0), 0.39898459383753504),   (('video_voxaudio', 2), 0.35410814606741575),   (('video_voxaudio', 4), 0.34066901408450706),   (('video_voxaudio', 9), 0.32269503546099293),   (('text_video_voxaudio', 0), 0.40843837535014005),   (('text_video_voxaudio', 2), 0.35252808988764045),   (('text_video_voxaudio', 4), 0.3436619718309859),   (('text_video_voxaudio', 9), 0.3170212765957447)],  10: [(('text', 0), 0.42066290550070523),   (('text', 2), 0.39002828854314003),   (('text', 4), 0.3567375886524823),   (('text', 9), 0.34285714285714286),   (('video', 0), 0.317524682651622),   (('video', 2), 0.3150636492220651),   (('video', 4), 0.3122340425531915),   (('video', 9), 0.29428571428571426),   (('voxaudio', 0), 0.43600141043723556),   (('voxaudio', 2), 0.39745403111739747),   (('voxaudio', 4), 0.3881205673758865),   (('voxaudio', 9), 0.3630357142857143),   (('text_video', 0), 0.35895627644569816),   (('text_video', 2), 0.33716407355021216),   (('text_video', 4), 0.3351063829787234),   (('text_video', 9), 0.3023214285714286),   (('text_voxaudio', 0), 0.45081100141043723),   (('text_voxaudio', 2), 0.40664780763790664),   (('text_voxaudio', 4), 0.376063829787234),   (('text_voxaudio', 9), 0.36178571428571427),   (('video_voxaudio', 0), 0.37323695345557123),   (('video_voxaudio', 2), 0.35148514851485146),   (('video_voxaudio', 4), 0.3448581560283688),   (('video_voxaudio', 9), 0.31339285714285714),   (('text_video_voxaudio', 0), 0.39580394922425954),   (('text_video_voxaudio', 2), 0.3668670438472419),   (('text_video_voxaudio', 4), 0.3574468085106383),   (('text_video_voxaudio', 9), 0.32107142857142856)],  30: [(('text', 0), 0.3047895500725689),   (('text', 2), 0.28457059679767105),   (('text', 4), 0.25985401459854013),   (('text', 9), 0.240625),   (('video', 0), 0.2997097242380261),   (('video', 2), 0.30585880640465796),   (('video', 4), 0.29744525547445255),   (('video', 9), 0.30974264705882354),   (('voxaudio', 0), 0.376088534107402),   (('voxaudio', 2), 0.34042940320232895),   (('voxaudio', 4), 0.3235401459854015),   (('voxaudio', 9), 0.29466911764705883),   (('text_video', 0), 0.3205732946298984),   (('text_video', 2), 0.30622270742358076),   (('text_video', 4), 0.2945255474452555),   (('text_video', 9), 0.28860294117647056),   (('text_voxaudio', 0), 0.3447024673439768),   (('text_voxaudio', 2), 0.31240902474526927),   (('text_voxaudio', 4), 0.28686131386861313),   (('text_voxaudio', 9), 0.26029411764705884),   (('video_voxaudio', 0), 0.33327285921625543),   (('video_voxaudio', 2), 0.3220524017467249),   (('video_voxaudio', 4), 0.3156934306569343),   (('video_voxaudio', 9), 0.32169117647058826),   (('text_video_voxaudio', 0), 0.34379535558780844),   (('text_video_voxaudio', 2), 0.325509461426492),   (('text_video_voxaudio', 4), 0.31715328467153286),   (('text_video_voxaudio', 9), 0.31231617647058824)]}, "use_crossval: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.5352384630237498),   (('text', 2), 0.484601975595584),   (('text', 4), 0.47250825723722556),   (('text', 9), 0.4534952026630116),   (('video', 0), 0.4711334234408187),   (('video', 2), 0.4600038737168313),   (('video', 4), 0.4596852535457548),   (('video', 9), 0.4593694928529469),   (('voxaudio', 0), 0.5425757868314346),   (('voxaudio', 2), 0.4935115243075731),   (('voxaudio', 4), 0.4783368952788032),   (('voxaudio', 9), 0.46289406696690816),   (('text_video', 0), 0.5311836261826608),   (('text_video', 2), 0.48169668797210924),   (('text_video', 4), 0.46531960365261316),   (('text_video', 9), 0.46446054435089096),   (('text_voxaudio', 0), 0.5398725622707087),   (('text_voxaudio', 2), 0.49293046678287816),   (('text_voxaudio', 4), 0.47891975908296097),   (('text_voxaudio', 9), 0.46034854121793617),   (('video_voxaudio', 0), 0.5466306236725237),   (('video_voxaudio', 2), 0.49312415262444315),   (('video_voxaudio', 4), 0.4781426073440839),   (('video_voxaudio', 9), 0.4662228314078716),   (('text_video_voxaudio', 0), 0.5437343116431743),   (('text_video_voxaudio', 2), 0.49428626767383305),   (('text_video_voxaudio', 4), 0.4769768797357684),   (('text_video_voxaudio', 9), 0.46543959271588015)],  5: [(('text', 0), 0.5169817927170869),   (('text', 2), 0.47349016853932585),   (('text', 4), 0.4623239436619718),   (('text', 9), 0.45),   (('video', 0), 0.4564075630252101),   (('video', 2), 0.45189606741573035),   (('video', 4), 0.4508802816901408),   (('video', 9), 0.4482269503546099),   (('voxaudio', 0), 0.5274859943977591),   (('voxaudio', 2), 0.4845505617977528),   (('voxaudio', 4), 0.461443661971831),   (('voxaudio', 9), 0.45195035460992905),   (('text_video', 0), 0.5133053221288515),   (('text_video', 2), 0.4748946629213483),   (('text_video', 4), 0.45985915492957746),   (('text_video', 9), 0.44556737588652484),   (('text_voxaudio', 0), 0.5148809523809523),   (('text_voxaudio', 2), 0.4808637640449438),   (('text_voxaudio', 4), 0.4572183098591549),   (('text_voxaudio', 9), 0.45372340425531915),   (('video_voxaudio', 0), 0.5281862745098039),   (('video_voxaudio', 2), 0.48191713483146065),   (('video_voxaudio', 4), 0.46566901408450706),   (('video_voxaudio', 9), 0.45656028368794327),   (('text_video_voxaudio', 0), 0.5162815126050421),   (('text_video_voxaudio', 2), 0.4820926966292135),   (('text_video_voxaudio', 4), 0.4625),   (('text_video_voxaudio', 9), 0.4533687943262411)],  10: [(('text', 0), 0.4922425952045134),   (('text', 2), 0.4662305516265912),   (('text', 4), 0.4524822695035461),   (('text', 9), 0.4475),   (('video', 0), 0.4585684062059238),   (('video', 2), 0.4524398868458274),   (('video', 4), 0.44858156028368795),   (('video', 9), 0.44857142857142857),   (('voxaudio', 0), 0.4938293370944993),   (('voxaudio', 2), 0.4618104667609618),   (('voxaudio', 4), 0.4521276595744681),   (('voxaudio', 9), 0.45125),   (('text_video', 0), 0.48730606488011285),   (('text_video', 2), 0.46304809052333806),   (('text_video', 4), 0.46063829787234045),   (('text_video', 9), 0.44910714285714287),   (('text_voxaudio', 0), 0.492771509167842),   (('text_voxaudio', 2), 0.471004243281471),   (('text_voxaudio', 4), 0.45691489361702126),   (('text_voxaudio', 9), 0.44785714285714284),   (('video_voxaudio', 0), 0.4975317348377997),   (('video_voxaudio', 2), 0.46304809052333806),   (('video_voxaudio', 4), 0.45868794326241136),   (('video_voxaudio', 9), 0.45053571428571426),   (('text_video_voxaudio', 0), 0.49100846262341324),   (('text_video_voxaudio', 2), 0.474009900990099),   (('text_video_voxaudio', 4), 0.4615248226950355),   (('text_video_voxaudio', 9), 0.45267857142857143)],  30: [(('text', 0), 0.45555152394775034),   (('text', 2), 0.44759825327510916),   (('text', 4), 0.44653284671532845),   (('text', 9), 0.44283088235294116),   (('video', 0), 0.44992743105950656),   (('video', 2), 0.45014556040756915),   (('video', 4), 0.44908759124087594),   (('video', 9), 0.44632352941176473),   (('voxaudio', 0), 0.4589985486211901),   (('voxaudio', 2), 0.4455967976710335),   (('voxaudio', 4), 0.4364963503649635),   (('voxaudio', 9), 0.4391544117647059),   (('text_video', 0), 0.45500725689404936),   (('text_video', 2), 0.45141921397379914),   (('text_video', 4), 0.44908759124087594),   (('text_video', 9), 0.44779411764705884),   (('text_voxaudio', 0), 0.46407837445573297),   (('text_voxaudio', 2), 0.4532387190684134),   (('text_voxaudio', 4), 0.44215328467153286),   (('text_voxaudio', 9), 0.4400735294117647),   (('video_voxaudio', 0), 0.4617198838896952),   (('video_voxaudio', 2), 0.4534206695778748),   (('video_voxaudio', 4), 0.4531021897810219),   (('video_voxaudio', 9), 0.4514705882352941),   (('text_video_voxaudio', 0), 0.4591799709724238),   (('text_video_voxaudio', 2), 0.45160116448326054),   (('text_video_voxaudio', 4), 0.4489051094890511),   (('text_video_voxaudio', 9), 0.44871323529411766)]}, "use_crossval: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>": {4: [(('text',     0),    0.47328011284199717),   (('text', 2), 0.42009985886675477),   (('text', 4), 0.3812053741139221),   (('text', 9), 0.34251248360361164),   (('video', 0), 0.3464497472000013),   (('video', 2), 0.33268281668722544),   (('video', 4), 0.3228921836876705),   (('video', 9), 0.31850701704356077),   (('voxaudio', 0), 0.5445880991652688),   (('voxaudio', 2), 0.4725381953334161),   (('voxaudio', 4), 0.4345332558323408),   (('voxaudio', 9), 0.38735920633411575),   (('text_video', 0), 0.38967112359829714),   (('text_video', 2), 0.34678927728022135),   (('text_video', 4), 0.3310851535801268),   (('text_video', 9), 0.31947950042514484),   (('text_voxaudio', 0), 0.5335030289222205),   (('text_voxaudio', 2), 0.46709845899963104),   (('text_voxaudio', 4), 0.42101793080629424),   (('text_voxaudio', 9), 0.3738104994021232),   (('video_voxaudio', 0), 0.4470651495953941),   (('video_voxaudio', 2), 0.38764695692106177),   (('video_voxaudio', 4), 0.35999387063220023),   (('video_voxaudio', 9), 0.33037734443405997),   (('text_video_voxaudio', 0), 0.4695703036201591),   (('text_video_voxaudio', 2), 0.39633915385770957),   (('text_video_voxaudio', 4), 0.365342852848258),   (('text_video_voxaudio', 9), 0.33016753281475925)],  5: [(('text', 0), 0.4694275984012341),   (('text', 2), 0.4088680252296566),   (('text', 4), 0.3740402683823255),   (('text', 9), 0.34411609770603074),   (('video', 0), 0.34577265446205263),   (('video', 2), 0.3296545204930509),   (('video', 4), 0.3192391946622637),   (('video', 9), 0.31076708068520736),   (('voxaudio', 0), 0.5331025811739564),   (('voxaudio', 2), 0.46475242502320785),   (('voxaudio', 4), 0.4298772689890303),   (('voxaudio', 9), 0.38663068193831895),   (('text_video', 0), 0.3853136707495169),   (('text_video', 2), 0.3472927063744698),   (('text_video', 4), 0.32666296572921105),   (('text_video', 9), 0.3129766104753921),   (('text_voxaudio', 0), 0.5223633170725014),   (('text_voxaudio', 2), 0.45148926173424575),   (('text_voxaudio', 4), 0.4153698870165961),   (('text_voxaudio', 9), 0.3693918838469124),   (('video_voxaudio', 0), 0.43282486141405246),   (('video_voxaudio', 2), 0.3792018999384052),   (('video_voxaudio', 4), 0.3540195975858112),   (('video_voxaudio', 9), 0.3282057805983774),   (('text_video_voxaudio', 0), 0.4511129205215879),   (('text_video_voxaudio', 2), 0.39184799169231715),   (('text_video_voxaudio', 4), 0.36141482866779545),   (('text_video_voxaudio', 9), 0.3286023166862545)],  10: [(('text', 0), 0.48314529386250504),   (('text', 2), 0.4380650790279713),   (('text', 4), 0.4060710700349247),   (('text', 9), 0.3703155520122249),   (('video', 0), 0.327641701557278),   (('video', 2), 0.3169740208373965),   (('video', 4), 0.3061972219070186),   (('video', 9), 0.2958140891851842),   (('voxaudio', 0), 0.47626043988104527),   (('voxaudio', 2), 0.4262958015041514),   (('voxaudio', 4), 0.4023648617380653),   (('voxaudio', 9), 0.3517443233679917),   (('text_video', 0), 0.37665658190512846),   (('text_video', 2), 0.34706323959179086),   (('text_video', 4), 0.33159555150302605),   (('text_video', 9), 0.307115367134048),   (('text_voxaudio', 0), 0.4942339667116188),   (('text_voxaudio', 2), 0.4431748716391306),   (('text_voxaudio', 4), 0.4069694171732873),   (('text_voxaudio', 9), 0.3700674572377843),   (('video_voxaudio', 0), 0.3889383796339288),   (('video_voxaudio', 2), 0.355979171423454),   (('video_voxaudio', 4), 0.3392263903365792),   (('video_voxaudio', 9), 0.3120805204463154),   (('text_video_voxaudio', 0), 0.41382555446545616),   (('text_video_voxaudio', 2), 0.37204333815537016),   (('text_video_voxaudio', 4), 0.35247053100954545),   (('text_video_voxaudio', 9), 0.32264183265229524)],  30: [(('text', 0), 0.3559453423250864),   (('text', 2), 0.3275266807004877),   (('text', 4), 0.29976649761033325),   (('text', 9), 0.2542077657694187),   (('video', 0), 0.2822727193586153),   (('video', 2), 0.27579688056332075),   (('video', 4), 0.2684245106778975),   (('video', 9), 0.26856811285776555),   (('voxaudio', 0), 0.35378725270335193),   (('voxaudio', 2), 0.3150438942848706),   (('voxaudio', 4), 0.29178484677345873),   (('voxaudio', 9), 0.24158111584529904),   (('text_video', 0), 0.31970336934358634),   (('text_video', 2), 0.297015250295548),   (('text_video', 4), 0.2854164820102498),   (('text_video', 9), 0.2696033712874484),   (('text_voxaudio', 0), 0.3834722907024821),   (('text_voxaudio', 2), 0.34203171641782854),   (('text_voxaudio', 4), 0.3068018104140496),   (('text_voxaudio', 9), 0.2507908587332751),   (('video_voxaudio', 0), 0.32089286906682724),   (('video_voxaudio', 2), 0.30511437700943955),   (('video_voxaudio', 4), 0.29248020676568737),   (('video_voxaudio', 9), 0.27993204956931683),   (('text_video_voxaudio', 0), 0.3407841476058759),   (('text_video_voxaudio', 2), 0.31679171339356965),   (('text_video_voxaudio', 4), 0.3013516424327839),   (('text_video_voxaudio', 9), 0.2783360110433907)]}, "use_crossval: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>": {4: [(('text',     0),    0.5732740661626561),   (('text', 2), 0.5148554576188002),   (('text', 4), 0.4858821992092956),   (('text', 9), 0.4613597338376147),   (('video', 0), 0.41518871850379285),   (('video', 2), 0.3975344526712269),   (('video', 4), 0.3914305711126639),   (('video', 9), 0.3891138473660516),   (('voxaudio', 0), 0.5641648816061338),   (('voxaudio', 2), 0.4935944466880535),   (('voxaudio', 4), 0.4607048339016913),   (('voxaudio', 9), 0.4419803432790501),   (('text_video', 0), 0.542280057938123),   (('text_video', 2), 0.4779519889355003),   (('text_video', 4), 0.4485712235325316),   (('text_video', 9), 0.41343150247576715),   (('text_voxaudio', 0), 0.571950921069565),   (('text_voxaudio', 2), 0.503274872030984),   (('text_voxaudio', 4), 0.4721056212906121),   (('text_voxaudio', 9), 0.44570910394777197),   (('video_voxaudio', 0), 0.5490829530607771),   (('video_voxaudio', 2), 0.4709791000761349),   (('video_voxaudio', 4), 0.44275388099290486),   (('video_voxaudio', 9), 0.41377408201569843),   (('text_video_voxaudio', 0), 0.5543250010744407),   (('text_video_voxaudio', 2), 0.48464038036262486),   (('text_video_voxaudio', 4), 0.4515718793327529),   (('text_video_voxaudio', 9), 0.42367387516251825)],  5: [(('text', 0), 0.5649559830820807),   (('text', 2), 0.5114941233088991),   (('text', 4), 0.4879358750820031),   (('text', 9), 0.46249278283679995),   (('video', 0), 0.41076273730741664),   (('video', 2), 0.3968929247307587),   (('video', 4), 0.3948680368582404),   (('video', 9), 0.3903064982093396),   (('voxaudio', 0), 0.5499681735784),   (('voxaudio', 2), 0.49365959179458885),   (('voxaudio', 4), 0.46291344560095266),   (('voxaudio', 9), 0.4394223090127941),   (('text_video', 0), 0.521582693486374),   (('text_video', 2), 0.46696506079092315),   (('text_video', 4), 0.43982623582741054),   (('text_video', 9), 0.4101222723935223),   (('text_voxaudio', 0), 0.5568745661063044),   (('text_voxaudio', 2), 0.5016227145585722),   (('text_voxaudio', 4), 0.4712626016671543),   (('text_voxaudio', 9), 0.44148841351976864),   (('video_voxaudio', 0), 0.5215271210631907),   (('video_voxaudio', 2), 0.4583848627092671),   (('video_voxaudio', 4), 0.4293962715574975),   (('video_voxaudio', 9), 0.4027055328432499),   (('text_video_voxaudio', 0), 0.5320974071839031),   (('text_video_voxaudio', 2), 0.47582571263691964),   (('text_video_voxaudio', 4), 0.4437348535696559),   (('text_video_voxaudio', 9), 0.41244335429158197)],  10: [(('text', 0), 0.5196639885630715),   (('text', 2), 0.49060624385226514),   (('text', 4), 0.47104308432468256),   (('text', 9), 0.4508823779858192),   (('video', 0), 0.4026817032627658),   (('video', 2), 0.39362203223466696),   (('video', 4), 0.38844217465767106),   (('video', 9), 0.38122951534849187),   (('voxaudio', 0), 0.4937721971471539),   (('voxaudio', 2), 0.45711897854077793),   (('voxaudio', 4), 0.4401122018902086),   (('voxaudio', 9), 0.4227091879748029),   (('text_video', 0), 0.4798377894917083),   (('text_video', 2), 0.4446692078565043),   (('text_video', 4), 0.4253131821062361),   (('text_video', 9), 0.39777124669337444),   (('text_voxaudio', 0), 0.5026913053939792),   (('text_voxaudio', 2), 0.46516402642767807),   (('text_voxaudio', 4), 0.447551725713666),   (('text_voxaudio', 9), 0.4259186769540853),   (('video_voxaudio', 0), 0.47571652416632865),   (('video_voxaudio', 2), 0.4382279453523452),   (('video_voxaudio', 4), 0.4184191745922492),   (('video_voxaudio', 9), 0.3978549848557913),   (('text_video_voxaudio', 0), 0.48889160567001955),   (('text_video_voxaudio', 2), 0.4454296232026712),   (('text_video_voxaudio', 4), 0.4274389238478841),   (('text_video_voxaudio', 9), 0.40303966970341537)],  30: [(('text', 0), 0.43185854118009004),   (('text', 2), 0.41633860144186025),   (('text', 4), 0.41245190618788874),   (('text', 9), 0.4045238079976493),   (('video', 0), 0.373871009402539),   (('video', 2), 0.37103996846696835),   (('video', 4), 0.3695393249719361),   (('video', 9), 0.36869205646953396),   (('voxaudio', 0), 0.38164540892712173),   (('voxaudio', 2), 0.3697941990283121),   (('voxaudio', 4), 0.3627268528250846),   (('voxaudio', 9), 0.35259530589638177),   (('text_video', 0), 0.402892453115018),   (('text_video', 2), 0.3867669701935051),   (('text_video', 4), 0.3831660597561313),   (('text_video', 9), 0.3710707001881017),   (('text_voxaudio', 0), 0.3898343826837004),   (('text_voxaudio', 2), 0.37428784626670725),   (('text_voxaudio', 4), 0.3688730343469443),   (('text_voxaudio', 9), 0.3605171332444099),   (('video_voxaudio', 0), 0.3836371446113673),   (('video_voxaudio', 2), 0.36957153483592187),   (('video_voxaudio', 4), 0.3641448903577872),   (('video_voxaudio', 9), 0.3531824713348667),   (('text_video_voxaudio', 0), 0.39146653242941304),   (('text_video_voxaudio', 2), 0.3750638990136947),   (('text_video_voxaudio', 4), 0.36559093379145147),   (('text_video_voxaudio', 9), 0.3551704354357227)]}}
for k, v in all_results.items():
    print(k)
    table = make_table(v)

TASK: PREDICTING MULTI-SPEAKER MULTI-CLASS LABEL (4 classes: speaker1, speaker2, speaker3, nospeaker)
*****************************************************************************************************

use_crossval: False | clf: <class 'sklearn.naive_bayes.GaussianNB'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.383  0.317  0.300  0.322  0.323  0.313    0.500  0.413  0.380      0.357  0.329  0.315         0.454  0.368  0.333          0.412  0.350  0.333               0.419  0.354  0.332
5   0.377  0.317  0.296  0.333  0.320  0.305    0.487  0.405  0.379      0.352  0.323  0.305         0.457  0.374  0.331          0.399  0.341  0.323               0.408  0.344  0.317
10  0.421  0.357  0.343  0.318  0.312  0.294    0.436  0.388  0.363      0.359  0.335  0.302         0.451  0.376  0.362          0.373  0.345  0.313               0.396  0.357  0.321
30  0.305  0.260  0.241  0.300  0.297  0.310    0.376  0.324  0.295      0.321  0.295  0.289         0.345  0.287  0.260          0.333  0.316  0.322               0.344  0.317  0.312

use_crossval: False | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.535  0.473  0.453  0.471  0.460  0.459    0.543  0.478  0.463      0.531  0.465  0.464         0.540  0.479  0.460          0.547  0.478  0.466               0.544  0.477  0.465
5   0.517  0.462  0.450  0.456  0.451  0.448    0.527  0.461  0.452      0.513  0.460  0.446         0.515  0.457  0.454          0.528  0.466  0.457               0.516  0.462  0.453
10  0.492  0.452  0.448  0.459  0.449  0.449    0.494  0.452  0.451      0.487  0.461  0.449         0.493  0.457  0.448          0.498  0.459  0.451               0.491  0.462  0.453
30  0.456  0.447  0.443  0.450  0.449  0.446    0.459  0.436  0.439      0.455  0.449  0.448         0.464  0.442  0.440          0.462  0.453  0.451               0.459  0.449  0.449

use_crossval: True | clf: <class 'sklearn.naive_bayes.GaussianNB'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.473  0.381  0.343  0.346  0.323  0.319    0.545  0.435  0.387      0.390  0.331  0.319         0.534  0.421  0.374          0.447  0.360  0.330               0.470  0.365  0.330
5   0.469  0.374  0.344  0.346  0.319  0.311    0.533  0.430  0.387      0.385  0.327  0.313         0.522  0.415  0.369          0.433  0.354  0.328               0.451  0.361  0.329
10  0.483  0.406  0.370  0.328  0.306  0.296    0.476  0.402  0.352      0.377  0.332  0.307         0.494  0.407  0.370          0.389  0.339  0.312               0.414  0.352  0.323
30  0.356  0.300  0.254  0.282  0.268  0.269    0.354  0.292  0.242      0.320  0.285  0.270         0.383  0.307  0.251          0.321  0.292  0.280               0.341  0.301  0.278

use_crossval: True | clf: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.573  0.486  0.461  0.415  0.391  0.389    0.564  0.461  0.442      0.542  0.449  0.413         0.572  0.472  0.446          0.549  0.443  0.414               0.554  0.452  0.424
5   0.565  0.488  0.462  0.411  0.395  0.390    0.550  0.463  0.439      0.522  0.440  0.410         0.557  0.471  0.441          0.522  0.429  0.403               0.532  0.444  0.412
10  0.520  0.471  0.451  0.403  0.388  0.381    0.494  0.440  0.423      0.480  0.425  0.398         0.503  0.448  0.426          0.476  0.418  0.398               0.489  0.427  0.403
30  0.432  0.412  0.405  0.374  0.370  0.369    0.382  0.363  0.353      0.403  0.383  0.371         0.390  0.369  0.361          0.384  0.364  0.353               0.391  0.366  0.355

# old tables

In [189]:
title = "Classifier: Random Forest (200 estimators), cross-validation scores"
print(title + "\n" + "*"*len(title) + "\n")

results = {4: [(('text', 0, False), 0.6605408943780327), (('text', 2, False), 0.6084040582701247), (('text', 4, False), 0.5828435622404251), (('text', 9, False), 0.5509915308073883), (('video', 0, False), 0.5012431843398073), (('video', 2, False), 0.4717902067114358), (('video', 4, False), 0.45287885411318995), (('video', 9, False), 0.4372361138758124), (('voxaudio', 0, False), 0.6793044384150295), (('voxaudio', 2, False), 0.61671443347856), (('voxaudio', 4, False), 0.5826993913939733), (('voxaudio', 9, False), 0.538041653803295), (('text_video', 0, False), 0.6165810424440099), (('text_video', 2, False), 0.5455404760636684), (('text_video', 4, False), 0.5151806870656417), (('text_video', 9, False), 0.4739563047083795), (('text_voxaudio', 0, False), 0.6804682662611299), (('text_voxaudio', 2, False), 0.6180245288283019), (('text_voxaudio', 4, False), 0.5854903789424819), (('text_voxaudio', 9, False), 0.5429776072010492), (('video_voxaudio', 0, False), 0.6511275950784465), (('video_voxaudio', 2, False), 0.5689986069349391), (('video_voxaudio', 4, False), 0.5278427109638723), (('video_voxaudio', 9, False), 0.47831363279134054), (('text_video_voxaudio', 0, False), 0.6536800094047199), (('text_video_voxaudio', 2, False), 0.5740751923739511), (('text_video_voxaudio', 4, False), 0.5282736696033175), (('text_video_voxaudio', 9, False), 0.4839107900547173)], 5: [(('text', 0, False), 0.6478396251170686), (('text', 2, False), 0.6023087975141143), (('text', 4, False), 0.5799017516118312), (('text', 9, False), 0.5427200454420971), (('video', 0, False), 0.4979460107730963), (('video', 2, False), 0.47264339217425677), (('video', 4, False), 0.4578052515055243), (('video', 9, False), 0.4450820338300797), (('voxaudio', 0, False), 0.6675306280313421), (('voxaudio', 2, False), 0.6086948173435245), (('voxaudio', 4, False), 0.577502310747503), (('voxaudio', 9, False), 0.5326631205763586), (('text_video', 0, False), 0.6074399647572016), (('text_video', 2, False), 0.5411883649714933), (('text_video', 4, False), 0.51211510278386), (('text_video', 9, False), 0.4729177567657987), (('text_voxaudio', 0, False), 0.6704357111178151), (('text_voxaudio', 2, False), 0.6089446978312518), (('text_voxaudio', 4, False), 0.579296206734337), (('text_voxaudio', 9, False), 0.5353504738028798), (('video_voxaudio', 0, False), 0.6368834170668414), (('video_voxaudio', 2, False), 0.5630523307721247), (('video_voxaudio', 4, False), 0.5246917137332403), (('video_voxaudio', 9, False), 0.47657005713623823), (('text_video_voxaudio', 0, False), 0.6450381128526053), (('text_video_voxaudio', 2, False), 0.5654239761353619), (('text_video_voxaudio', 4, False), 0.5277369740409477), (('text_video_voxaudio', 9, False), 0.480244024752199)], 10: [(('text', 0, False), 0.5989443867563818), (('text', 2, False), 0.5699782958282434), (('text', 4, False), 0.5462463926149669), (('text', 9, False), 0.5191935276046132), (('video', 0, False), 0.4815910988523028), (('video', 2, False), 0.46107002649561935), (('video', 4, False), 0.45065780927117655), (('video', 9, False), 0.44109869035962623), (('voxaudio', 0, False), 0.6085760553817565), (('voxaudio', 2, False), 0.5640460227823392), (('voxaudio', 4, False), 0.5404270666290619), (('voxaudio', 9, False), 0.4964110634295532), (('text_video', 0, False), 0.5458307138036802), (('text_video', 2, False), 0.5057589930264077), (('text_video', 4, False), 0.4855895495850004), (('text_video', 9, False), 0.4562064653501399), (('text_voxaudio', 0, False), 0.6167342365816717), (('text_voxaudio', 2, False), 0.5678361062129553), (('text_voxaudio', 4, False), 0.5406846049470259), (('text_voxaudio', 9, False), 0.49960992318344993), (('video_voxaudio', 0, False), 0.5693017367464496), (('video_voxaudio', 2, False), 0.5219043581018068), (('video_voxaudio', 4, False), 0.49877212224463363), (('video_voxaudio', 9, False), 0.4685049948837866), (('text_video_voxaudio', 0, False), 0.5748116230714442), (('text_video_voxaudio', 2, False), 0.5275147075607829), (('text_video_voxaudio', 4, False), 0.5001894680998876), (('text_video_voxaudio', 9, False), 0.46673238648579635)], 30: [(('text', 0, False), 0.49398732976387566), (('text', 2, False), 0.4795111222872611), (('text', 4, False), 0.4648754628538028), (('text', 9, False), 0.4430008952032529), (('video', 0, False), 0.44717895997175744), (('video', 2, False), 0.43844921902806877), (('video', 4, False), 0.43852402079332087), (('video', 9, False), 0.43104645913501366), (('voxaudio', 0, False), 0.4874767101832786), (('voxaudio', 2, False), 0.4632645648460961), (('voxaudio', 4, False), 0.44969549906752215), (('voxaudio', 9, False), 0.43128286060313414), (('text_video', 0, False), 0.47016981859287255), (('text_video', 2, False), 0.4527597222793805), (('text_video', 4, False), 0.4441313511924375), (('text_video', 9, False), 0.4329993708891548), (('text_voxaudio', 0, False), 0.49133670215093056), (('text_voxaudio', 2, False), 0.4663903220318525), (('text_voxaudio', 4, False), 0.45103879401065977), (('text_voxaudio', 9, False), 0.4341368526915623), (('video_voxaudio', 0, False), 0.47604504592957786), (('video_voxaudio', 2, False), 0.4563323172481045), (('video_voxaudio', 4, False), 0.4486936009943211), (('video_voxaudio', 9, False), 0.43407199151642173), (('text_video_voxaudio', 0, False), 0.48043523359883267), (('text_video_voxaudio', 2, False), 0.4623501214634351), (('text_video_voxaudio', 4, False), 0.44922670683183574), (('text_video_voxaudio', 9, False), 0.43357870097973505)]}
print("Task: predict binary label whether HOST is speaking")
print("Input: features of the host.")
print("Input: use_all_perspectives = False, predict_only_host = True")
print("Number of examples: (48730, 300)")
table = make_table(results)

results = {4: [(('text', 0, True), 0.67783629596835), (('text', 2, True), 0.6144518051913449), (('text', 4, True), 0.5827583073237359), (('text', 9, True), 0.5440221756288179), (('video', 0, True), 0.5220127252131594), (('video', 2, True), 0.49546872474992226), (('video', 4, True), 0.48366289692197373), (('video', 9, True), 0.47165316303756644), (('voxaudio', 0, True), 0.6741506742327911), (('voxaudio', 2, True), 0.6121618733087315), (('voxaudio', 4, True), 0.5759219637525747), (('voxaudio', 9, True), 0.5328358007243859), (('text_video', 0, True), 0.6351619897262973), (('text_video', 2, True), 0.5641198672031316), (('text_video', 4, True), 0.5291917094009466), (('text_video', 9, True), 0.48927205827191056), (('text_voxaudio', 0, True), 0.6822754169016761), (('text_voxaudio', 2, True), 0.6151803823226666), (('text_voxaudio', 4, True), 0.5814273620929548), (('text_voxaudio', 9, True), 0.5356455402168745), (('video_voxaudio', 0, True), 0.6593807341458764), (('video_voxaudio', 2, True), 0.5817645893891189), (('video_voxaudio', 4, True), 0.5425760708207454), (('video_voxaudio', 9, True), 0.4951279024804093), (('text_video_voxaudio', 0, True), 0.6631965190975907), (('text_video_voxaudio', 2, True), 0.5819724349916957), (('text_video_voxaudio', 4, True), 0.545942179901647), (('text_video_voxaudio', 9, True), 0.5004320451102852)], 5: [(('text', 0, True), 0.6647075352313838), (('text', 2, True), 0.6065772319362133), (('text', 4, True), 0.5746445043289101), (('text', 9, True), 0.5340233398307933), (('video', 0, True), 0.5333046971050114), (('video', 2, True), 0.5103192377438239), (('video', 4, True), 0.49431962629374143), (('video', 9, True), 0.48083410524034714), (('voxaudio', 0, True), 0.6667487619351633), (('voxaudio', 2, True), 0.609212845527164), (('voxaudio', 4, True), 0.5791713952457341), (('voxaudio', 9, True), 0.5377315173309797), (('text_video', 0, True), 0.6318496558342174), (('text_video', 2, True), 0.5708839280652617), (('text_video', 4, True), 0.5443924121241183), (('text_video', 9, True), 0.5009030579101623), (('text_voxaudio', 0, True), 0.670663719511157), (('text_voxaudio', 2, True), 0.6088203313393571), (('text_voxaudio', 4, True), 0.5791149507047979), (('text_voxaudio', 9, True), 0.5394293100789612), (('video_voxaudio', 0, True), 0.643482589552182), (('video_voxaudio', 2, True), 0.5775849202465864), (('video_voxaudio', 4, True), 0.5439708032993714), (('video_voxaudio', 9, True), 0.5098476345408038), (('text_video_voxaudio', 0, True), 0.6472856629355674), (('text_video_voxaudio', 2, True), 0.5807534734445031), (('text_video_voxaudio', 4, True), 0.5495940724101922), (('text_video_voxaudio', 9, True), 0.5081214891759813)], 10: [(('text', 0, True), 0.6217353163044365), (('text', 2, True), 0.5789983159082841), (('text', 4, True), 0.5554703682894845), (('text', 9, True), 0.5223695452890634), (('video', 0, True), 0.5141902665612416), (('video', 2, True), 0.4987611950854303), (('video', 4, True), 0.4894669330179278), (('video', 9, True), 0.4771610791071995), (('voxaudio', 0, True), 0.6144697913652171), (('voxaudio', 2, True), 0.5744870106874262), (('voxaudio', 4, True), 0.5560149598238069), (('voxaudio', 9, True), 0.5241807152352224), (('text_video', 0, True), 0.5883348807785624), (('text_video', 2, True), 0.5486907566199977), (('text_video', 4, True), 0.5202610816787774), (('text_video', 9, True), 0.4868722795857644), (('text_voxaudio', 0, True), 0.6217619793543048), (('text_voxaudio', 2, True), 0.5795154440151944), (('text_voxaudio', 4, True), 0.555906365316291), (('text_voxaudio', 9, True), 0.5258263354558663), (('video_voxaudio', 0, True), 0.5961696914090915), (('video_voxaudio', 2, True), 0.5561117797151811), (('video_voxaudio', 4, True), 0.5330966365153085), (('video_voxaudio', 9, True), 0.49798197601992966), (('text_video_voxaudio', 0, True), 0.6035705523606986), (('text_video_voxaudio', 2, True), 0.5610589905316687), (('text_video_voxaudio', 4, True), 0.5359582140321958), (('text_video_voxaudio', 9, True), 0.49732402517471613)], 30: [(('text', 0, True), 0.5150213361849183), (('text', 2, True), 0.4970886445575925), (('text', 4, True), 0.489571150843758), (('text', 9, True), 0.47190634038501117), (('video', 0, True), 0.48835148195746847), (('video', 2, True), 0.48080598489574744), (('video', 4, True), 0.47800128745160836), (('video', 9, True), 0.46202402728439607), (('voxaudio', 0, True), 0.52996072182754), (('voxaudio', 2, True), 0.507462262800459), (('voxaudio', 4, True), 0.4966070243842376), (('voxaudio', 9, True), 0.4812285390771912), (('text_video', 0, True), 0.5089346754762735), (('text_video', 2, True), 0.4860767850703545), (('text_video', 4, True), 0.4759159074290892), (('text_video', 9, True), 0.4655788237738011), (('text_voxaudio', 0, True), 0.5348575307654144), (('text_voxaudio', 2, True), 0.5120670285914167), (('text_voxaudio', 4, True), 0.5023635325271408), (('text_voxaudio', 9, True), 0.4814246487871854), (('video_voxaudio', 0, True), 0.5272493703802132), (('video_voxaudio', 2, True), 0.5032741629632312), (('video_voxaudio', 4, True), 0.4935477992899603), (('video_voxaudio', 9, True), 0.47632897433528054), (('text_video_voxaudio', 0, True), 0.5301541949046096), (('text_video_voxaudio', 2, True), 0.5082396236568443), (('text_video_voxaudio', 4, True), 0.4961618690029203), (('text_video_voxaudio', 9, True), 0.47658105157313707)]}
print("Task: predict binary label whether HOST is speaking")
print("Input: features of all speakers.")
print("Input: use_all_perspectives = True, predict_only_host = True")
print("Number of examples: (36146, 900)")
table = make_table(results)

results = {4: [(('text', 0, False), 0.728346158476511),  (('text', 2, False), 0.6972156936839846),  (('text', 4, False), 0.6829334497024286),  (('text', 9, False), 0.6722420248806096),  (('video', 0, False), 0.6750150134699409),  (('video', 2, False), 0.6735000328582362),  (('video', 4, False), 0.6737297857629229),  (('video', 9, False), 0.673842701240878),  (('voxaudio', 0, False), 0.7381222129931121),  (('voxaudio', 2, False), 0.7080257535133339),  (('voxaudio', 4, False), 0.6970794477578149),  (('voxaudio', 9, False), 0.683943221881538),  (('text_video', 0, False), 0.7280160341462837),  (('text_video', 2, False), 0.6953319960724194),  (('text_video', 4, False), 0.6831036660411007),  (('text_video', 9, False), 0.6745275136517653),  (('text_voxaudio', 0, False), 0.7443855906610712),  (('text_voxaudio', 2, False), 0.7105458335315538),  (('text_voxaudio', 4, False), 0.6986190975834783),  (('text_voxaudio', 9, False), 0.6852614317826715),  (('video_voxaudio', 0, False), 0.7308177503510691),  (('video_voxaudio', 2, False), 0.7050135289355064),  (('video_voxaudio', 4, False), 0.6946381111604873),  (('video_voxaudio', 9, False), 0.6836093848296653),  (('text_video_voxaudio', 0, False), 0.7387146304061594),  (('text_video_voxaudio', 2, False), 0.7078305797303436),  (('text_video_voxaudio', 4, False), 0.6957609445886002),  (('text_video_voxaudio', 9, False), 0.6850559614982745)], 5: [(('text', 0, False), 0.723718307807475),  (('text', 2, False), 0.6970646005298672),  (('text', 4, False), 0.6819915699022789),  (('text', 9, False), 0.6726844506596017),  (('video', 0, False), 0.6708659589311796),  (('video', 2, False), 0.6699541163128337),  (('video', 4, False), 0.6700715926131344),  (('video', 9, False), 0.6700431055779225),  (('voxaudio', 0, False), 0.7347425319440493),  (('voxaudio', 2, False), 0.7066891075305419),  (('voxaudio', 4, False), 0.6973401523300063),  (('voxaudio', 9, False), 0.6847879255517864),  (('text_video', 0, False), 0.7235600610263138),  (('text_video', 2, False), 0.6919791062837628),  (('text_video', 4, False), 0.6791198785694856),  (('text_video', 9, False), 0.6711281073464196),  (('text_voxaudio', 0, False), 0.7395073814090356),  (('text_voxaudio', 2, False), 0.7090159222404184),  (('text_voxaudio', 4, False), 0.6982856150131064),  (('text_voxaudio', 9, False), 0.6848590722364418),  (('video_voxaudio', 0, False), 0.7257315695919035),  (('video_voxaudio', 2, False), 0.7035866964616941),  (('video_voxaudio', 4, False), 0.6931605771903763),  (('video_voxaudio', 9, False), 0.6814796308189172),  (('text_video_voxaudio', 0, False), 0.7333535132259934),  (('text_video_voxaudio', 2, False), 0.7045738222774289),  (('text_video_voxaudio', 4, False), 0.6947687905171265),  (('text_video_voxaudio', 9, False), 0.6826179844947434)], 10: [(('text', 0, False), 0.70814367343517),  (('text', 2, False), 0.6897359653883818),  (('text', 4, False), 0.6801732960061381),  (('text', 9, False), 0.6732745921673782),  (('video', 0, False), 0.6738114165058172),  (('video', 2, False), 0.6733503243563709),  (('video', 4, False), 0.6730537671877175),  (('video', 9, False), 0.6729561533043256),  (('voxaudio', 0, False), 0.7179729975418044),  (('voxaudio', 2, False), 0.7008701470281533),  (('voxaudio', 4, False), 0.692179956064044),  (('voxaudio', 9, False), 0.6827486601886121),  (('text_video', 0, False), 0.7043571394376901),  (('text_video', 2, False), 0.6841951619103849),  (('text_video', 4, False), 0.6763938311512042),  (('text_video', 9, False), 0.6739380439922581),  (('text_voxaudio', 0, False), 0.7192322716030786),  (('text_voxaudio', 2, False), 0.7017731801367044),  (('text_voxaudio', 4, False), 0.6948783506535667),  (('text_voxaudio', 9, False), 0.6835536458310205),  (('video_voxaudio', 0, False), 0.7121488730759703),  (('video_voxaudio', 2, False), 0.6977577957732919),  (('video_voxaudio', 4, False), 0.6909669670699734),  (('video_voxaudio', 9, False), 0.6826071154162424),  (('text_video_voxaudio', 0, False), 0.7147023441648235),  (('text_video_voxaudio', 2, False), 0.6996076805042508),  (('text_video_voxaudio', 4, False), 0.6924612044326401),  (('text_video_voxaudio', 9, False), 0.6826071075913551)], 30: [(('text', 0, False), 0.6751794048228492),  (('text', 2, False), 0.6734709029889541),  (('text', 4, False), 0.6729871015281452),  (('text', 9, False), 0.6714631664435162),  (('video', 0, False), 0.6725637677825941),  (('video', 2, False), 0.6713335734125561),  (('video', 4, False), 0.6717987031440273),  (('video', 9, False), 0.6726230159975495),  (('voxaudio', 0, False), 0.6869137821780837),  (('voxaudio', 2, False), 0.6798649295286294),  (('voxaudio', 4, False), 0.6741665435957058),  (('voxaudio', 9, False), 0.6674037656717902),  (('text_video', 0, False), 0.6752241783983501),  (('text_video', 2, False), 0.6744317885913522),  (('text_video', 4, False), 0.6742385811866373),  (('text_video', 9, False), 0.6741271808900233),  (('text_voxaudio', 0, False), 0.689081496910109),  (('text_voxaudio', 2, False), 0.6821639256658756),  (('text_voxaudio', 4, False), 0.6774798019170707),  (('text_voxaudio', 9, False), 0.6684639172085067),  (('video_voxaudio', 0, False), 0.687227262199325),  (('video_voxaudio', 2, False), 0.6814634067265907),  (('video_voxaudio', 4, False), 0.6767234556202345),  (('video_voxaudio', 9, False), 0.6734385489814915),  (('text_video_voxaudio', 0, False), 0.6898876643900969),  (('text_video_voxaudio', 2, False), 0.6830888574522793),  (('text_video_voxaudio', 4, False), 0.6782540693520611),  (('text_video_voxaudio', 9, False), 0.6731576505156371)]}
print("Task: predict binary label whether any person is speaking")
print("Input: features of the person who is speaking.")
print("Input: use_all_perspectives = False, predict_only_host = False")
print("Number of examples: (136431, 300)")
table = make_table(results)

Classifier: Random Forest (200 estimators), cross-validation scores
*******************************************************************

Task: predict binary label whether HOST is speaking
Input: features of the host.
Input: use_all_perspectives = False, predict_only_host = True
Number of examples: (48730, 300)


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.661  0.583  0.551  0.501  0.453  0.437    0.679  0.583  0.538      0.617  0.515  0.474         0.680  0.585  0.543          0.651  0.528  0.478               0.654  0.528  0.484
5   0.648  0.580  0.543  0.498  0.458  0.445    0.668  0.578  0.533      0.607  0.512  0.473         0.670  0.579  0.535          0.637  0.525  0.477               0.645  0.528  0.480
10  0.599  0.546  0.519  0.482  0.451  0.441    0.609  0.540  0.496      0.546  0.486  0.456         0.617  0.541  0.500          0.569  0.499  0.469               0.575  0.500  0.467
30  0.494  0.465  0.443  0.447  0.439  0.431    0.487  0.450  0.431      0.470  0.444  0.433         0.491  0.451  0.434          0.476  0.449  0.434               0.480  0.449  0.434

Task: predict binary label whether HOST is speaking
Input: features of all speakers.
Input: use_all_perspectives = True, predict_only_host = True
Number of examples: (36146, 900)


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.678  0.583  0.544  0.522  0.484  0.472    0.674  0.576  0.533      0.635  0.529  0.489         0.682  0.581  0.536          0.659  0.543  0.495               0.663  0.546  0.500
5   0.665  0.575  0.534  0.533  0.494  0.481    0.667  0.579  0.538      0.632  0.544  0.501         0.671  0.579  0.539          0.643  0.544  0.510               0.647  0.550  0.508
10  0.622  0.555  0.522  0.514  0.489  0.477    0.614  0.556  0.524      0.588  0.520  0.487         0.622  0.556  0.526          0.596  0.533  0.498               0.604  0.536  0.497
30  0.515  0.490  0.472  0.488  0.478  0.462    0.530  0.497  0.481      0.509  0.476  0.466         0.535  0.502  0.481          0.527  0.494  0.476               0.530  0.496  0.477

Task: predict binary label whether any person is speaking
Input: features of the person who is speaking.
Input: use_all_perspectives = False, predict_only_host = False
Number of examples: (136431, 300)


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.728  0.683  0.672  0.675  0.674  0.674    0.738  0.697  0.684      0.728  0.683  0.675         0.744  0.699  0.685          0.731  0.695  0.684               0.739  0.696  0.685
5   0.724  0.682  0.673  0.671  0.670  0.670    0.735  0.697  0.685      0.724  0.679  0.671         0.740  0.698  0.685          0.726  0.693  0.681               0.733  0.695  0.683
10  0.708  0.680  0.673  0.674  0.673  0.673    0.718  0.692  0.683      0.704  0.676  0.674         0.719  0.695  0.684          0.712  0.691  0.683               0.715  0.692  0.683
30  0.675  0.673  0.671  0.673  0.672  0.673    0.687  0.674  0.667      0.675  0.674  0.674         0.689  0.677  0.668          0.687  0.677  0.673               0.690  0.678  0.673

In [ ]:
text label_shift: 0 (48978, 300) (48978,) num features: 300 0.6602958839134058
text label_shift: 4 (48730, 300) (48730,) num features: 300 0.5832538270160967
text label_shift: 9 (48420, 300) (48420,) num features: 300 0.5487197017629901
voxaudio label_shift: 0 (48978, 512) (48978,) num features: 512 0.6788757461817816
voxaudio label_shift: 4 (48730, 512) (48730,) num features: 512 0.5845872864862703
voxaudio label_shift: 9 (48420, 512) (48420,) num features: 512 0.5409743880968044
video label_shift: 0 (48978, 2048) (48978,) num features: 2048 0.49834387091354804

In [191]:
title = "Classifier: Naive Bayes, cross-validation scores"
print(title + "\n" + "*"*len(title) + "\n")

results = {4: [(('text', 0, False), 0.6622354344086439), (('text', 2, False), 0.6142788698789332), (('text', 4, False), 0.5923248634569396), (('text', 9, False), 0.5455600786798769), (('video', 0, False), 0.5240508663618886), (('video', 2, False), 0.5090461351094048), (('video', 4, False), 0.49847979645342244), (('video', 9, False), 0.4894046457326583), (('voxaudio', 0, False), 0.6771201317494551), (('voxaudio', 2, False), 0.6203176995184589), (('voxaudio', 4, False), 0.5945618302074402), (('voxaudio', 9, False), 0.5617720058872255), (('text_video', 0, False), 0.5821584791200417), (('text_video', 2, False), 0.5398112164970759), (('text_video', 4, False), 0.5205604797468495), (('text_video', 9, False), 0.49861578594143746), (('text_voxaudio', 0, False), 0.6760991881133678), (('text_voxaudio', 2, False), 0.6201332567538473), (('text_voxaudio', 4, False), 0.5955056682549278), (('text_voxaudio', 9, False), 0.563589632928027), (('video_voxaudio', 0, False), 0.6485759811538749), (('video_voxaudio', 2, False), 0.5886099146338433), (('video_voxaudio', 4, False), 0.5589765688518213), (('video_voxaudio', 9, False), 0.5223868888140804), (('text_video_voxaudio', 0, False), 0.6583969287919709), (('text_video_voxaudio', 2, False), 0.5982510696875998), (('text_video_voxaudio', 4, False), 0.5694424937237746), (('text_video_voxaudio', 9, False), 0.5298219674420256)], 5: [(('text', 0, False), 0.6535871545054965), (('text', 2, False), 0.6111295208590596), (('text', 4, False), 0.568993791104759), (('text', 9, False), 0.520637014787835), (('video', 0, False), 0.5260404266999892), (('video', 2, False), 0.5069270990076472), (('video', 4, False), 0.49768385549275845), (('video', 9, False), 0.47967816202561026), (('voxaudio', 0, False), 0.6666801714114434), (('voxaudio', 2, False), 0.6167046371470442), (('voxaudio', 4, False), 0.591227209178731), (('voxaudio', 9, False), 0.559156959378497), (('text_video', 0, False), 0.582291906102476), (('text_video', 2, False), 0.5430826146473824), (('text_video', 4, False), 0.5248602419559717), (('text_video', 9, False), 0.4975638683983908), (('text_voxaudio', 0, False), 0.6663066606981422), (('text_voxaudio', 2, False), 0.616184648814588), (('text_voxaudio', 4, False), 0.5916235792500611), (('text_voxaudio', 9, False), 0.5606056614539958), (('video_voxaudio', 0, False), 0.6390827744407732), (('video_voxaudio', 2, False), 0.5866435380202816), (('video_voxaudio', 4, False), 0.559461187570421), (('video_voxaudio', 9, False), 0.520403441325134), (('text_video_voxaudio', 0, False), 0.6467601325036558), (('text_video_voxaudio', 2, False), 0.593987064747505), (('text_video_voxaudio', 4, False), 0.5689512043585678), (('text_video_voxaudio', 9, False), 0.5316767639961519)], 10: [(('text', 0, False), 0.6091316039686244), (('text', 2, False), 0.5790363707669173), (('text', 4, False), 0.5628004786581935), (('text', 9, False), 0.5248781354501963), (('video', 0, False), 0.5075176118373624), (('video', 2, False), 0.49618828097658874), (('video', 4, False), 0.4881475948200252), (('video', 9, False), 0.4878736164943245), (('voxaudio', 0, False), 0.6273059071638538), (('voxaudio', 2, False), 0.5913273208110502), (('voxaudio', 4, False), 0.5738577217193968), (('voxaudio', 9, False), 0.542428994610573), (('text_video', 0, False), 0.5610368935749892), (('text_video', 2, False), 0.5322051985072983), (('text_video', 4, False), 0.517624912722511), (('text_video', 9, False), 0.5023113315133424), (('text_voxaudio', 0, False), 0.6252129214063562), (('text_voxaudio', 2, False), 0.589400130539448), (('text_voxaudio', 4, False), 0.5717749786491118), (('text_voxaudio', 9, False), 0.5417160885125941), (('video_voxaudio', 0, False), 0.5914268661454747), (('video_voxaudio', 2, False), 0.5576223723104609), (('video_voxaudio', 4, False), 0.5378931174952077), (('video_voxaudio', 9, False), 0.512859047129431), (('text_video_voxaudio', 0, False), 0.6063553092542625), (('text_video_voxaudio', 2, False), 0.5664234397896134), (('text_video_voxaudio', 4, False), 0.5475977842097561), (('text_video_voxaudio', 9, False), 0.5188465258383493)], 30: [(('text', 0, False), 0.5547095370468154), (('text', 2, False), 0.5382018334181828), (('text', 4, False), 0.5282599383772776), (('text', 9, False), 0.5018243774483537), (('video', 0, False), 0.48423122995357276), (('video', 2, False), 0.47900144954187496), (('video', 4, False), 0.4787761544616636), (('video', 9, False), 0.47435418538114515), (('voxaudio', 0, False), 0.5661410978314572), (('voxaudio', 2, False), 0.5460479365622382), (('voxaudio', 4, False), 0.5310732782662514), (('voxaudio', 9, False), 0.4972094232045915), (('text_video', 0, False), 0.5175507546662015), (('text_video', 2, False), 0.5013922657603975), (('text_video', 4, False), 0.4929324232599769), (('text_video', 9, False), 0.4763711432474424), (('text_voxaudio', 0, False), 0.5682833999527865), (('text_voxaudio', 2, False), 0.5462183211275046), (('text_voxaudio', 4, False), 0.5330565138302108), (('text_voxaudio', 9, False), 0.4975313991123021), (('video_voxaudio', 0, False), 0.5246982577947348), (('video_voxaudio', 2, False), 0.506665861868644), (('video_voxaudio', 4, False), 0.4981340864390632), (('video_voxaudio', 9, False), 0.4755560540877834), (('text_video_voxaudio', 0, False), 0.5338182770669715), (('text_video_voxaudio', 2, False), 0.5120244287004865), (('text_video_voxaudio', 4, False), 0.5026541013377209), (('text_video_voxaudio', 9, False), 0.47628555959788327)]}
print("Task: predict binary label whether HOST is speaking")
print("Input: Host's features")
print("Input: use_all_perspectives = False, predict_only_host = True")
print("Number of examples: 48978")
table = make_table(results)

results = {4: [(('text', 0, True), 0.6495421578279535), (('text', 2, True), 0.5878564811484824), (('text', 4, True), 0.5533003759881916), (('text', 9, True), 0.5189191297826732), (('video', 0, True), 0.5192359413820778), (('video', 2, True), 0.5051804947521831), (('video', 4, True), 0.4972346271973011), (('video', 9, True), 0.49625909688412584), (('voxaudio', 0, True), 0.6847674467968973), (('voxaudio', 2, True), 0.623508876445786), (('voxaudio', 4, True), 0.5934301380226243), (('voxaudio', 9, True), 0.5547619737142593), (('text_video', 0, True), 0.5598077909248287), (('text_video', 2, True), 0.5242564040462163), (('text_video', 4, True), 0.5126536453371967), (('text_video', 9, True), 0.4989114560751437), (('text_voxaudio', 0, True), 0.6856497951795943), (('text_voxaudio', 2, True), 0.623586628906604), (('text_voxaudio', 4, True), 0.5962215354146921), (('text_voxaudio', 9, True), 0.5555759631540683), (('video_voxaudio', 0, True), 0.6439362344447342), (('video_voxaudio', 2, True), 0.5791904546527735), (('video_voxaudio', 4, True), 0.5506154899432978), (('video_voxaudio', 9, True), 0.5195239039885351), (('text_video_voxaudio', 0, True), 0.6570448111602356), (('text_video_voxaudio', 2, True), 0.5884028140972526), (('text_video_voxaudio', 4, True), 0.5567203312580375), (('text_video_voxaudio', 9, True), 0.5212832267605743)], 5: [(('text', 0, True), 0.6293332682577489), (('text', 2, True), 0.5725381480733935), (('text', 4, True), 0.5509984628634959), (('text', 9, True), 0.5146047308539099), (('video', 0, True), 0.5131997544034145), (('video', 2, True), 0.5009821759299846), (('video', 4, True), 0.49538954697376675), (('video', 9, True), 0.48720497403446644), (('voxaudio', 0, True), 0.6781307293621237), (('voxaudio', 2, True), 0.6240455462862923), (('voxaudio', 4, True), 0.5953664608759018), (('voxaudio', 9, True), 0.556584695911876), (('text_video', 0, True), 0.5485174753508428), (('text_video', 2, True), 0.5159269842859872), (('text_video', 4, True), 0.5053696658950353), (('text_video', 9, True), 0.49167687778685903), (('text_voxaudio', 0, True), 0.6758935655362045), (('text_voxaudio', 2, True), 0.6219146645590208), (('text_voxaudio', 4, True), 0.5929198799364814), (('text_voxaudio', 9, True), 0.5542064601463389), (('video_voxaudio', 0, True), 0.6367992693470291), (('video_voxaudio', 2, True), 0.5803327672389804), (('video_voxaudio', 4, True), 0.5515348460754879), (('video_voxaudio', 9, True), 0.5148602571679588), (('text_video_voxaudio', 0, True), 0.6443497027146645), (('text_video_voxaudio', 2, True), 0.5869780437216483), (('text_video_voxaudio', 4, True), 0.5575232676128528), (('text_video_voxaudio', 9, True), 0.5180867523869178)], 10: [(('text', 0, True), 0.6178053693988834), (('text', 2, True), 0.5818258581896032), (('text', 4, True), 0.5585227779257285), (('text', 9, True), 0.5312027592742684), (('video', 0, True), 0.510017427611629), (('video', 2, True), 0.5000424730897097), (('video', 4, True), 0.4949456954532815), (('video', 9, True), 0.49452904971774253), (('voxaudio', 0, True), 0.6384634536644452), (('voxaudio', 2, True), 0.598868891473415), (('voxaudio', 4, True), 0.5768354206541544), (('voxaudio', 9, True), 0.5456873438450707), (('text_video', 0, True), 0.5604133805155787), (('text_video', 2, True), 0.5263519468901487), (('text_video', 4, True), 0.5140485921163134), (('text_video', 9, True), 0.5038823293744128), (('text_voxaudio', 0, True), 0.6388698291130707), (('text_voxaudio', 2, True), 0.5970204153245715), (('text_voxaudio', 4, True), 0.5751184652218585), (('text_voxaudio', 9, True), 0.5435472004175821), (('video_voxaudio', 0, True), 0.605389037026129), (('video_voxaudio', 2, True), 0.5620124601398042), (('video_voxaudio', 4, True), 0.5398553919339418), (('video_voxaudio', 9, True), 0.5178999911351139), (('text_video_voxaudio', 0, True), 0.6141996704576366), (('text_video_voxaudio', 2, True), 0.5720691790307317), (('text_video_voxaudio', 4, True), 0.5493119402730458), (('text_video_voxaudio', 9, True), 0.5231664349198406)], 30: [(('text', 0, True), 0.5786259010933945), (('text', 2, True), 0.5620501700560834), (('text', 4, True), 0.5459432048681542), (('text', 9, True), 0.5281201234374271), (('video', 0, True), 0.4919217145786135), (('video', 2, True), 0.48577166850293196), (('video', 4, True), 0.484370943842948), (('video', 9, True), 0.4862690727222514), (('voxaudio', 0, True), 0.5708241681394307), (('voxaudio', 2, True), 0.5476818879818169), (('voxaudio', 4, True), 0.5343189422756695), (('voxaudio', 9, True), 0.5137034411128298), (('text_video', 0, True), 0.5322858197359758), (('text_video', 2, True), 0.5145081748940191), (('text_video', 4, True), 0.5062856580242049), (('text_video', 9, True), 0.4901602111269184), (('text_voxaudio', 0, True), 0.5805626102181413), (('text_voxaudio', 2, True), 0.5579727078321679), (('text_voxaudio', 4, True), 0.5434962617806927), (('text_voxaudio', 9, True), 0.5211499126398458), (('video_voxaudio', 0, True), 0.5331987019395331), (('video_voxaudio', 2, True), 0.5128989789300309), (('video_voxaudio', 4, True), 0.5040330469631348), (('video_voxaudio', 9, True), 0.4883124401513572), (('text_video_voxaudio', 0, True), 0.5504345663224062), (('text_video_voxaudio', 2, True), 0.5306792466098086), (('text_video_voxaudio', 4, True), 0.51727122597475), (('text_video_voxaudio', 9, True), 0.49497493921464975)]}
print("Task: predict binary label whether HOST is speaking")
print("Input: All speaker's features")
print("Input: use_all_perspectives = True, predict_only_host = True")
print("Number of examples: (36695, 7680)")
table = make_table(results)


results = {4: [(('text', 0), 0.6495421578279535), (('text', 2), 0.5878564811484824), (('text', 4), 0.5533003759881916), (('text', 9), 0.5189191297826732), (('video', 0), 0.5192359413820778), (('video', 2), 0.5051804947521831), (('video', 4), 0.4972346271973011), (('video', 9), 0.49625909688412584), (('voxaudio', 0), 0.6847674467968973), (('voxaudio', 2), 0.623508876445786), (('voxaudio', 4), 0.5934301380226243), (('voxaudio', 9), 0.5547619737142593), (('text_video', 0), 0.5598077909248287), (('text_video', 2), 0.5242564040462163), (('text_video', 4), 0.5126536453371967), (('text_video', 9), 0.4989114560751437), (('text_voxaudio', 0), 0.6856497951795943), (('text_voxaudio', 2), 0.623586628906604), (('text_voxaudio', 4), 0.5962215354146921), (('text_voxaudio', 9), 0.5555759631540683), (('video_voxaudio', 0), 0.6439362344447342), (('video_voxaudio', 2), 0.5791904546527735), (('video_voxaudio', 4), 0.5506154899432978), (('video_voxaudio', 9), 0.5195239039885351), (('text_video_voxaudio', 0), 0.6570448111602356), (('text_video_voxaudio', 2), 0.5884028140972526), (('text_video_voxaudio', 4), 0.5567203312580375), (('text_video_voxaudio', 9), 0.5212832267605743)]}
print("Task: predict binary label whether HOST is speaking")
print("Input: All speaker's features")
print("Input: use_all_perspectives = True, predict_only_host = True")
print("Number of examples: (38524, 900)")
table = make_table(results)

results = {4: [(('text', 0, False), 0.671555662971522), (('text', 2, False), 0.6123672704559651), (('text', 4, False), 0.599591738660769), (('text', 9, False), 0.5748549256198585), (('video', 0, False), 0.5576084483926695), (('video', 2, False), 0.5461145607524709), (('video', 4, False), 0.542276239221862), (('video', 9, False), 0.5310812002880885), (('voxaudio', 0, False), 0.7031173301892262), (('voxaudio', 2, False), 0.6603813792850057), (('voxaudio', 4, False), 0.6436909856762745), (('voxaudio', 9, False), 0.625969299650994), (('text_video', 0, False), 0.6414308004973109), (('text_video', 2, False), 0.6015064114340302), (('text_video', 4, False), 0.585388173515563), (('text_video', 9, False), 0.5584102798197941), (('text_voxaudio', 0, False), 0.6990200529449961), (('text_voxaudio', 2, False), 0.643935775434471), (('text_voxaudio', 4, False), 0.6280211827849997), (('text_voxaudio', 9, False), 0.6061136811592092), (('video_voxaudio', 0, False), 0.6645193458483327), (('video_voxaudio', 2, False), 0.6246463607304259), (('video_voxaudio', 4, False), 0.6089922861517474), (('video_voxaudio', 9, False), 0.5845457285507417), (('text_video_voxaudio', 0, False), 0.6900486348399804), (('text_video_voxaudio', 2, False), 0.6407980306426131), (('text_video_voxaudio', 4, False), 0.6231443382952732), (('text_video_voxaudio', 9, False), 0.5955040202445318)], 5: [(('text', 0, False), 0.6768598986991343), (('text', 2, False), 0.6358361885295272), (('text', 4, False), 0.6157670525686695), (('text', 9, False), 0.5903164393472323), (('video', 0, False), 0.5497097132219093), (('video', 2, False), 0.538431091174597), (('video', 4, False), 0.5336887848726968), (('video', 9, False), 0.5216805206621352), (('voxaudio', 0, False), 0.6997071686488399), (('voxaudio', 2, False), 0.6626996172497414), (('voxaudio', 4, False), 0.6455940272865319), (('voxaudio', 9, False), 0.6282407319458313), (('text_video', 0, False), 0.6193035772083382), (('text_video', 2, False), 0.5861956819922309), (('text_video', 4, False), 0.5717011667920494), (('text_video', 9, False), 0.54240555986723), (('text_voxaudio', 0, False), 0.6949345463957047), (('text_voxaudio', 2, False), 0.6543679143796668), (('text_voxaudio', 4, False), 0.6347353029760334), (('text_voxaudio', 9, False), 0.6090486509604645), (('video_voxaudio', 0, False), 0.6533998675490608), (('video_voxaudio', 2, False), 0.618162936158367), (('video_voxaudio', 4, False), 0.6015280675745843), (('video_voxaudio', 9, False), 0.5752786693449676), (('text_video_voxaudio', 0, False), 0.678201085546779), (('text_video_voxaudio', 2, False), 0.6368692326877674), (('text_video_voxaudio', 4, False), 0.6169705213430655), (('text_video_voxaudio', 9, False), 0.5841696274003094)], 10: [(('text', 0, False), 0.5811783428377613), (('text', 2, False), 0.523266273394551), (('text', 4, False), 0.49142406670047645), (('text', 9, False), 0.45736721161926097), (('video', 0, False), 0.5408929586124918), (('video', 2, False), 0.530378690668295), (('video', 4, False), 0.5250830214934218), (('video', 9, False), 0.5182381968600402), (('voxaudio', 0, False), 0.6798408298789592), (('voxaudio', 2, False), 0.6582110517150042), (('voxaudio', 4, False), 0.6497171748639153), (('voxaudio', 9, False), 0.6374654655431622), (('text_video', 0, False), 0.5767013863623583), (('text_video', 2, False), 0.552737957725175), (('text_video', 4, False), 0.5395351125560695), (('text_video', 9, False), 0.5185697731498239), (('text_voxaudio', 0, False), 0.642117552145183), (('text_voxaudio', 2, False), 0.6137364720771329), (('text_voxaudio', 4, False), 0.5982409382357182), (('text_voxaudio', 9, False), 0.5687714464323997), (('video_voxaudio', 0, False), 0.6347275506172159), (('video_voxaudio', 2, False), 0.6115731744271695), (('video_voxaudio', 4, False), 0.5986092994389285), (('video_voxaudio', 9, False), 0.5819537627177971), (('text_video_voxaudio', 0, False), 0.6386857835879949), (('text_video_voxaudio', 2, False), 0.6098677277725841), (('text_video_voxaudio', 4, False), 0.5930889724087021), (('text_video_voxaudio', 9, False), 0.5651757764826092)], 30: [(('text', 0, False), 0.5229911514822895), (('text', 2, False), 0.5063262187497839), (('text', 4, False), 0.49713388955774623), (('text', 9, False), 0.4801891871949933), (('video', 0, False), 0.5305670958281145), (('video', 2, False), 0.5242317016369821), (('video', 4, False), 0.5226766517726793), (('video', 9, False), 0.5208450345767573), (('voxaudio', 0, False), 0.6571161993204042), (('voxaudio', 2, False), 0.6512731066345603), (('voxaudio', 4, False), 0.6473285816123214), (('voxaudio', 9, False), 0.6398188130665725), (('text_video', 0, False), 0.552516997540663), (('text_video', 2, False), 0.5344939702512123), (('text_video', 4, False), 0.527451718677359), (('text_video', 9, False), 0.5162805620124757), (('text_voxaudio', 0, False), 0.565832251054358), (('text_voxaudio', 2, False), 0.5562060298449196), (('text_voxaudio', 4, False), 0.5529697529402194), (('text_voxaudio', 9, False), 0.5563478781742364), (('video_voxaudio', 0, False), 0.6093084253641647), (('video_voxaudio', 2, False), 0.6045274376866789), (('video_voxaudio', 4, False), 0.6015183879038801), (('video_voxaudio', 9, False), 0.5949912370783761), (('text_video_voxaudio', 0, False), 0.5824986991977159), (('text_video_voxaudio', 2, False), 0.5704493027120277), (('text_video_voxaudio', 4, False), 0.5656089226061074), (('text_video_voxaudio', 9, False), 0.5629714896852759)]}
print("Task: predict binary label whether anyone is speaking")
print("Input: Singles speaker's features")
print("Input: use_all_perspectives = False, predict_only_host = False")
print("Number of examples: (136431, 300)")
table = make_table(results)


Classifier: Naive Bayes, cross-validation scores
************************************************

Task: predict binary label whether HOST is speaking
Input: Host's features
Input: use_all_perspectives = False, predict_only_host = True
Number of examples: 48978


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.662  0.592  0.546  0.524  0.498  0.489    0.677  0.595  0.562      0.582  0.521  0.499         0.676  0.596  0.564          0.649  0.559  0.522               0.658  0.569  0.530
5   0.654  0.569  0.521  0.526  0.498  0.480    0.667  0.591  0.559      0.582  0.525  0.498         0.666  0.592  0.561          0.639  0.559  0.520               0.647  0.569  0.532
10  0.609  0.563  0.525  0.508  0.488  0.488    0.627  0.574  0.542      0.561  0.518  0.502         0.625  0.572  0.542          0.591  0.538  0.513               0.606  0.548  0.519
30  0.555  0.528  0.502  0.484  0.479  0.474    0.566  0.531  0.497      0.518  0.493  0.476         0.568  0.533  0.498          0.525  0.498  0.476               0.534  0.503  0.476

Task: predict binary label whether HOST is speaking
Input: All speaker's features
Input: use_all_perspectives = True, predict_only_host = True
Number of examples: (36695, 7680)


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.650  0.553  0.519  0.519  0.497  0.496    0.685  0.593  0.555      0.560  0.513  0.499         0.686  0.596  0.556          0.644  0.551  0.520               0.657  0.557  0.521
5   0.629  0.551  0.515  0.513  0.495  0.487    0.678  0.595  0.557      0.549  0.505  0.492         0.676  0.593  0.554          0.637  0.552  0.515               0.644  0.558  0.518
10  0.618  0.559  0.531  0.510  0.495  0.495    0.638  0.577  0.546      0.560  0.514  0.504         0.639  0.575  0.544          0.605  0.540  0.518               0.614  0.549  0.523
30  0.579  0.546  0.528  0.492  0.484  0.486    0.571  0.534  0.514      0.532  0.506  0.490         0.581  0.543  0.521          0.533  0.504  0.488               0.550  0.517  0.495

Task: predict binary label whether HOST is speaking
Input: All speaker's features
Input: use_all_perspectives = True, predict_only_host = True
Number of examples: (38524, 900)


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio              text_video_voxaudio              
     1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5     10                  1      5      10
4  0.65  0.553  0.519  0.519  0.497  0.496    0.685  0.593  0.555       0.56  0.513  0.499         0.686  0.596  0.556          0.644  0.551  0.52               0.657  0.557  0.521

Task: predict binary label whether anyone is speaking
Input: Singles speaker's features
Input: use_all_perspectives = False, predict_only_host = False
Number of examples: (136431, 300)


text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.672  0.600  0.575  0.558  0.542  0.531    0.703  0.644  0.626      0.641  0.585  0.558         0.699  0.628  0.606          0.665  0.609  0.585               0.690  0.623  0.596
5   0.677  0.616  0.590  0.550  0.534  0.522    0.700  0.646  0.628      0.619  0.572  0.542         0.695  0.635  0.609          0.653  0.602  0.575               0.678  0.617  0.584
10  0.581  0.491  0.457  0.541  0.525  0.518    0.680  0.650  0.637      0.577  0.540  0.519         0.642  0.598  0.569          0.635  0.599  0.582               0.639  0.593  0.565
30  0.523  0.497  0.480  0.531  0.523  0.521    0.657  0.647  0.640      0.553  0.527  0.516         0.566  0.553  0.556          0.609  0.602  0.595               0.582  0.566  0.563

In [165]:
print("Classifier: Naive Bayes")
print("Task: predict binary label whether host is speaking")
print("Input: Only hosts features")

results = {4: [(('audio', 0), 0.6399680359313951), (('audio', 2), 0.6455329728045065), (('audio', 4), 0.6427421942307121), (('audio', 9), 0.5943138381081071), (('text', 0), 0.5816209781947976), (('text', 2), 0.5479026225874966), (('text', 4), 0.5143819256706574), (('text', 9), 0.5008881591525446), (('video', 0), 0.5371513742072292), (('video', 2), 0.5068477871917998), (('video', 4), 0.4917096937108226), (('video', 9), 0.49632983359357397), (('voxaudio', 0), 0.6741655883918019), (('voxaudio', 2), 0.6016998626373626), (('voxaudio', 4), 0.5638764445500728), (('voxaudio', 9), 0.5391206653050395), (('audio_text', 0), 0.6508527417888577), (('audio_text', 2), 0.644689462323391), (('audio_text', 4), 0.6412826416327502), (('audio_text', 9), 0.5938081500187806), (('audio_video', 0), 0.6186661880645905), (('audio_video', 2), 0.6191163484624619), (('audio_video', 4), 0.6087227058019666), (('audio_video', 9), 0.5624042785658843), (('audio_voxaudio', 0), 0.6825608847020662), (('audio_voxaudio', 2), 0.6511749411302983), (('audio_voxaudio', 4), 0.6434762781854372), (('audio_voxaudio', 9), 0.5967193829029308), (('text_video', 0), 0.5789118280340053), (('text_video', 2), 0.5301407101302059), (('text_video', 4), 0.5113337393422654), (('text_video', 9), 0.49785130405633743), (('text_voxaudio', 0), 0.6760559288205769), (('text_voxaudio', 2), 0.6027815934065934), (('text_voxaudio', 4), 0.5702143430082292), (('text_voxaudio', 9), 0.5359548247726404), (('video_voxaudio', 0), 0.6365316045317748), (('video_voxaudio', 2), 0.5646008403361344), (('video_voxaudio', 4), 0.5285190576631711), (('video_voxaudio', 9), 0.5141763355937894), (('audio_text_video', 0), 0.6372432738360132), (('audio_text_video', 2), 0.6193542801736079), (('audio_text_video', 4), 0.6093348385371795), (('audio_text_video', 9), 0.5664571561281935), (('audio_text_voxaudio', 0), 0.6890680818165911), (('audio_text_voxaudio', 2), 0.6478118651306677), (('audio_text_voxaudio', 4), 0.6370161314280621), (('audio_text_voxaudio', 9), 0.5944399402012582), (('audio_video_voxaudio', 0), 0.6616179179524778), (('audio_video_voxaudio', 2), 0.6280020662110999), (('audio_video_voxaudio', 4), 0.6082398027390749), (('audio_video_voxaudio', 9), 0.5670856613258143), (('text_video_voxaudio', 0), 0.6482463237174472), (('text_video_voxaudio', 2), 0.5734833837381107), (('text_video_voxaudio', 4), 0.5410740916787974), (('text_video_voxaudio', 9), 0.5145567236566537), (('audio_text_video_voxaudio', 0), 0.6720309308970529), (('audio_text_video_voxaudio', 2), 0.6265625), (('audio_text_video_voxaudio', 4), 0.6059255518255549), (('audio_text_video_voxaudio', 9), 0.5696201922667131)], 5: [(('audio', 0), 0.6488674519963087), (('audio', 2), 0.6565800646897788), (('audio', 4), 0.6582075730642186), (('audio', 9), 0.5947453015714051), (('text', 0), 0.5776699447415528), (('text', 2), 0.5258341079276071), (('text', 4), 0.5178480602482793), (('text', 9), 0.5048561313565634), (('video', 0), 0.5293519501673319), (('video', 2), 0.503347565302378), (('video', 4), 0.49344667804413467), (('video', 9), 0.4835461697944449), (('voxaudio', 0), 0.6592410580269288), (('voxaudio', 2), 0.5930618690509227), (('voxaudio', 4), 0.5620451495515049), (('voxaudio', 9), 0.5390241178788735), (('audio_text', 0), 0.652757502140292), (('audio_text', 2), 0.647370465545762), (('audio_text', 4), 0.649467203069239), (('audio_text', 9), 0.5923490195397679), (('audio_video', 0), 0.6210523426468463), (('audio_video', 2), 0.6222406835333661), (('audio_video', 4), 0.6253021278198789), (('audio_video', 9), 0.5576728088093076), (('audio_voxaudio', 0), 0.6750362042894786), (('audio_voxaudio', 2), 0.653710108506148), (('audio_voxaudio', 4), 0.6500748838536111), (('audio_voxaudio', 9), 0.5963836130507), (('text_video', 0), 0.5685990677221735), (('text_video', 2), 0.5295377350028727), (('text_video', 4), 0.5172417822547326), (('text_video', 9), 0.4927487785154364), (('text_voxaudio', 0), 0.6553504519629535), (('text_voxaudio', 2), 0.5909095448890859), (('text_voxaudio', 4), 0.5653246504914606), (('text_voxaudio', 9), 0.5330978924347578), (('video_voxaudio', 0), 0.6163366123347529), (('video_voxaudio', 2), 0.5624338415295789), (('video_voxaudio', 4), 0.5302317882457274), (('video_voxaudio', 9), 0.5003126785559966), (('audio_text_video', 0), 0.6293026122680424), (('audio_text_video', 2), 0.6228399850490739), (('audio_text_video', 4), 0.6211761352826976), (('audio_text_video', 9), 0.5565405787135893), (('audio_text_voxaudio', 0), 0.6790423722217899), (('audio_text_voxaudio', 2), 0.6436626876128809), (('audio_text_voxaudio', 4), 0.6414551357840995), (('audio_text_voxaudio', 9), 0.5917205668808985), (('audio_video_voxaudio', 0), 0.6533492789717703), (('audio_video_voxaudio', 2), 0.628822636583099), (('audio_video_voxaudio', 4), 0.6180199110776017), (('audio_video_voxaudio', 9), 0.560197258410675), (('text_video_voxaudio', 0), 0.6321308552273157), (('text_video_voxaudio', 2), 0.5714045627757007), (('text_video_voxaudio', 4), 0.5401878137437301), (('text_video_voxaudio', 9), 0.5046008295004825), (('audio_text_video_voxaudio', 0), 0.661010551361448), (('audio_text_video_voxaudio', 2), 0.6234421488209339), (('audio_text_video_voxaudio', 4), 0.6119476738537188), (('audio_text_video_voxaudio', 9), 0.5619614356145655)], 10: [(('audio', 0), 0.6605446646697916), (('audio', 2), 0.6446073801459093), (('audio', 4), 0.6120910638511282), (('audio', 9), 0.5477792054823493), (('text', 0), 0.5995687059563531), (('text', 2), 0.5346918425395171), (('text', 4), 0.5189940180489423), (('text', 9), 0.5034905313310567), (('video', 0), 0.5236645673555157), (('video', 2), 0.4970858845752996), (('video', 4), 0.48691532488374617), (('video', 9), 0.49782331582312017), (('voxaudio', 0), 0.6231249942933084), (('voxaudio', 2), 0.5635535543685948), (('voxaudio', 4), 0.5518219735701966), (('voxaudio', 9), 0.5250916038980133), (('audio_text', 0), 0.6558173935397861), (('audio_text', 2), 0.6364805345666146), (('audio_text', 4), 0.605651285966237), (('audio_text', 9), 0.5443977826616228), (('audio_video', 0), 0.6470772976272942), (('audio_video', 2), 0.6265861125586243), (('audio_video', 4), 0.5856001315081929), (('audio_video', 9), 0.5231301691112836), (('audio_voxaudio', 0), 0.668985274222914), (('audio_voxaudio', 2), 0.6412737754038561), (('audio_voxaudio', 4), 0.6123022570929445), (('audio_voxaudio', 9), 0.5531253932891378), (('text_video', 0), 0.5692304945117186), (('text_video', 2), 0.5191728547854785), (('text_video', 4), 0.5049622053255245), (('text_video', 9), 0.508403847303591), (('text_voxaudio', 0), 0.6200377269144607), (('text_voxaudio', 2), 0.5620941896821261), (('text_voxaudio', 4), 0.5436935413317434), (('text_voxaudio', 9), 0.5248732037935527), (('video_voxaudio', 0), 0.597209380920616), (('video_voxaudio', 2), 0.5374071239360779), (('video_voxaudio', 4), 0.5176287846915548), (('video_voxaudio', 9), 0.5118960449987495), (('audio_text_video', 0), 0.6510915505731625), (('audio_text_video', 2), 0.619085026923745), (('audio_text_video', 4), 0.5849661617160704), (('audio_text_video', 9), 0.5260741371020494), (('audio_text_voxaudio', 0), 0.6605550386660923), (('audio_text_voxaudio', 2), 0.6310639221816919), (('audio_text_voxaudio', 4), 0.602274200154355), (('audio_text_voxaudio', 9), 0.5439629450189638), (('audio_video_voxaudio', 0), 0.6588107763201696), (('audio_video_voxaudio', 2), 0.6225227983324648), (('audio_video_voxaudio', 4), 0.5898232719349815), (('audio_video_voxaudio', 9), 0.5264049776345079), (('text_video_voxaudio', 0), 0.6115050340220635), (('text_video_voxaudio', 2), 0.5396967278964737), (('text_video_voxaudio', 4), 0.524594149557135), (('text_video_voxaudio', 9), 0.5169146039396733), (('audio_text_video_voxaudio', 0), 0.6534628341121957), (('audio_text_video_voxaudio', 2), 0.6186684688205663), (('audio_text_video_voxaudio', 4), 0.5833836612215216), (('audio_text_video_voxaudio', 9), 0.527712344166835)]}
table = make_table(results)

results = {4: [(('text', 0, False), 0.6633679914828454),  (('text', 2, False), 0.5995262034938736),  (('text', 4, False), 0.5889041644131042),  (('text', 9, False), 0.5566834214229728),  (('video', 0, False), 0.5511958087683017),  (('video', 2, False), 0.5376027570463098),  (('video', 4, False), 0.5333252325848552),  (('video', 9, False), 0.5229678820173717),  (('voxaudio', 0, False), 0.7011685588862359),  (('voxaudio', 2, False), 0.6560882585723411),  (('voxaudio', 4, False), 0.6378665864599782),  (('voxaudio', 9, False), 0.6213019344842696),  (('text_video', 0, False), 0.6391955431043813),  (('text_video', 2, False), 0.5970491683950316),  (('text_video', 4, False), 0.5813344680805572),  (('text_video', 9, False), 0.5546303389760616),  (('text_voxaudio', 0, False), 0.6946087097662099),  (('text_voxaudio', 2, False), 0.6341371712473172),  (('text_voxaudio', 4, False), 0.619867226419107),  (('text_voxaudio', 9, False), 0.5948776320175677),  (('video_voxaudio', 0, False), 0.6640714135593517),  (('video_voxaudio', 2, False), 0.6212318640040894),  (('video_voxaudio', 4, False), 0.6036294899222207),  (('video_voxaudio', 9, False), 0.5752167692650987),  (('text_video_voxaudio', 0, False), 0.6861535133001008),  (('text_video_voxaudio', 2, False), 0.6329070967616287),  (('text_video_voxaudio', 4, False), 0.6156315326949819),  (('text_video_voxaudio', 9, False), 0.587054665705547)], 5: [(('text', 0, False), 0.67255344555747),  (('text', 2, False), 0.6283794533247645),  (('text', 4, False), 0.6089950697335917),  (('text', 9, False), 0.5860374998930717),  (('video', 0, False), 0.5463045746398312),  (('video', 2, False), 0.5343226365943116),  (('video', 4, False), 0.5283568892644674),  (('video', 9, False), 0.5160147529460923),  (('voxaudio', 0, False), 0.6978282860097912),  (('voxaudio', 2, False), 0.6591127184490466),  (('voxaudio', 4, False), 0.6414682838466353),  (('voxaudio', 9, False), 0.6255321794489257),  (('text_video', 0, False), 0.6225764935546715),  (('text_video', 2, False), 0.589230925166244),  (('text_video', 4, False), 0.5735008660187659),  (('text_video', 9, False), 0.5431918923378849),  (('text_voxaudio', 0, False), 0.6922808844794199),  (('text_voxaudio', 2, False), 0.6513037080976398),  (('text_voxaudio', 4, False), 0.6318985298592384),  (('text_voxaudio', 9, False), 0.6083148759422526),  (('video_voxaudio', 0, False), 0.6577675253946969),  (('video_voxaudio', 2, False), 0.6183421389675664),  (('video_voxaudio', 4, False), 0.5998239700925654),  (('video_voxaudio', 9, False), 0.5696134170437791),  (('text_video_voxaudio', 0, False), 0.6808176084435862),  (('text_video_voxaudio', 2, False), 0.638727482496269),  (('text_video_voxaudio', 4, False), 0.6175049342641346),  (('text_video_voxaudio', 9, False), 0.5856473680117718)], 10: [(('text', 0, False), 0.6156474249056758),  (('text', 2, False), 0.5861242585986569),  (('text', 4, False), 0.5183916712232506),  (('text', 9, False), 0.4280468982255484),  (('video', 0, False), 0.5310601051877221),  (('video', 2, False), 0.5209866797240685),  (('video', 4, False), 0.5139805928636625),  (('video', 9, False), 0.505609012031918),  (('voxaudio', 0, False), 0.6751399353948832),  (('voxaudio', 2, False), 0.6541230915372205),  (('voxaudio', 4, False), 0.6453049332376687),  (('voxaudio', 9, False), 0.6347503408178442),  (('text_video', 0, False), 0.567980407439667),  (('text_video', 2, False), 0.5450692035759217),  (('text_video', 4, False), 0.5312693828638493),  (('text_video', 9, False), 0.5057325407844359),  (('text_voxaudio', 0, False), 0.637807313858367),  (('text_voxaudio', 2, False), 0.6099097052789004),  (('text_voxaudio', 4, False), 0.5947472373060584),  (('text_voxaudio', 9, False), 0.5675122409108997),  (('video_voxaudio', 0, False), 0.6310317532683292),  (('video_voxaudio', 2, False), 0.6065887786206208),  (('video_voxaudio', 4, False), 0.5928930764588498),  (('video_voxaudio', 9, False), 0.5750229015858679),  (('text_video_voxaudio', 0, False), 0.633435864722806),  (('text_video_voxaudio', 2, False), 0.6055798136504638),  (('text_video_voxaudio', 4, False), 0.5893858767859158),  (('text_video_voxaudio', 9, False), 0.5605331044658215)], 30: [(('text', 0, False), 0.5097233809695847),  (('text', 2, False), 0.49231711337484896),  (('text', 4, False), 0.48320408189829933),  (('text', 9, False), 0.468027011298293),  (('video', 0, False), 0.5209934109081271),  (('video', 2, False), 0.5115276335305253),  (('video', 4, False), 0.5093340508907962),  (('video', 9, False), 0.5092296642443055),  (('voxaudio', 0, False), 0.649192874077763),  (('voxaudio', 2, False), 0.6433143207552103),  (('voxaudio', 4, False), 0.6395570342845079),  (('voxaudio', 9, False), 0.6346467734225352),  (('text_video', 0, False), 0.5379404026086512),  (('text_video', 2, False), 0.5189087985997364),  (('text_video', 4, False), 0.5111426970748989),  (('text_video', 9, False), 0.49890841304289174),  (('text_voxaudio', 0, False), 0.5481871716793691),  (('text_voxaudio', 2, False), 0.5393656017094565),  (('text_voxaudio', 4, False), 0.5356398449903715),  (('text_voxaudio', 9, False), 0.5369565384467397),  (('video_voxaudio', 0, False), 0.5966038331078354),  (('video_voxaudio', 2, False), 0.5878248521997583),  (('video_voxaudio', 4, False), 0.5843130805589004),  (('video_voxaudio', 9, False), 0.583813083854769),  (('text_video_voxaudio', 0, False), 0.5685035275890229),  (('text_video_voxaudio', 2, False), 0.5549023516640964),  (('text_video_voxaudio', 4, False), 0.5486326210610919),  (('text_video_voxaudio', 9, False), 0.544586521645875)]}
table = make_table(results)

Classifier: Naive Bayes
Task: predict binary label whether host is speaking
Input: Only hosts features


audio                 text                video               voxaudio               audio_text               audio_video               audio_voxaudio               text_video               text_voxaudio               video_voxaudio               audio_text_video               audio_text_voxaudio               audio_video_voxaudio               text_video_voxaudio               audio_text_video_voxaudio              
       1      5      10     1      5      10     1      5      10       1      5      10         1      5      10          1      5      10             1      5      10         1      5      10            1      5      10             1      5      10               1      5      10                  1      5      10                   1      5      10                  1      5      10                        1      5      10
4   0.640  0.643  0.594  0.582  0.514  0.501  0.537  0.492  0.496    0.674  0.564  0.539      0.651  0.641  0.594       0.619  0.609  0.562          0.683  0.643  0.597      0.579  0.511  0.498         0.676  0.570  0.536          0.637  0.529  0.514            0.637  0.609  0.566               0.689  0.637  0.594                0.662  0.608  0.567               0.648  0.541  0.515                     0.672  0.606  0.570
5   0.649  0.658  0.595  0.578  0.518  0.505  0.529  0.493  0.484    0.659  0.562  0.539      0.653  0.649  0.592       0.621  0.625  0.558          0.675  0.650  0.596      0.569  0.517  0.493         0.655  0.565  0.533          0.616  0.530  0.500            0.629  0.621  0.557               0.679  0.641  0.592                0.653  0.618  0.560               0.632  0.540  0.505                     0.661  0.612  0.562
10  0.661  0.612  0.548  0.600  0.519  0.503  0.524  0.487  0.498    0.623  0.552  0.525      0.656  0.606  0.544       0.647  0.586  0.523          0.669  0.612  0.553      0.569  0.505  0.508         0.620  0.544  0.525          0.597  0.518  0.512            0.651  0.585  0.526               0.661  0.602  0.544                0.659  0.590  0.526               0.612  0.525  0.517                     0.653  0.583  0.528

text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.663  0.589  0.557  0.551  0.533  0.523    0.701  0.638  0.621      0.639  0.581  0.555         0.695  0.620  0.595          0.664  0.604  0.575               0.686  0.616  0.587
5   0.673  0.609  0.586  0.546  0.528  0.516    0.698  0.641  0.626      0.623  0.574  0.543         0.692  0.632  0.608          0.658  0.600  0.570               0.681  0.618  0.586
10  0.616  0.518  0.428  0.531  0.514  0.506    0.675  0.645  0.635      0.568  0.531  0.506         0.638  0.595  0.568          0.631  0.593  0.575               0.633  0.589  0.561
30  0.510  0.483  0.468  0.521  0.509  0.509    0.649  0.640  0.635      0.538  0.511  0.499         0.548  0.536  0.537          0.597  0.584  0.584               0.569  0.549  0.545

In [166]:
print("Classifier: Random Forest")
print("Task: predict multi-class label (4 classes, 3 speakers and non-speaking)")
print("Input: All 3 speakers' features")

results = {4: [(('audio', 0), 0.5486435057423897), (('audio', 1), 0.5449273164516126), (('audio', 2), 0.5501687394642565), (('audio', 3), 0.5499053274117014), (('audio', 4), 0.5487224572264429), (('audio', 5), 0.543125942295134), (('audio', 6), 0.5266259631575922), (('audio', 7), 0.5189681852977663), (('audio', 8), 0.5067598534162746), (('audio', 9), 0.491218664309954), (('text', 0), 0.5410331861792538), (('text', 1), 0.4999294198727342), (('text', 2), 0.4734629962900777), (('text', 3), 0.4442006114899133), (('text', 4), 0.44054110089405657), (('text', 5), 0.4277741230687851), (('text', 6), 0.42141026319340663), (('text', 7), 0.43574539815416174), (('text', 8), 0.42354846037483584), (('text', 9), 0.41708100487807476), (('video', 0), 0.3512652215321326), (('video', 1), 0.326833961620719), (('video', 2), 0.3159737791942733), (('video', 3), 0.3143572095533367), (('video', 4), 0.3066621215206235), (('video', 5), 0.3123529270606596), (('video', 6), 0.30199038915693444), (('video', 7), 0.3025572746869881), (('video', 8), 0.3091379487219224), (('video', 9), 0.30425534376588087), (('voxaudio', 0), 0.5412169860273679), (('voxaudio', 1), 0.49225289149389234), (('voxaudio', 2), 0.45276350669763293), (('voxaudio', 3), 0.44143042102521407), (('voxaudio', 4), 0.42520663713995416), (('voxaudio', 5), 0.41606262070416455), (('voxaudio', 6), 0.4120800447048798), (('voxaudio', 7), 0.40581419092072757), (('voxaudio', 8), 0.40400012497220905), (('voxaudio', 9), 0.4002787471330964), (('audio_text', 0), 0.5842434197624843), (('audio_text', 1), 0.5473180245906409), (('audio_text', 2), 0.5441319887797473), (('audio_text', 3), 0.544949526481958), (('audio_text', 4), 0.5426439675450437), (('audio_text', 5), 0.5483159384421535), (('audio_text', 6), 0.5249574459291302), (('audio_text', 7), 0.5180295728096344), (('audio_text', 8), 0.504882973824647), (('audio_text', 9), 0.4832944849541517), (('audio_video', 0), 0.5025213948310042), (('audio_video', 1), 0.48820955561294976), (('audio_video', 2), 0.491569059181136), (('audio_video', 3), 0.4886754834983372), (('audio_video', 4), 0.4748749880750669), (('audio_video', 5), 0.47559030034094574), (('audio_video', 6), 0.44988393250459635), (('audio_video', 7), 0.42733263736072075), (('audio_video', 8), 0.4068232067762886), (('audio_video', 9), 0.3819844980282111), (('audio_voxaudio', 0), 0.5731752675242021), (('audio_voxaudio', 1), 0.5447616275129933), (('audio_voxaudio', 2), 0.5437599911524889), (('audio_voxaudio', 3), 0.5410921050786864), (('audio_voxaudio', 4), 0.5341481838202906), (('audio_voxaudio', 5), 0.5318088431640072), (('audio_voxaudio', 6), 0.5214201341221367), (('audio_voxaudio', 7), 0.506819351999462), (('audio_voxaudio', 8), 0.48929488222577955), (('audio_voxaudio', 9), 0.46707766738845446), (('text_video', 0), 0.46838936302322576), (('text_video', 1), 0.4063681435825889), (('text_video', 2), 0.371273404850715), (('text_video', 3), 0.3439710813648208), (('text_video', 4), 0.3434235490389067), (('text_video', 5), 0.33072082999521923), (('text_video', 6), 0.31931232078248684), (('text_video', 7), 0.31172036238675965), (('text_video', 8), 0.30988665278925454), (('text_video', 9), 0.3110399706656912), (('text_voxaudio', 0), 0.5513880821500002), (('text_voxaudio', 1), 0.4970009106322177), (('text_voxaudio', 2), 0.4633870551528374), (('text_voxaudio', 3), 0.43720433758638516), (('text_voxaudio', 4), 0.4290861220514338), (('text_voxaudio', 5), 0.4149590548091111), (('text_voxaudio', 6), 0.4184089141870798), (('text_voxaudio', 7), 0.40880798656615613), (('text_voxaudio', 8), 0.4071938470624941), (('text_voxaudio', 9), 0.4063164883557212), (('video_voxaudio', 0), 0.4870884445729523), (('video_voxaudio', 1), 0.41639067563888144), (('video_voxaudio', 2), 0.374021160297464), (('video_voxaudio', 3), 0.3597794227400244), (('video_voxaudio', 4), 0.34323631478580463), (('video_voxaudio', 5), 0.33572769857198326), (('video_voxaudio', 6), 0.32545827220408674), (('video_voxaudio', 7), 0.32014611129436454), (('video_voxaudio', 8), 0.32021952689773475), (('video_voxaudio', 9), 0.3148160509220816), (('audio_text_video', 0), 0.5448329170208651), (('audio_text_video', 1), 0.5026344309959621), (('audio_text_video', 2), 0.49486827597532085), (('audio_text_video', 3), 0.4947416652565093), (('audio_text_video', 4), 0.485209481189397), (('audio_text_video', 5), 0.47374119231002015), (('audio_text_video', 6), 0.45454047504396583), (('audio_text_video', 7), 0.43929859141543987), (('audio_text_video', 8), 0.4051318294547245), (('audio_text_video', 9), 0.37462546691479787), (('audio_text_voxaudio', 0), 0.5827966265601621), (('audio_text_voxaudio', 1), 0.5431289268113952), (('audio_text_voxaudio', 2), 0.5424789452694302), (('audio_text_voxaudio', 3), 0.5353913710061233), (('audio_text_voxaudio', 4), 0.5348785116806096), (('audio_text_voxaudio', 5), 0.5384842838218403), (('audio_text_voxaudio', 6), 0.517682200760056), (('audio_text_voxaudio', 7), 0.5058746174204771), (('audio_text_voxaudio', 8), 0.493799810150856), (('audio_text_voxaudio', 9), 0.4676396116408078), (('audio_video_voxaudio', 0), 0.5386614873297062), (('audio_video_voxaudio', 1), 0.5099273865786917), (('audio_video_voxaudio', 2), 0.4943186578593715), (('audio_video_voxaudio', 3), 0.49805286131184967), (('audio_video_voxaudio', 4), 0.485216817148786), (('audio_video_voxaudio', 5), 0.4670533811277816), (('audio_video_voxaudio', 6), 0.4571480791482808), (('audio_video_voxaudio', 7), 0.431997834861769), (('audio_video_voxaudio', 8), 0.4094528445813951), (('audio_video_voxaudio', 9), 0.37066480123246814), (('text_video_voxaudio', 0), 0.5057980701789289), (('text_video_voxaudio', 1), 0.43882246525513435), (('text_video_voxaudio', 2), 0.39050216328349907), (('text_video_voxaudio', 3), 0.36437923626188506), (('text_video_voxaudio', 4), 0.3550469854574648), (('text_video_voxaudio', 5), 0.3407376736217329), (('text_video_voxaudio', 6), 0.3366243687455131), (('text_video_voxaudio', 7), 0.3287453030228388), (('text_video_voxaudio', 8), 0.3256653631012234), (('text_video_voxaudio', 9), 0.31839508579128317), (('audio_text_video_voxaudio', 0), 0.5468385185787947), (('audio_text_video_voxaudio', 1), 0.5075675873648959), (('audio_text_video_voxaudio', 2), 0.4956030550724055), (('audio_text_video_voxaudio', 3), 0.48776025478642415), (('audio_text_video_voxaudio', 4), 0.4848483273809293), (('audio_text_video_voxaudio', 5), 0.4728128613710655), (('audio_text_video_voxaudio', 6), 0.4539785241730783), (('audio_text_video_voxaudio', 7), 0.42975781734497626), (('audio_text_video_voxaudio', 8), 0.41621217479658235), (('audio_text_video_voxaudio', 9), 0.37727018268973184)], 5: [(('audio', 0), 0.553573708144118), (('audio', 1), 0.5618853241909051), (('audio', 2), 0.5603947756052785), (('audio', 3), 0.5592513501914497), (('audio', 4), 0.5625958406830618), (('audio', 5), 0.5532604504968225), (('audio', 6), 0.5429160111105139), (('audio', 7), 0.5223539446083697), (('audio', 8), 0.5049938634690847), (('audio', 9), 0.4778322596364884), (('text', 0), 0.5205970040659583), (('text', 1), 0.47986872840486755), (('text', 2), 0.4533931535890696), (('text', 3), 0.43260411053965886), (('text', 4), 0.42240392783211567), (('text', 5), 0.42307826266266685), (('text', 6), 0.42021578986355823), (('text', 7), 0.4077695705850255), (('text', 8), 0.407650594206652), (('text', 9), 0.41207770712463915), (('video', 0), 0.33280588073758904), (('video', 1), 0.32018857933771017), (('video', 2), 0.32190277518733357), (('video', 3), 0.311292762875994), (('video', 4), 0.3025319174496956), (('video', 5), 0.30050791133571214), (('video', 6), 0.2935825108525803), (('video', 7), 0.29842948577214246), (('video', 8), 0.3023706856271677), (('video', 9), 0.3006461582902403), (('voxaudio', 0), 0.5176756313497697), (('voxaudio', 1), 0.4668620492042267), (('voxaudio', 2), 0.4429015178380505), (('voxaudio', 3), 0.42779170636640645), (('voxaudio', 4), 0.4143954343838535), (('voxaudio', 5), 0.3997773771900547), (('voxaudio', 6), 0.40057844785069124), (('voxaudio', 7), 0.39667459216971956), (('voxaudio', 8), 0.39084370847606664), (('voxaudio', 9), 0.3843931516974835), (('audio_text', 0), 0.5701705885007661), (('audio_text', 1), 0.5589622941864479), (('audio_text', 2), 0.5648163531093499), (('audio_text', 3), 0.5609198851488414), (('audio_text', 4), 0.5588862847908741), (('audio_text', 5), 0.5473102589556293), (('audio_text', 6), 0.5389897201773596), (('audio_text', 7), 0.5227293552963878), (('audio_text', 8), 0.4974438389496608), (('audio_text', 9), 0.4802974377979642), (('audio_video', 0), 0.5014394043134043), (('audio_video', 1), 0.5150070880628977), (('audio_video', 2), 0.5171767835092849), (('audio_video', 3), 0.5073713688005702), (('audio_video', 4), 0.5069338726572029), (('audio_video', 5), 0.49011871445200494), (('audio_video', 6), 0.45688164483343224), (('audio_video', 7), 0.43800644987730275), (('audio_video', 8), 0.3923698757296865), (('audio_video', 9), 0.36526436246191346), (('audio_voxaudio', 0), 0.5614350595532922), (('audio_voxaudio', 1), 0.5547473264765121), (('audio_voxaudio', 2), 0.5559865616127766), (('audio_voxaudio', 3), 0.5535324744431354), (('audio_voxaudio', 4), 0.552576068577713), (('audio_voxaudio', 5), 0.546370144339478), (('audio_voxaudio', 6), 0.5288906607704773), (('audio_voxaudio', 7), 0.5093889597417955), (('audio_voxaudio', 8), 0.4853765444789467), (('audio_voxaudio', 9), 0.4727157594454794), (('text_video', 0), 0.4360166834608739), (('text_video', 1), 0.39489961091058984), (('text_video', 2), 0.3586859493029249), (('text_video', 3), 0.35173927072718525), (('text_video', 4), 0.332576973146408), (('text_video', 5), 0.3247094122286861), (('text_video', 6), 0.3130406782399278), (('text_video', 7), 0.30425426095958924), (('text_video', 8), 0.29972240986330534), (('text_video', 9), 0.304244643818083), (('text_voxaudio', 0), 0.5174979579017192), (('text_voxaudio', 1), 0.4738234318289213), (('text_voxaudio', 2), 0.44842113229527475), (('text_voxaudio', 3), 0.4253957463185939), (('text_voxaudio', 4), 0.4116169742281768), (('text_voxaudio', 5), 0.4126385226202429), (('text_voxaudio', 6), 0.4058138313971232), (('text_voxaudio', 7), 0.3953596812102661), (('text_voxaudio', 8), 0.39462085672913555), (('text_voxaudio', 9), 0.3921636857642087), (('video_voxaudio', 0), 0.45770061156247577), (('video_voxaudio', 1), 0.3936138731655658), (('video_voxaudio', 2), 0.3649283433395952), (('video_voxaudio', 3), 0.3423161934628512), (('video_voxaudio', 4), 0.33239058213345296), (('video_voxaudio', 5), 0.3180061838632836), (('video_voxaudio', 6), 0.31080242003556136), (('video_voxaudio', 7), 0.31571689377236395), (('video_voxaudio', 8), 0.3110538546225121), (('video_voxaudio', 9), 0.30728069627158855), (('audio_text_video', 0), 0.5304384169572423), (('audio_text_video', 1), 0.5151934220095244), (('audio_text_video', 2), 0.5212251308013276), (('audio_text_video', 3), 0.5136490407559767), (('audio_text_video', 4), 0.5047123375856074), (('audio_text_video', 5), 0.48545765636858834), (('audio_text_video', 6), 0.46155961893917696), (('audio_text_video', 7), 0.43706730591243054), (('audio_text_video', 8), 0.3974633298890441), (('audio_text_video', 9), 0.3633716729089125), (('audio_text_voxaudio', 0), 0.5659874695002228), (('audio_text_voxaudio', 1), 0.5505363803625469), (('audio_text_voxaudio', 2), 0.5546943853517867), (('audio_text_voxaudio', 3), 0.5515031254553258), (('audio_text_voxaudio', 4), 0.5509092239151252), (('audio_text_voxaudio', 5), 0.5463751673058836), (('audio_text_voxaudio', 6), 0.5298198204621832), (('audio_text_voxaudio', 7), 0.5071393205684188), (('audio_text_voxaudio', 8), 0.4902850572134721), (('audio_text_voxaudio', 9), 0.45907166554621986), (('audio_video_voxaudio', 0), 0.5235037725263308), (('audio_video_voxaudio', 1), 0.5234324991873025), (('audio_video_voxaudio', 2), 0.523243126600967), (('audio_video_voxaudio', 3), 0.5156852089442581), (('audio_video_voxaudio', 4), 0.5030372462363814), (('audio_video_voxaudio', 5), 0.4945827852212418), (('audio_video_voxaudio', 6), 0.4604389929590559), (('audio_video_voxaudio', 7), 0.438003983215974), (('audio_video_voxaudio', 8), 0.3931263733769788), (('audio_video_voxaudio', 9), 0.3675444504162108), (('text_video_voxaudio', 0), 0.4828666550773583), (('text_video_voxaudio', 1), 0.41393985703226327), (('text_video_voxaudio', 2), 0.3708135693005795), (('text_video_voxaudio', 3), 0.3550551817361368), (('text_video_voxaudio', 4), 0.34110855792307804), (('text_video_voxaudio', 5), 0.3325400159308362), (('text_video_voxaudio', 6), 0.32126884138409195), (('text_video_voxaudio', 7), 0.31383737102182563), (('text_video_voxaudio', 8), 0.3080285760311286), (('text_video_voxaudio', 9), 0.3093625934355805), (('audio_text_video_voxaudio', 0), 0.5291643513103439), (('audio_text_video_voxaudio', 1), 0.5151967733394998), (('audio_text_video_voxaudio', 2), 0.5250901569213186), (('audio_text_video_voxaudio', 3), 0.5071936927750765), (('audio_text_video_voxaudio', 4), 0.5061955283286654), (('audio_text_video_voxaudio', 5), 0.4977511402079788), (('audio_text_video_voxaudio', 6), 0.4587600773830747), (('audio_text_video_voxaudio', 7), 0.43781636429948645), (('audio_text_video_voxaudio', 8), 0.3931253053803002), (('audio_text_video_voxaudio', 9), 0.3764536886905962)], 10: [(('audio', 0), 0.6003960777076633), (('audio', 1), 0.5887971988249554), (('audio', 2), 0.5710515064914705), (('audio', 3), 0.5465924286286205), (('audio', 4), 0.5195016741746425), (('audio', 5), 0.4872538003504895), (('audio', 6), 0.467324788947025), (('audio', 7), 0.45066712892822985), (('audio', 8), 0.4425252938276024), (('audio', 9), 0.4365546868253271), (('text', 0), 0.42318492894561566), (('text', 1), 0.40278556125195475), (('text', 2), 0.3902578724955418), (('text', 3), 0.37474743424734386), (('text', 4), 0.36930237843838293), (('text', 5), 0.3677412455985316), (('text', 6), 0.37388663443983344), (('text', 7), 0.3694584334092954), (('text', 8), 0.37521246926210494), (('text', 9), 0.3796034208900606), (('video', 0), 0.29709176884692895), (('video', 1), 0.29152751748785277), (('video', 2), 0.2877313794948749), (('video', 3), 0.2851336064096253), (('video', 4), 0.28708789668500156), (('video', 5), 0.28588279567996133), (('video', 6), 0.28807827908124495), (('video', 7), 0.2845913176080054), (('video', 8), 0.2876458044097948), (('video', 9), 0.28371291201418336), (('voxaudio', 0), 0.41648500929673615), (('voxaudio', 1), 0.3891380852119792), (('voxaudio', 2), 0.36904548907149387), (('voxaudio', 3), 0.3585105193373696), (('voxaudio', 4), 0.34959192498786895), (('voxaudio', 5), 0.33880191929762704), (('voxaudio', 6), 0.3387126653941964), (('voxaudio', 7), 0.33220115596332567), (('voxaudio', 8), 0.3310311543292427), (('voxaudio', 9), 0.3294231112512269), (('audio_text', 0), 0.5929510894670569), (('audio_text', 1), 0.5824383782622904), (('audio_text', 2), 0.5665571452620008), (('audio_text', 3), 0.5431865892320804), (('audio_text', 4), 0.5079416265909517), (('audio_text', 5), 0.4815434928122849), (('audio_text', 6), 0.4524232272837364), (('audio_text', 7), 0.435306872525228), (('audio_text', 8), 0.431521979443264), (('audio_text', 9), 0.4266726481780504), (('audio_video', 0), 0.5453494145984454), (('audio_video', 1), 0.5308331271512994), (('audio_video', 2), 0.48731318044738636), (('audio_video', 3), 0.4345232561128995), (('audio_video', 4), 0.3876696802113688), (('audio_video', 5), 0.35077226934411987), (('audio_video', 6), 0.32380107629047505), (('audio_video', 7), 0.3108902366784714), (('audio_video', 8), 0.3061427305622494), (('audio_video', 9), 0.30326723948376916), (('audio_voxaudio', 0), 0.5765941857785434), (('audio_voxaudio', 1), 0.5710371889195778), (('audio_voxaudio', 2), 0.541759925092707), (('audio_voxaudio', 3), 0.5196094920427283), (('audio_voxaudio', 4), 0.47611891052681743), (('audio_voxaudio', 5), 0.4358505412559472), (('audio_voxaudio', 6), 0.4088402464258656), (('audio_voxaudio', 7), 0.39344563669865135), (('audio_voxaudio', 8), 0.3784727915493146), (('audio_voxaudio', 9), 0.3753237214526465), (('text_video', 0), 0.3558725500535822), (('text_video', 1), 0.31900646818619355), (('text_video', 2), 0.3012486458131152), (('text_video', 3), 0.29853162625025814), (('text_video', 4), 0.29504245029377535), (('text_video', 5), 0.2906416267085087), (('text_video', 6), 0.28692283426653004), (('text_video', 7), 0.2890038799495785), (('text_video', 8), 0.28802668543203264), (('text_video', 9), 0.2897161657342455), (('text_voxaudio', 0), 0.42282081437202035), (('text_voxaudio', 1), 0.3900752756638771), (('text_voxaudio', 2), 0.37279959760737386), (('text_voxaudio', 3), 0.35965344523484744), (('text_voxaudio', 4), 0.35377133795349136), (('text_voxaudio', 5), 0.34851060090741576), (('text_voxaudio', 6), 0.3434898592243522), (('text_voxaudio', 7), 0.34371895531175556), (('text_voxaudio', 8), 0.3379796678450874), (('text_voxaudio', 9), 0.33814029515332705), (('video_voxaudio', 0), 0.35606326535402005), (('video_voxaudio', 1), 0.3300647395456249), (('video_voxaudio', 2), 0.316656886712668), (('video_voxaudio', 3), 0.3053217726413775), (('video_voxaudio', 4), 0.3013095186975813), (('video_voxaudio', 5), 0.29884878745447085), (('video_voxaudio', 6), 0.2964818038676141), (('video_voxaudio', 7), 0.29803568505830624), (('video_voxaudio', 8), 0.2980655003171656), (('video_voxaudio', 9), 0.2941776669036689), (('audio_text_video', 0), 0.5481323173391385), (('audio_text_video', 1), 0.5201766246391236), (('audio_text_video', 2), 0.5000771524440516), (('audio_text_video', 3), 0.44338318760936), (('audio_text_video', 4), 0.39070698624324796), (('audio_text_video', 5), 0.3505780010247237), (('audio_text_video', 6), 0.32494702512473417), (('audio_text_video', 7), 0.31587843567821716), (('audio_text_video', 8), 0.30729954027084283), (('audio_text_video', 9), 0.3052027832692401), (('audio_text_voxaudio', 0), 0.5784482569058552), (('audio_text_voxaudio', 1), 0.5678564718244757), (('audio_text_voxaudio', 2), 0.5438334127262572), (('audio_text_voxaudio', 3), 0.5188455234469193), (('audio_text_voxaudio', 4), 0.47441812786190257), (('audio_text_voxaudio', 5), 0.4333806687957322), (('audio_text_voxaudio', 6), 0.4088413330837978), (('audio_text_voxaudio', 7), 0.39152864041206636), (('audio_text_voxaudio', 8), 0.38405839835087524), (('audio_text_voxaudio', 9), 0.37591073958470955), (('audio_video_voxaudio', 0), 0.5421999354192414), (('audio_video_voxaudio', 1), 0.5207426266727028), (('audio_video_voxaudio', 2), 0.49725795921056826), (('audio_video_voxaudio', 3), 0.4367963030098253), (('audio_video_voxaudio', 4), 0.3874774108790821), (('audio_video_voxaudio', 5), 0.34353198998099715), (('audio_video_voxaudio', 6), 0.3280075891159875), (('audio_video_voxaudio', 7), 0.31377558426061686), (('audio_video_voxaudio', 8), 0.3157928373475052), (('audio_video_voxaudio', 9), 0.3057900024488839), (('text_video_voxaudio', 0), 0.3774465403983297), (('text_video_voxaudio', 1), 0.333416837187016), (('text_video_voxaudio', 2), 0.3153439175596184), (('text_video_voxaudio', 3), 0.3062660617159379), (('text_video_voxaudio', 4), 0.30300885736990396), (('text_video_voxaudio', 5), 0.30092085864074775), (('text_video_voxaudio', 6), 0.29495161684657967), (('text_video_voxaudio', 7), 0.30033692196403006), (('text_video_voxaudio', 8), 0.2957431258736777), (('text_video_voxaudio', 9), 0.2926315964585758), (('audio_text_video_voxaudio', 0), 0.5433136273363345), (('audio_text_video_voxaudio', 1), 0.5229808835884525), (('audio_text_video_voxaudio', 2), 0.4869404145739787), (('audio_text_video_voxaudio', 3), 0.43679150356147484), (('audio_text_video_voxaudio', 4), 0.38331971310491086), (('audio_text_video_voxaudio', 5), 0.3477297093268542), (('audio_text_video_voxaudio', 6), 0.3350704763568021), (('audio_text_video_voxaudio', 7), 0.3103147875782867), (('audio_text_video_voxaudio', 8), 0.31231531754183506), (('audio_text_video_voxaudio', 9), 0.3094738705522654)]}
table = make_table(results)

results = {4: [(('text', 0), 0.5740528419600442),  (('text', 2), 0.5156626643534831),  (('text', 4), 0.48387343094770285),  (('text', 9), 0.4613859495173531),  (('video', 0), 0.41415068773245284),  (('video', 2), 0.3992266253610506),  (('video', 4), 0.3906216111106834),  (('video', 9), 0.3892451360983997),  (('voxaudio', 0), 0.5639050403117057),  (('voxaudio', 2), 0.4938288690160813),  (('voxaudio', 4), 0.4626355693472526),  (('voxaudio', 9), 0.44266333362986215),  (('text_video', 0), 0.541890730708897),  (('text_video', 2), 0.4803464464808475),  (('text_video', 4), 0.447162344044209),  (('text_video', 9), 0.4141139583905938),  (('text_voxaudio', 0), 0.5712240932576342),  (('text_voxaudio', 2), 0.5028847111109787),  (('text_voxaudio', 4), 0.47296683123176936),  (('text_voxaudio', 9), 0.4463396387659516),  (('video_voxaudio', 0), 0.5499653149032067),  (('video_voxaudio', 2), 0.4689750642020507),  (('video_voxaudio', 4), 0.4426494628695384),  (('video_voxaudio', 9), 0.4171091213334928),  (('text_video_voxaudio', 0), 0.5546101154206862),  (('text_video_voxaudio', 2), 0.4869828697159552),  (('text_video_voxaudio', 4), 0.45345049757742456),  (('text_video_voxaudio', 9), 0.4218619382456967)], 5: [(('text', 0), 0.5642855068977534),  (('text', 2), 0.5105969754959487),  (('text', 4), 0.4884425045155233),  (('text', 9), 0.4630873548222157),  (('video', 0), 0.4100915807103857),  (('video', 2), 0.3972014958031277),  (('video', 4), 0.3950084703335084),  (('video', 9), 0.39084436888573226),  (('voxaudio', 0), 0.5492132025887548),  (('voxaudio', 2), 0.4936037423723748),  (('voxaudio', 4), 0.46519085893597306),  (('voxaudio', 9), 0.4399030379018402),  (('text_video', 0), 0.5210516511820029),  (('text_video', 2), 0.46735792479665045),  (('text_video', 4), 0.44131648475297),  (('text_video', 9), 0.41111290389996913),  (('text_voxaudio', 0), 0.5546093128030721),  (('text_voxaudio', 2), 0.49957548901787857),  (('text_voxaudio', 4), 0.4736523428889766),  (('text_voxaudio', 9), 0.4425924195252728),  (('video_voxaudio', 0), 0.5217788732145345),  (('video_voxaudio', 2), 0.45813264812282606),  (('video_voxaudio', 4), 0.42936847418131896),  (('video_voxaudio', 9), 0.4044890844921548),  (('text_video_voxaudio', 0), 0.5338591600446756),  (('text_video_voxaudio', 2), 0.4723483446764827),  (('text_video_voxaudio', 4), 0.44216084080676055),  (('text_video_voxaudio', 9), 0.41300974983107286)], 10: [(('text', 0), 0.5184983830325602),  (('text', 2), 0.48606708868073206),  (('text', 4), 0.4712063938262623),  (('text', 9), 0.4492910629942187),  (('video', 0), 0.40276333475706483),  (('video', 2), 0.392534621426603),  (('video', 4), 0.3893416988932283),  (('video', 9), 0.38531745736233103),  (('voxaudio', 0), 0.49233502303217297),  (('voxaudio', 2), 0.4581515724499313),  (('voxaudio', 4), 0.44264658119392275),  (('voxaudio', 9), 0.42276459191814464),  (('text_video', 0), 0.4815732870206565),  (('text_video', 2), 0.44515825701874545),  (('text_video', 4), 0.4258581412009034),  (('text_video', 9), 0.3981828445910857),  (('text_voxaudio', 0), 0.5032333364895052),  (('text_voxaudio', 2), 0.46336999418003055),  (('text_voxaudio', 4), 0.44548085504375967),  (('text_voxaudio', 9), 0.424986249827346),  (('video_voxaudio', 0), 0.47438791280736636),  (('video_voxaudio', 2), 0.4373850074333886),  (('video_voxaudio', 4), 0.42046317608332684),  (('video_voxaudio', 9), 0.3955505558755499),  (('text_video_voxaudio', 0), 0.48921725345957007),  (('text_video_voxaudio', 2), 0.44665345763902975),  (('text_video_voxaudio', 4), 0.4263489758987701),  (('text_video_voxaudio', 9), 0.4010367493297329)]}
results.update({30: [(('text', 0), 0.4285386717514575), (('text', 2), 0.4150077213667685), (('text', 4), 0.41092215055619674), (('text', 9), 0.40645542450328465), (('video', 0), 0.3757522672147894), (('video', 2), 0.3715668180650328), (('video', 4), 0.3705681763865937), (('video', 9), 0.3661444836380083), (('voxaudio', 0), 0.3828074489430634), (('voxaudio', 2), 0.36696530316096965), (('voxaudio', 4), 0.36136416967878987), (('voxaudio', 9), 0.3506637324569154), (('text_video', 0), 0.40325202757374007), (('text_video', 2), 0.38651737462329144), (('text_video', 4), 0.3830271207876487), (('text_video', 9), 0.37283478238233836), (('text_voxaudio', 0), 0.3898897000650714), (('text_voxaudio', 2), 0.3715692149246127), (('text_voxaudio', 4), 0.3679831296350694), (('text_voxaudio', 9), 0.35833371378790657), (('video_voxaudio', 0), 0.384024523171711), (('video_voxaudio', 2), 0.3705981102781093), (('video_voxaudio', 4), 0.364478582872021), (('video_voxaudio', 9), 0.35472252275510774), (('text_video_voxaudio', 0), 0.3872336506927013), (('text_video_voxaudio', 2), 0.3728168734520659), (('text_video_voxaudio', 4), 0.3656188314122105), (('text_video_voxaudio', 9), 0.35578605871701124)]})
table = make_table(results)

results = {4: [(('all', 0), 0.5419320728830777), (('all', 1), 0.5006277946932102), (('all', 2), 0.48754176594981724), (('all', 3), 0.48830648324101456), (('all', 4), 0.484114587556427), (('all', 5), 0.4724428316960821), (('all', 6), 0.45825705939036093), (('all', 7), 0.43592988706343894), (('all', 8), 0.4100156986817386), (('all', 9), 0.3750056737323547), (('text_audio', 0), 0.5838852258012114), (('text_audio', 1), 0.5495063163757844), (('text_audio', 2), 0.5454106887318233), (('text_audio', 3), 0.5491813618768685), (('text_audio', 4), 0.5455883166000051), (('text_audio', 5), 0.5438725285446646), (('text_audio', 6), 0.5275631329025511), (('text_audio', 7), 0.5197249493090895), (('text_audio', 8), 0.5014996926690323), (('text_audio', 9), 0.48272862471397715), (('text_video', 0), 0.4709448787732695), (('text_video', 1), 0.40655078320240967), (('text_video', 2), 0.3817114571917865), (('text_video', 3), 0.3439665119425971), (('text_video', 4), 0.34601015530400486), (('text_video', 5), 0.33127758183840095), (('text_video', 6), 0.3329081237509691), (('text_video', 7), 0.3218330890241661), (('text_video', 8), 0.30951177358082205), (('text_video', 9), 0.3084016628708285), (('video_audio', 0), 0.501068166598183), (('video_audio', 1), 0.4909524328321255), (('video_audio', 2), 0.48955541256547663), (('video_audio', 3), 0.4835275806767272), (('video_audio', 4), 0.482074828531937), (('video_audio', 5), 0.4657512479252916), (('video_audio', 6), 0.4476406171252304), (('video_audio', 7), 0.42918797182631463), (('video_audio', 8), 0.40231192791852005), (('video_audio', 9), 0.37726804669637476), (('text', 0), 0.5426637801271839), (('text', 1), 0.4930009251116757), (('text', 2), 0.4688827336128343), (('text', 3), 0.44144190255244264), (('text', 4), 0.4346319158697698), (('text', 5), 0.4257413878512712), (('text', 6), 0.4305409443847393), (('text', 7), 0.42901397368937266), (('text', 8), 0.42335925165555055), (('text', 9), 0.4206678717229189), (('audio', 0), 0.556816915304857), (('audio', 1), 0.5467533074888621), (('audio', 2), 0.5443081011699492), (('audio', 3), 0.5534057598007454), (('audio', 4), 0.5485400072878294), (('audio', 5), 0.54164308557131), (('audio', 6), 0.5260742382223688), (('audio', 7), 0.5206555078174034), (('audio', 8), 0.5069467643555947), (('audio', 9), 0.4895217955872157), (('video', 0), 0.34272817381104137), (('video', 1), 0.32099736707516974), (('video', 2), 0.3209179963068344), (('video', 3), 0.31380184816390094), (('video', 4), 0.3164620551933006), (('video', 5), 0.3066096023645683), (('video', 6), 0.31055772590230857), (('video', 7), 0.30742392105800687), (('video', 8), 0.3066943393682261), (('video', 9), 0.3119842577289585)], 5: [(('all', 0), 0.5251484295675107), (('all', 1), 0.51574220229298), (('all', 2), 0.5151542142447324), (('all', 3), 0.509585563193764), (('all', 4), 0.5091713814679075), (('all', 5), 0.49309510248894134), (('all', 6), 0.46343438223767447), (('all', 7), 0.44063573501838516), (('all', 8), 0.3982166235463008), (('all', 9), 0.3658364862494018), (('text_audio', 0), 0.5694418207698332), (('text_audio', 1), 0.5569442908418205), (('text_audio', 2), 0.5602146684768847), (('text_audio', 3), 0.5561139632347479), (('text_audio', 4), 0.5599982562669522), (('text_audio', 5), 0.5584813881518444), (('text_audio', 6), 0.5419704012888363), (('text_audio', 7), 0.5206602764607791), (('text_audio', 8), 0.49858588339790255), (('text_audio', 9), 0.4808674131219351), (('text_video', 0), 0.440199640376295), (('text_video', 1), 0.38171430582226556), (('text_video', 2), 0.3581392045002832), (('text_video', 3), 0.3439768813754107), (('text_video', 4), 0.33646603239095824), (('text_video', 5), 0.3256396422073453), (('text_video', 6), 0.3147294165992499), (('text_video', 7), 0.30463019749781395), (('text_video', 8), 0.3065170047321153), (('text_video', 9), 0.3036748456347475), (('video_audio', 0), 0.5047217572968327), (('video_audio', 1), 0.5080483865021834), (('video_audio', 2), 0.5164467253111061), (('video_audio', 3), 0.5101457213497927), (('video_audio', 4), 0.5061876309382395), (('video_audio', 5), 0.4942129512674202), (('video_audio', 6), 0.4617423256209123), (('video_audio', 7), 0.43293761052209045), (('video_audio', 8), 0.39765503529284024), (('video_audio', 9), 0.3663978493466685), (('text', 0), 0.515321150071647), (('text', 1), 0.4754768104722359), (('text', 2), 0.4513687209930012), (('text', 3), 0.4359142150632353), (('text', 4), 0.4292654159382141), (('text', 5), 0.4200980519833187), (('text', 6), 0.4235839555813555), (('text', 7), 0.4145322539231307), (('text', 8), 0.40934532693603326), (('text', 9), 0.40961324602057925), (('audio', 0), 0.5475516282126931), (('audio', 1), 0.5587722737768483), (('audio', 2), 0.5616806966210864), (('audio', 3), 0.5616493612000882), (('audio', 4), 0.5629636290015069), (('audio', 5), 0.552887843930815), (('audio', 6), 0.5369261169715631), (('audio', 7), 0.5219767696068117), (('audio', 8), 0.501222767197194), (('audio', 9), 0.48522544777217647), (('video', 0), 0.3315351626297283), (('video', 1), 0.31963611259128183), (('video', 2), 0.3150977473609611), (('video', 3), 0.30926341010586855), (('video', 4), 0.3021618887352779), (('video', 5), 0.30571409619558915), (('video', 6), 0.3010707058774423), (('video', 7), 0.29805248859272615), (('video', 8), 0.2953883013423828), (('video', 9), 0.2972349083750159)], 10: [(('all', 0), 0.5473988608137793), (('all', 1), 0.5233608364809171), (('all', 2), 0.49989711002309756), (('all', 3), 0.4484857132037822), (('all', 4), 0.38766501505688994), (('all', 5), 0.3507711860750379), (('all', 6), 0.3287619941970811), (('all', 7), 0.30954593652228113), (('all', 8), 0.30671983299547545), (('all', 9), 0.30404356183916925), (('text_audio', 0), 0.5910980535226205), (('text_audio', 1), 0.5777729905424998), (('text_audio', 2), 0.5648617174284498), (('text_audio', 3), 0.5433791861415787), (('text_audio', 4), 0.5028295987750193), (('text_audio', 5), 0.4718355261387675), (('text_audio', 6), 0.4547114798463747), (('text_audio', 7), 0.4358847121950225), (('text_audio', 8), 0.42554280817005685), (('text_audio', 9), 0.42647246597526395), (('text_video', 0), 0.3527052509180426), (('text_video', 1), 0.3220086733754689), (('text_video', 2), 0.3072596292581603), (('text_video', 3), 0.29813754251540664), (('text_video', 4), 0.29201070565830095), (('text_video', 5), 0.2841618136121151), (('text_video', 6), 0.29533165054534966), (('text_video', 7), 0.29073169910349617), (('text_video', 8), 0.2887914479551011), (('text_video', 9), 0.28390464314617697), (('video_audio', 0), 0.5464669072332123), (('video_audio', 1), 0.5259752250367996), (('video_audio', 2), 0.4880560736591466), (('video_audio', 3), 0.44622386971269173), (('video_audio', 4), 0.3865306247882496), (('video_audio', 5), 0.349241764503356), (('video_audio', 6), 0.32914988935322265), (('video_audio', 7), 0.3131940559503916), (('video_audio', 8), 0.3042081326892472), (('video_audio', 9), 0.30094334219420815), (('text', 0), 0.4257906217238069), (('text', 1), 0.40297404689755156), (('text', 2), 0.38669385086017566), (('text', 3), 0.37361126428365743), (('text', 4), 0.3675997992057344), (('text', 5), 0.36698131993367517), (('text', 6), 0.3671900856690452), (('text', 7), 0.36984415541864457), (('text', 8), 0.3700092316990572), (('text', 9), 0.3737890783679872), (('audio', 0), 0.6005797061084728), (('audio', 1), 0.5867390271434588), (('audio', 2), 0.5658012104222551), (('audio', 3), 0.5490376180862281), (('audio', 4), 0.5229123879105219), (('audio', 5), 0.48839321922806744), (('audio', 6), 0.46713431344459144), (('audio', 7), 0.4523921792176127), (('audio', 8), 0.4465745159125728), (('audio', 9), 0.4386870369573064), (('video', 0), 0.29654106556577), (('video', 1), 0.2889126050324423), (('video', 2), 0.2914935973201923), (('video', 3), 0.28720284997427775), (('video', 4), 0.28368131260458085), (('video', 5), 0.2906523136459212), (('video', 6), 0.28367728006013926), (('video', 7), 0.28305507145461306), (('video', 8), 0.28436468428705236), (('video', 9), 0.27867433266039965)], 30: [(('all', 0), 0.2726295132299594), (('all', 1), 0.2638429651626062), (('all', 2), 0.2564969137490317), (('all', 3), 0.2649850023485764), (('all', 4), 0.2593079812841709), (('all', 5), 0.2657161400873646), (('all', 6), 0.2605923334363217), (('all', 7), 0.26744269632437423), (('all', 8), 0.25548060896898106), (('all', 9), 0.2702235609694955), (('text_audio', 0), 0.32189931398299965), (('text_audio', 1), 0.3213939218942815), (('text_audio', 2), 0.3157551313337704), (('text_audio', 3), 0.32349927222352176), (('text_audio', 4), 0.3151724923703052), (('text_audio', 5), 0.31285838092187196), (('text_audio', 6), 0.31024825140217993), (('text_audio', 7), 0.3164539389125244), (('text_audio', 8), 0.31069230511090973), (('text_audio', 9), 0.3120733887999266), (('text_video', 0), 0.2671402782841791), (('text_video', 1), 0.2644587675465058), (('text_video', 2), 0.2554704372662159), (('text_video', 3), 0.25919987352795343), (('text_video', 4), 0.259526235161245), (('text_video', 5), 0.2521557662490207), (('text_video', 6), 0.2545185918309859), (('text_video', 7), 0.2569201200913641), (('text_video', 8), 0.248293790619372), (('text_video', 9), 0.2506768775469654), (('video_audio', 0), 0.26529067255272804), (('video_audio', 1), 0.2579099560505146), (('video_audio', 2), 0.2632836567182808), (('video_audio', 3), 0.26168560375202754), (('video_audio', 4), 0.26034877362096503), (('video_audio', 5), 0.26341836890351866), (('video_audio', 6), 0.2612230130798234), (('video_audio', 7), 0.26280177394306825), (('video_audio', 8), 0.2641574102039218), (('video_audio', 9), 0.26533355684725735), (('text', 0), 0.2777278030944359), (('text', 1), 0.26160165724541906), (('text', 2), 0.26885360667813596), (('text', 3), 0.2683149066382432), (('text', 4), 0.26617773304863696), (('text', 5), 0.26050389915479266), (('text', 6), 0.269188390423421), (('text', 7), 0.26996235873678076), (('text', 8), 0.2660670939740707), (('text', 9), 0.26129935710463437), (('audio', 0), 0.42351862793048134), (('audio', 1), 0.4166343173921835), (('audio', 2), 0.4159393578865623), (('audio', 3), 0.4140161697226875), (('audio', 4), 0.4106918049776855), (('audio', 5), 0.41507173148187493), (('audio', 6), 0.41166664261284003), (('audio', 7), 0.4117928214745884), (('audio', 8), 0.40713894202266293), (('audio', 9), 0.4089491754011597), (('video', 0), 0.26408092229356306), (('video', 1), 0.2593488023158782), (('video', 2), 0.25670373634124155), (('video', 3), 0.259422904504396), (('video', 4), 0.2605683228264374), (('video', 5), 0.2486197628091348), (('video', 6), 0.24781898535930558), (('video', 7), 0.24935644939855744), (('video', 8), 0.25274203832343367), (('video', 9), 0.2504729186910014)], 60: [(('all', 0), 0.27308272836979147), (('all', 1), 0.2707369077306733), (('all', 2), 0.258432334271191), (('all', 3), 0.25918144714361313), (('all', 4), 0.26043996590041163), (('all', 5), 0.25196591023039355), (('all', 6), 0.2521855148604247), (('all', 7), 0.2547106316594713), (('all', 8), 0.24816770344452266), (('all', 9), 0.2499907931963737), (('text_audio', 0), 0.26138625946292543), (('text_audio', 1), 0.2744426433915212), (('text_audio', 2), 0.27368819729211835), (('text_audio', 3), 0.2674651627473039), (('text_audio', 4), 0.27814352847013624), (('text_audio', 5), 0.2692720196766124), (('text_audio', 6), 0.2657101599252149), (('text_audio', 7), 0.27215902231941913), (('text_audio', 8), 0.26929938662271746), (('text_audio', 9), 0.26812816317503724), (('text_video', 0), 0.2539824957194706), (('text_video', 1), 0.26002057356608477), (('text_video', 2), 0.2531826707470622), (('text_video', 3), 0.24911850937255037), (('text_video', 4), 0.25234381935631317), (('text_video', 5), 0.24484022382347467), (('text_video', 6), 0.24964102634512758), (('text_video', 7), 0.23878318275074134), (('text_video', 8), 0.23423687710859503), (('text_video', 9), 0.24766827857733284), (('video_audio', 0), 0.28051117492317645), (('video_audio', 1), 0.28719014962593514), (('video_audio', 2), 0.2822243410371515), (('video_audio', 3), 0.2770351707651557), (('video_audio', 4), 0.2801631098733707), (('video_audio', 5), 0.2702500678093385), (('video_audio', 6), 0.2802738791325076), (('video_audio', 7), 0.2610958299904315), (('video_audio', 8), 0.26105930601054383), (('video_audio', 9), 0.27486966163142046), (('text', 0), 0.231612032948814), (('text', 1), 0.23529925187032416), (('text', 2), 0.23261615233689917), (('text', 3), 0.23574545244861564), (('text', 4), 0.24398968712790028), (('text', 5), 0.23568668844187518), (('text', 6), 0.23585730444424194), (('text', 7), 0.22389021370157547), (('text', 8), 0.23138417680353163), (('text', 9), 0.2318342171393521), (('audio', 0), 0.4081237407349647), (('audio', 1), 0.4144077306733167), (('audio', 2), 0.4178766070480255), (('audio', 3), 0.41333003017274506), (('audio', 4), 0.4079904553442216), (('audio', 5), 0.4066263924886881), (('audio', 6), 0.40017083127622605), (('audio', 7), 0.4031479438166864), (('audio', 8), 0.40993138889613007), (('audio', 9), 0.40273609027747054), (('video', 0), 0.2668712277298719), (('video', 1), 0.26748316708229425), (('video', 2), 0.2662027524290566), (('video', 3), 0.25614227729748656), (('video', 4), 0.25409802944076076), (('video', 5), 0.25041557709691287), (('video', 6), 0.24962177573119285), (('video', 7), 0.24725590391001884), (('video', 8), 0.244547147082781), (('video', 9), 0.2528354930352874)]}
table = make_table(results)

Classifier: Random Forest
Task: predict multi-class label (4 classes, 3 speakers and non-speaking)
Input: All 3 speakers' features


audio                 text                video               voxaudio               audio_text               audio_video               audio_voxaudio               text_video               text_voxaudio               video_voxaudio               audio_text_video               audio_text_voxaudio               audio_video_voxaudio               text_video_voxaudio               audio_text_video_voxaudio              
       1      5      10     1      5      10     1      5      10       1      5      10         1      5      10          1      5      10             1      5      10         1      5      10            1      5      10             1      5      10               1      5      10                  1      5      10                   1      5      10                  1      5      10                        1      5      10
4   0.549  0.549  0.491  0.541  0.441  0.417  0.351  0.307  0.304    0.541  0.425  0.400      0.584  0.543  0.483       0.503  0.475  0.382          0.573  0.534  0.467      0.468  0.343  0.311         0.551  0.429  0.406          0.487  0.343  0.315            0.545  0.485  0.375               0.583  0.535  0.468                0.539  0.485  0.371               0.506  0.355  0.318                     0.547  0.485  0.377
5   0.554  0.563  0.478  0.521  0.422  0.412  0.333  0.303  0.301    0.518  0.414  0.384      0.570  0.559  0.480       0.501  0.507  0.365          0.561  0.553  0.473      0.436  0.333  0.304         0.517  0.412  0.392          0.458  0.332  0.307            0.530  0.505  0.363               0.566  0.551  0.459                0.524  0.503  0.368               0.483  0.341  0.309                     0.529  0.506  0.376
10  0.600  0.520  0.437  0.423  0.369  0.380  0.297  0.287  0.284    0.416  0.350  0.329      0.593  0.508  0.427       0.545  0.388  0.303          0.577  0.476  0.375      0.356  0.295  0.290         0.423  0.354  0.338          0.356  0.301  0.294            0.548  0.391  0.305               0.578  0.474  0.376                0.542  0.387  0.306               0.377  0.303  0.293                     0.543  0.383  0.309

text                video               voxaudio               text_video               text_voxaudio               video_voxaudio               text_video_voxaudio              
       1      5      10     1      5      10       1      5      10         1      5      10            1      5      10             1      5      10                  1      5      10
4   0.574  0.484  0.461  0.414  0.391  0.389    0.564  0.463  0.443      0.542  0.447  0.414         0.571  0.473  0.446          0.550  0.443  0.417               0.555  0.453  0.422
5   0.564  0.488  0.463  0.410  0.395  0.391    0.549  0.465  0.440      0.521  0.441  0.411         0.555  0.474  0.443          0.522  0.429  0.404               0.534  0.442  0.413
10  0.518  0.471  0.449  0.403  0.389  0.385    0.492  0.443  0.423      0.482  0.426  0.398         0.503  0.445  0.425          0.474  0.420  0.396               0.489  0.426  0.401
30  0.429  0.411  0.406  0.376  0.371  0.366    0.383  0.361  0.351      0.403  0.383  0.373         0.390  0.368  0.358          0.384  0.364  0.355               0.387  0.366  0.356

all               text_audio               text_video               video_audio                 text                audio                video              
       1      5      10         1      5      10         1      5      10          1      5      10     1      5      10     1      5      10     1      5      10
4   0.542  0.484  0.375      0.584  0.546  0.483      0.471  0.346  0.308       0.501  0.482  0.377  0.543  0.435  0.421  0.557  0.549  0.490  0.343  0.316  0.312
5   0.525  0.509  0.366      0.569  0.560  0.481      0.440  0.336  0.304       0.505  0.506  0.366  0.515  0.429  0.410  0.548  0.563  0.485  0.332  0.302  0.297
10  0.547  0.388  0.304      0.591  0.503  0.426      0.353  0.292  0.284       0.546  0.387  0.301  0.426  0.368  0.374  0.601  0.523  0.439  0.297  0.284  0.279
30  0.273  0.259  0.270      0.322  0.315  0.312      0.267  0.260  0.251       0.265  0.260  0.265  0.278  0.266  0.261  0.424  0.411  0.409  0.264  0.261  0.250
60  0.273  0.260  0.250      0.261  0.278  0.268      0.254  0.252  0.248       0.281  0.280  0.275  0.232  0.244  0.232  0.408  0.408  0.403  0.267  0.254  0.253

# Baselines

In [78]:
egocom_loc = "/datasets/cgn/EGOCOM/egocom_features/no_audio/"
cols = ['video_id', 'clip_id', 'video_speaker_id', 'is_speaking', 'multiclass_speaker_label']
egocom_features = pd.read_csv(egocom_loc + "egocom_features_history_4sec.csv.gz")[cols]
host_baseline = egocom_features[egocom_features['video_speaker_id'] == 1]['is_speaking'].mean()
print("prob(host is speaking) from fraction of labels: {:.1%}".format(host_baseline))

prob(host is speaking) from fraction of labels: 49.3%


55      56
62      63
85      86
88      89
166    167
172    173
Name: video_id, dtype: int64

In [93]:
videos_human = [
    'vid_173__day_6__con_6__person_1',
    'vid_063__day_1__con_5__person_2_part4',
    'vid_056__day_1__con_5__person_1_part2',
    'vid_089__day_2__con_3__person_2',
    'vid_167__day_6__con_4__person_1',
    'vid_086__day_2__con_2__person_2_part3',
]

In [101]:
lol = egocom_features[egocom_features.video_id.isin(video_info[video_info['video_name'].isin(videos_human)]['video_id'])]

In [102]:
print("prob(anyone is speaking) from fraction of labels: {:.1%}".format(lol['is_speaking'].mean()))

prob(anyone is speaking) from fraction of labels: 36.7%


In [103]:
dist_speaking = lol['multiclass_speaker_label'].value_counts() / len(lol)
nonhost = dist_speaking[2] + dist_speaking[3]
host = dist_speaking[1]
noone = dist_speaking[-1]
print("Baseline distribution over speaker labels used during training:")
print("p(Host is speaking): {:.1%}".format(host))
print("p(either non-host is speaking): {:.1%}".format(nonhost))
print("p(No one is speaking): {:.1%}".format(noone))

Baseline distribution over speaker labels used during training:
p(Host is speaking): 42.3%
p(either non-host is speaking): 52.5%
p(No one is speaking): 5.2%


# other info used in the paper

In [5]:
video_info = pd.read_csv('/datasets/cgn/EGOCOM/video_info.csv')

In [9]:
video_info[video_info['speaker_is_host']]['word_count'].sum() / video_info['word_count'].sum()

0.5144186338918052

In [882]:
len(list(" ".join(video_info['tokenized_words']).split()))

159064

In [889]:
sum(pd.Series(" ".join(video_info['tokenized_words']).split()).value_counts() < 2)

3044

In [892]:
video_info.groupby(['train', 'test', 'val'])['duration_seconds'].sum()

train  test   val  
False  False  True       8661
       True   False     20993
True   False  False    109125
Name: duration_seconds, dtype: int64

In [893]:
video_info.groupby(['train', 'test', 'val'])['duration_seconds'].sum() / sum(video_info.groupby(['train', 'test', 'val'])['duration_seconds'].sum())

train  test   val  
False  False  True     0.062409
       True   False    0.151269
True   False  False    0.786322
Name: duration_seconds, dtype: float64

In [91]:
gt_transcripts.groupby('conversation_id')['endTime'].max().sum() / 3600 * 3

41.064175

In [896]:
video_info['duration_seconds'].sum() / 3600

38.54972222222222

In [898]:
video_info['word_count'].sum()

159065